# Fix sample table bam entries
## Author: Jonn Smith
## Date: 2023 August 1

This notebook processes raw bam files and modifes the `sample` table to add them into the `input_bam` column for the appropriate sample to which they belong.


In [142]:
import os
import json
import math
import time

import firecloud.api as fapi
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

import lrmaCU.terra.table_utils as ltu

bucket = os.environ['WORKSPACE_BUCKET']
workspace = os.environ['WORKSPACE_NAME']
namespace = os.environ['WORKSPACE_NAMESPACE']

flowcell_table = "sample"
sample_table = "sample_set"

#################################################################

print(f"Namespace: {namespace}")
print(f"Workspace: {workspace}")
print(f"Bucket:    {bucket}")
print()
print(f"Flowcell table: {flowcell_table}")
print(f"Sample table:   {sample_table}")

Namespace: broad-firecloud-dsde-methods
Workspace: sr-malaria
Bucket:    gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd

Flowcell table: sample
Sample table:   sample_set


In [143]:
def fix_nans(df, quiet=True):
    if not quiet: print("Replacing all `nan` values with empty strings: ")
    for c in df.columns.values:
        nan_types = ("nan", float('nan'))
        has_nan = False
        num_denaned = 0
        for n in nan_types:
            if (sum(df[c] == n) > 0):
                num_denaned += sum(df[c] == n)
                df.loc[df[c] == n, c] = ""
                has_nan = True
        if has_nan and not quiet:
            print(f"\t{c}: {num_denaned}")

    if not quiet: print("Replacing numpy nan values...")
    if not quiet: print("Done.")
    return df.replace(np.nan, "")

In [72]:
t = ltu.fetch_existing_root_table(namespace, workspace, "sample")
t = fix_nans(t)

***
## Test rename

In [26]:
df_to_upload = t.iloc[0:2]
df_to_upload = df_to_upload.rename(columns={"sample": "entity:testing_rename_samples_id"})
df_to_upload

,entity:testing_rename_samples_id,ENA,additional_metadata,admin_level_1,admin_level_1_latitude,admin_level_1_longitude,aligned_bai,aligned_bam,aligned_est_fold_cov,aligned_frac_bases,aligned_num_bases,aligned_num_reads,aligned_read_length,average_identity,country,country_latitude,country_longitude,dataset_provenance,exclusion_reason,fastqc_report,fq1,fq2,fq_unpaired,in_pf6,input_bai,input_bam,input_fq_end1,input_fq_end2,insert_size_average,insert_size_standard_deviation,num_bases,num_reads,num_reads_Q10,num_reads_Q12,num_reads_Q15,num_reads_Q5,num_reads_Q7,participant,pct_callable,pct_properly_paired_reads,population,qc_pass,raw_est_fold_cov,read_length,read_qual_mean,read_qual_median,sample_type,study,unaligned_bai,unaligned_bam,year
0,FP0008-C,ERR1081237,,Hodh el Gharbi,16.565426,-9.832345,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,33.385324,0.884333,778974400.0,8916670.0,100.0,99.283686,Mauritania,20.265149,-10.337093,PF7,Analysis_set,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://broad-dsp-lrma-pf7/inputs/ERR1081237/ERR1...,gs://broad-dsp-lrma-pf7/inputs/ERR1081237/ERR1...,,True,,,gs://broad-dsp-lrma-pf7/inputs/ERR1081237/ERR1...,gs://broad-dsp-lrma-pf7/inputs/ERR1081237/ERR1...,231.6,81.5,891667000.0,8916670.0,8914561.0,8909609.0,8897109.0,8915736.0,8915564.0,FP0008-C,82.16,83.6,AF-W,True,38.21511,100.0,28.465702,29.2,gDNA,1147-PF-MR-CONWAY,,,2014.0
1,FP0009-C,ERR1081238,,Hodh el Gharbi,16.565426,-9.832345,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,115.771497,0.971838,2700000000.0,28198096.0,100.0,99.268062,Mauritania,20.265149,-10.337093,PF7,Analysis_set,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://broad-dsp-lrma-pf7/inputs/ERR1081238/ERR1...,gs://broad-dsp-lrma-pf7/inputs/ERR1081238/ERR1...,,True,,,gs://broad-dsp-lrma-pf7/inputs/ERR1081238/ERR1...,gs://broad-dsp-lrma-pf7/inputs/ERR1081238/ERR1...,248.8,84.9,2820000000.0,28198096.0,28192895.0,28184018.0,28159960.0,28195246.0,28194826.0,FP0009-C,88.85,91.4,AF-W,True,120.851543,100.0,28.680673,29.3,gDNA,1147-PF-MR-CONWAY,,,2014.0


In [27]:
# ltu.upload_root_table(namespace, workspace, df_to_upload)

In [4]:
!pip install firecloud>=0.16.35


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [5]:
fapi.__version__

'0.16.35'

In [6]:
# response = fapi.rename_entity(namespace, workspace, "testing_rename_samples", "FP0009-C", "FP0009-C_THIS_IS_A_NEW_NAME")
# response

<Response [204]>

In [8]:
# response.ok

True

In [ ]:
raise Exception("DO NOT RUN ALL CELLS!")

***
## Fix the input bams for the 2022 Data:

In [6]:
name_to_bam_map = {
    'SEN-DBL-2022-086': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_086.bam',
    'SEN-DBL-2022-174': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_174.bam',
    'SEN-BAN-PP-DM1-2022-179': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_179.bam',
    'SEN-DBL-2022-039': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_039.bam',
    'SEN-DBL-2022-147': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_147.bam',
    'SEN-DBL-2022-036': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_036.bam',
    'SEN-BAN-PP-DM1-2022-145': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_145.bam',
    'SEN-DBL-2022-157': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_157.bam',
    'SEN-DBL-2022-148': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_148.bam',
    'SEN-DBL-2022-093': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_093.bam',
    'SEN-DBL-2022-173': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_173.bam',
    'SEN-DBL-2022-158': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_158.bam',
    'SEN-DBL-2022-137': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_137.bam',
    'SEN-DBL-2022-164': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_164.bam',
    'SEN-DBL-2022-108': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_108.bam',
    'SEN-BAN-PP-DM2-2022-100': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_100.bam',
    'SEN-DBL-2022-145': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_145.bam',
    'SEN-DBL-2022-151': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_151.bam',
    'SEN-DBL-2022-124': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_124.bam',
    'SEN-DBL-2022-109': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_109.bam',
    'SEN-DBL-2022-125': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_125.bam',
    'SEN-DBL-2022-007': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_007.bam',
    'SEN-BAN-PP-DM2-2022-123': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_123.bam',
    'SEN-SES-PP-DM1-2022-089': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SES_PP_DM1_089.bam',
    'SEN-DBL-2022-075': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_075.bam',
    'SEN-DBL-2022-117': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_117.bam',
    'SEN-DBL-2022-171': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_171.bam',
    'SEN-DBL-2022-065': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_065.bam',
    'SEN-DBL-2022-156': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_156.bam',
    'SEN-BAN-PP-DM1-2022-156': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_156.bam',
    'SEN-BAN-PP-DM1-2022-167': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_167.bam',
    'SEN-BAN-PP-DM2-2022-110': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_110.bam',
    'SEN-DBL-2022-153': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_153.bam',
    'SEN-DBL-2022-021': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_021.bam',
    'SEN-DBL-2022-006': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_006.bam',
    'SEN-DBL-2022-056': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_056.bam',
    'SEN-DBL-2022-071': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_071.bam',
    'SEN-DBL-2022-090': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_090.bam',
    'SEN-DBL-2022-107': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_107.bam',
    'SEN-DBL-2022-129': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_129.bam',
    'SEN-DBL-2022-150': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_150.bam',
    'SEN-DBL-2022-168': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_168.bam',
    'SEN-NDO-PP-PS-2022-251': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_PS_251.bam',
    'SEN-SES-PP-DM1-2022-091': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SES_PP_DM1_091.bam',
    'SEN-DBL-2022-008': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_008.bam',
    'SEN-DBL-2022-025': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_025.bam',
    'SEN-DBL-2022-040': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_040.bam',
    'SEN-DBL-2022-057': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_057.bam',
    'SEN-DBL-2022-074': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_074.bam',
    'SEN-DBL-2022-091': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_091.bam',
    'SEN-DBL-2022-018': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_018.bam',
    'SEN-DBL-2022-131': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_131.bam',
    'SEN-SES-PP-DM1-2022-077': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SES_PP_DM1_077.bam',
    'SEN-DBL-2022-169': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_169.bam',
    'SEN-NDO-PP-DM2-2022-090': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_DM2_090.bam',
    'SEN-SES-PP-DM1-2022-109': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SES_PP_DM1_109.bam',
    'SEN-DBL-2022-009': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_009.bam',
    'SEN-DBL-2022-030': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_030.bam',
    'SEN-DBL-2022-043': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_043.bam',
    'SEN-DBL-2022-059': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_059.bam',
    'SEN-DBL-2022-135': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_135.bam',
    'SEN-DBL-2022-092': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_092.bam',
    'SEN-NDO-PP-DM3-2022-061': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_DM3_061.bam',
    'SEN-SES-PP-DM1-2022-122': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SES_PP_DM1_122.bam',
    'SEN-DBL-2022-012': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_012.bam',
    'SEN-DBL-2022-031': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_031.bam',
    'SEN-DBL-2022-044': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_044.bam',
    'SEN-DBL-2022-061': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_061.bam',
    'SEN-SES-PP-DM1-2022-124': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SES_PP_DM1_124.bam',
    'SEN-NDO-PP-DM3-2022-064': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_DM3_064.bam',
    'SEN-DBL-2022-013': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_013.bam',
    'SEN-DBL-2022-032': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_032.bam',
    'SEN-DBL-2022-049': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_049.bam',
    'SEN-DBL-2022-062': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_062.bam',
    'SEN-DBL-2022-085': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_085.bam',
    'SEN-DBL-2022-095': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_095.bam',
    'SEN-DBL-2022-119': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_119.bam',
    'SEN-NDO-PP-DM3-2022-067': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_DM3_067.bam',
    'SEN-DBL-2022-001': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_001.bam',
    'SEN-DBL-2022-014': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_014.bam',
    'SEN-DBL-2022-087': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_087.bam',
    'SEN-DBL-2022-099': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_099.bam',
    'SEN-DBL-2022-146': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_146.bam',
    'SEN-SES-PP-DM1-2022-066': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SES_PP_DM1_066.bam',
    'SEN-DBL-2022-003': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_003.bam',
    'SEN-DBL-2022-017': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_017.bam',
    'SEN-DBL-2022-052': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_052.bam',
    'SEN-DBL-2022-066': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_066.bam',
    'SEN-DBL-2022-100': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_100.bam',
    'SEN-DBL-2022-159': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_159.bam',
    'SEN-DBL-2022-082': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_082.bam',
    'SEN-DBL-2022-034': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_034.bam',
    'SEN-DBL-2022-050': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_050.bam',
    'SEN-BAN-PP-DM3-2022-178': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_178.bam',
    'SEN-MAD-2022-177': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_MAD_177.bam',
    'SEN-SLP-2022-005': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_005.bam',
    'SEN-BAN-PP-PS-2022-269': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_269.bam',
    'SEN-BAN-PP-PS-2022-274': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_274.bam',
    'SEN-BAN-PP-DM1-2022-102': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_102.bam',
    'SEN-BAN-PP-PS-2022-264': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_264.bam',
    'SEN-BAN-PP-PS-2022-265': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_265.bam',
    'SEN-BAN-PP-PS-2022-278': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_278.bam',
    'SEN-BAN-PP-PS-2022-252': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_252.bam',
    'SEN-BAN-PP-DM1-2022-111': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_111.bam',
    'SEN-BAN-PP-PS-2022-272': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_272.bam',
    'SEN-BAN-PP-PS-2022-254': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_254.bam',
    'SEN-DBL-2022-126': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_126.bam',
    'SEN-DBL-2022-106': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_106.bam',
    'SEN-BAN-PP-PS-2022-270': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_270.bam',
    'SEN-BAN-PP-DM1-2022-114': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_114.bam',
    'SEN-BAN-PP-DM2-2022-122': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_122.bam',
    'SEN-BAN-PP-PS-2022-192': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_192.bam',
    'SEN-BAN-PP-PS-2022-234': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_234.bam',
    'SEN-BAN-PP-DM1-2022-116': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_116.bam',
    'SEN-BAN-PP-DM1-2022-137': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_137.bam',
    'SEN-BAN-PP-DM1-2022-147': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_147.bam',
    'SEN-BAN-PP-DM1-2022-158': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_158.bam',
    'SEN-BAN-PP-DM1-2022-168': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_168.bam',
    'SEN-BAN-PP-DM1-2022-180': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_180.bam',
    'SEN-BAN-PP-DM2-2022-102': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_102.bam',
    'SEN-BAN-PP-DM2-2022-112': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_112.bam',
    'SEN-BAN-PP-PS-2022-198': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_198.bam',
    'SEN-BAN-PP-PS-2022-236': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_236.bam',
    'SEN-BAN-PP-DM1-2022-103': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_103.bam',
    'SEN-BAN-PP-DM1-2022-118': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_118.bam',
    'SEN-BAN-PP-DM1-2022-138': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_138.bam',
    'SEN-BAN-PP-DM1-2022-148': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_148.bam',
    'SEN-BAN-PP-DM1-2022-159': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_159.bam',
    'SEN-BAN-PP-DM1-2022-169': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_169.bam',
    'SEN-BAN-PP-DM1-2022-182': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_182.bam',
    'SEN-BAN-PP-DM2-2022-104': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_104.bam',
    'SEN-BAN-PP-DM2-2022-116': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_116.bam',
    'SEN-BAN-PP-PS-2022-201': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_201.bam',
    'SEN-BAN-PP-PS-2022-238': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_238.bam',
    'SEN-BAN-PP-DM1-2022-104': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_104.bam',
    'SEN-BAN-PP-DM1-2022-120': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_120.bam',
    'SEN-BAN-PP-DM1-2022-139': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_139.bam',
    'SEN-BAN-PP-DM1-2022-160': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_160.bam',
    'SEN-BAN-PP-DM1-2022-173': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_173.bam',
    'SEN-BAN-PP-DM1-2022-184': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_184.bam',
    'SEN-BAN-PP-DM2-2022-105': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_105.bam',
    'SEN-BAN-PP-DM2-2022-118': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_118.bam',
    'SEN-BAN-PP-PS-2022-204': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_204.bam',
    'SEN-BAN-PP-DM1-2022-105': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_105.bam',
    'SEN-BAN-PP-DM1-2022-126': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_126.bam',
    'SEN-BAN-PP-DM1-2022-140': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_140.bam',
    'SEN-BAN-PP-DM1-2022-151': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_151.bam',
    'SEN-BAN-PP-DM1-2022-161': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_161.bam',
    'SEN-BAN-PP-DM1-2022-175': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_175.bam',
    'SEN-BAN-PP-DM1-2022-186': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_186.bam',
    'SEN-BAN-PP-DM2-2022-106': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_106.bam',
    'SEN-BAN-PP-DM2-2022-119': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_119.bam',
    'SEN-BAN-PP-PS-2022-212': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_212.bam',
    'SEN-BAN-PP-DM1-2022-107': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_107.bam',
    'SEN-BAN-PP-DM1-2022-130': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_130.bam',
    'SEN-BAN-PP-DM1-2022-141': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_141.bam',
    'SEN-BAN-PP-DM1-2022-152': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_152.bam',
    'SEN-BAN-PP-DM1-2022-162': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_162.bam',
    'SEN-BAN-PP-DM1-2022-176': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_176.bam',
    'SEN-BAN-PP-DM2-2022-093': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_093.bam',
    'SEN-BAN-PP-DM2-2022-107': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_107.bam',
    'SEN-SLP-2022-013': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_013.bam',
    'SEN-BAN-PP-PS-2022-214': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_214.bam',
    'SEN-BAN-PP-PS-2022-256': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_256.bam',
    'SEN-BAN-PP-DM1-2022-131': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_131.bam',
    'SEN-BAN-PP-DM1-2022-143': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_143.bam',
    'SEN-BAN-PP-DM1-2022-153': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_153.bam',
    'SEN-BAN-PP-DM1-2022-163': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_163.bam',
    'SEN-BAN-PP-DM1-2022-177': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_177.bam',
    'SEN-BAN-PP-DM2-2022-097': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_097.bam',
    'SEN-BAN-PP-DM2-2022-108': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_108.bam',
    'SEN-BAN-PP-PS-2022-229': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_229.bam',
    'SEN-BAN-PP-PS-2022-261': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_261.bam',
    'SEN-BAN-PP-DM1-2022-149': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_149.bam',
    'SEN-BAN-PP-DM1-2022-132': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_132.bam',
    'SEN-BAN-PP-DM1-2022-144': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_144.bam',
    'SEN-BAN-PP-DM1-2022-155': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_155.bam',
    'SEN-BAN-PP-DM1-2022-165': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_165.bam',
    'SEN-BAN-PP-DM1-2022-178': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_178.bam',
    'SEN-BAN-PP-DM2-2022-099': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_099.bam',
    'SEN-BAN-PP-DM2-2022-109': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_109.bam',
    'SEN-BAN-PP-PS-2022-230': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_230.bam',
    'SEN-BAN-PP-PS-2022-263': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_263.bam',
    'SEN-BAN-PP-DM1-2022-115': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_115.bam',
    'SEN-BAN-PP-DM1-2022-134': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM1_134.bam',
    'SEN-BAN-PP-PS-2022-273': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_PS_273.bam',
    'SEN-DBL-2022-037': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_037.bam',
    'SEN-DBL-2022-054': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_054.bam',
    'SEN-NDO-PP-PS-2022-151': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_PS_151.bam',
    'SEN-NDO-PP-PS-2022-201': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_PS_201.bam',
    'SEN-NDO-PP-PS-2022-212': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_PS_212.bam',
    'SEN-NDO-PP-PS-2022-214': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_PS_214.bam',
    'SEN-NDO-PP-PS-2022-215': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_PS_215.bam',
    'SEN-NDO-PP-PS-2022-221': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_PS_221.bam',
    'SEN-NDO-PP-PS-2022-233': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_PS_233.bam',
    'SEN-NDO-PP-PS-2022-240': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_PS_240.bam',
    'SEN-NDO-PP-PS-2022-242': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_PS_242.bam',
    'SEN-NDO-PP-PS-2022-245': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_PS_245.bam',
    'SEN-SLP-2022-019': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_019.bam',
    'SEN-SLP-2022-020': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_020.bam',
    'SEN-SLP-2022-021': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_021.bam',
    'SEN-SLP-2022-022': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_022.bam',
    'SEN-SLP-2022-023': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_023.bam',
    'SEN-SLP-2022-024': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_024.bam',
    'SEN-SLP-2022-025': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_025.bam',
    'SEN-SLP-2022-026': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_026.bam',
    'SEN-SLP-2022-027': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_027.bam',
    'SEN-SLP-2022-028': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_028.bam',
    'SEN-SLP-2022-029': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_029.bam',
    'SEN-SLP-2022-030': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_030.bam',
    'SEN-SLP-2022-031': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_031.bam',
    'SEN-SLP-2022-032': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_032.bam',
    'SEN-SLP-2022-033': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_033.bam',
    'SEN-SLP-2022-034': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_034.bam',
    'SEN-SLP-2022-035': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_035.bam',
    'SEN-SLP-2022-036': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_036.bam',
    'SEN-SLP-2022-037': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_037.bam',
    'SEN-SLP-2022-038': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_038.bam',
    'SEN-SLP-2022-039': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_039.bam',
    'SEN-SLP-2022-040': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_040.bam',
    'SEN-SLP-2022-041': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_041.bam',
    'SEN-SLP-2022-042': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_042.bam',
    'SEN-SLP-2022-043': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_043.bam',
    'SEN-SLP-2022-044': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_044.bam',
    'SEN-SLP-2022-045': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_045.bam',
    'SEN-SLP-2022-046': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_046.bam',
    'SEN-SLP-2022-047': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_047.bam',
    'SEN-SLP-2022-048': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_048.bam',
    'SEN-SLP-2022-049': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_049.bam',
    'SEN-SLP-2022-050': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_050.bam',
    'SEN-SLP-2022-051': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_051.bam',
    'SEN-SLP-2022-052': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_052.bam',
    'SEN-SLP-2022-053': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_053.bam',
    'SEN-SLP-2022-054': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_054.bam',
    'SEN-SLP-2022-055': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_055.bam',
    'SEN-SLP-2022-056': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_056.bam',
    'SEN-SLP-2022-057': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_057.bam',
    'SEN-SLP-2022-058': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_058.bam',
    'SEN-SLP-2022-059': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_059.bam',
    'SEN-SLP-2022-060': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_060.bam',
    'SEN-SLP-2022-061': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_061.bam',
    'SEN-SLP-2022-062': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_062.bam',
    'SEN-SLP-2022-063': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_063.bam',
    'SEN-SLP-2022-064': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_064.bam',
    'SEN-SLP-2022-065': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_065.bam',
    'SEN-SLP-2022-066': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_066.bam',
    'SEN-SLP-2022-067': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_067.bam',
    'SEN-SLP-2022-068': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_068.bam',
    'SEN-SLP-2022-069': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_069.bam',
    'SEN-SLP-2022-070': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_070.bam',
    'SEN-SLP-2022-072': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_072.bam',
    'SEN-SLP-2022-073': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_073.bam',
    'SEN-SLP-2022-074': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_074.bam',
    'SEN-SLP-2022-075': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_075.bam',
    'SEN-SLP-2022-076': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_076.bam',
    'SEN-SLP-2022-077': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_077.bam',
    'SEN-SLP-2022-078': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_078.bam',
    'SEN-SLP-2022-079': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_079.bam',
    'SEN-SLP-2022-080': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_080.bam',
    'SEN-SLP-2022-081': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_081.bam',
    'SEN-SLP-2022-082': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_082.bam',
    'SEN-SLP-2022-083': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_083.bam',
    'SEN-SLP-2022-084': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_084.bam',
    'SEN-SLP-2022-085': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_085.bam',
    'SEN-SLP-2022-086': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_086.bam',
    'SEN-SLP-2022-087': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_087.bam',
    'SEN-SLP-2022-088': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_088.bam',
    'SEN-SLP-2022-089': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_089.bam',
    'SEN-SLP-2022-090': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_090.bam',
    'SEN-SLP-2022-091': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_091.bam',
    'SEN-SLP-2022-092': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_092.bam',
    'SEN-SLP-2022-093': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_093.bam',
    'SEN-SLP-2022-094': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_094.bam',
    'SEN-SLP-2022-095': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_095.bam',
    'SEN-SLP-2022-096': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_096.bam',
    'SEN-SLP-2022-097': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_097.bam',
    'SEN-SLP-2022-099': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_099.bam',
    'SEN-SLP-2022-100': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_100.bam',
    'SEN-SLP-2022-098': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_098.bam',
    'SEN-NDO-PP-PS-2022-216': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_NDO_PP_PS_216.bam',
    'SEN-SLP-2022-002': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_002.bam',
    'SEN-BAN-PP-DM2-2022-136': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_136.bam',
    'SEN-BAN-PP-DM3-2022-071': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_071.bam',
    'SEN-BAN-PP-DM3-2022-086': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_086.bam',
    'SEN-BAN-PP-DM3-2022-100': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_100.bam',
    'SEN-BAN-PP-DM3-2022-139': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_139.bam',
    'SEN-BAN-PP-DM3-2022-160': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_160.bam',
    'SEN-MAD-2022-124': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_MAD_124.bam',
    'SEN-MAD-2022-179': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_MAD_179.bam',
    'SEN-SLP-2022-006': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_006.bam',
    'SEN-SLP-2022-014': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_014.bam',
    'SEN-BAN-PP-DM2-2022-125': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_125.bam',
    'SEN-BAN-PP-DM2-2022-139': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_139.bam',
    'SEN-BAN-PP-DM3-2022-060': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_060.bam',
    'SEN-BAN-PP-DM3-2022-075': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_075.bam',
    'SEN-BAN-PP-DM3-2022-087': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_087.bam',
    'SEN-BAN-PP-DM3-2022-104': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_104.bam',
    'SEN-BAN-PP-DM3-2022-144': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_144.bam',
    'SEN-BAN-PP-DM3-2022-162': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_162.bam',
    'SEN-MAD-2022-126': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_MAD_126.bam',
    'SEN-MAD-2022-180': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_MAD_180.bam',
    'SEN-SLP-2022-007': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_007.bam',
    'SEN-SLP-2022-015': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_015.bam',
    'SEN-BAN-PP-DM2-2022-129': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_129.bam',
    'SEN-BAN-PP-DM2-2022-140': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_140.bam',
    'SEN-BAN-PP-DM3-2022-062': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_062.bam',
    'SEN-BAN-PP-DM3-2022-076': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_076.bam',
    'SEN-BAN-PP-DM3-2022-088': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_88.bam',
    'SEN-BAN-PP-DM3-2022-106': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_106.bam',
    'SEN-BAN-PP-DM3-2022-145': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_145.bam',
    'SEN-BAN-PP-DM3-2022-166': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_166.bam',
    'SEN-MAD-2022-127': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_MAD_127.bam',
    'SEN-MAD-2022-181': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_MAD_181.bam',
    'SEN-SLP-2022-008': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_008.bam',
    'SEN-SLP-2022-016': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_016.bam',
    'SEN-BAN-PP-DM2-2022-130': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_130.bam',
    'SEN-BAN-PP-DM2-2022-142': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_142.bam',
    'SEN-BAN-PP-DM3-2022-063': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_063.bam',
    'SEN-BAN-PP-DM3-2022-078': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_078.bam',
    'SEN-BAN-PP-DM3-2022-089': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_089.bam',
    'SEN-BAN-PP-DM3-2022-119': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_119.bam',
    'SEN-BAN-PP-DM3-2022-152': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_152.bam',
    'SEN-BAN-PP-DM3-2022-168': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_168.bam',
    'SEN-MAD-2022-137': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_MAD_137.bam',
    'SEN-SLP-2022-001': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_001.bam',
    'SEN-SLP-2022-009': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_009.bam',
    'SEN-SLP-2022-017': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_017.bam',
    'SEN-BAN-PP-DM2-2022-131': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_131.bam',
    'SEN-BAN-PP-DM2-2022-143': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_143.bam',
    'SEN-BAN-PP-DM3-2022-065': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_065.bam',
    'SEN-BAN-PP-DM3-2022-079': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_079.bam',
    'SEN-BAN-PP-DM3-2022-092': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_092.bam',
    'SEN-BAN-PP-DM3-2022-128': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_128.bam',
    'SEN-BAN-PP-DM3-2022-153': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_153.bam',
    'SEN-BAN-PP-DM3-2022-169': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_169.bam',
    'SEN-MAD-2022-139': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_MAD_139.bam',
    'SEN-SLP-2022-010': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_010.bam',
    'SEN-SLP-2022-018': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_018.bam',
    'SEN-BAN-PP-DM2-2022-132': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_132.bam',
    'SEN-BAN-PP-DM2-2022-150': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_150.bam',
    'SEN-BAN-PP-DM3-2022-066': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_066.bam',
    'SEN-BAN-PP-DM3-2022-081': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_081.bam',
    'SEN-BAN-PP-DM3-2022-093': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_093.bam',
    'SEN-BAN-PP-DM3-2022-132': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_132.bam',
    'SEN-BAN-PP-DM3-2022-154': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_154.bam',
    'SEN-MAD-2022-140': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_MAD_140.bam',
    'SEN-BAN-PP-DM3-2022-171': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_171.bam',
    'SEN-SLP-2022-011': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_011.bam',
    'SEN-DBL-2022-069': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_069.bam',
    'SEN-BAN-PP-DM2-2022-134': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_134.bam',
    'SEN-BAN-PP-DM3-2022-054': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_054.bam',
    'SEN-BAN-PP-DM3-2022-067': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_067.bam',
    'SEN-BAN-PP-DM3-2022-083': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_083.bam',
    'SEN-BAN-PP-DM3-2022-094': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_094.bam',
    'SEN-BAN-PP-DM3-2022-133': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_133.bam',
    'SEN-BAN-PP-DM3-2022-155': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_155.bam',
    'SEN-BAN-PP-DM3-2022-175': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_175.bam',
    'SEN-MAD-2022-173': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_MAD_173.bam',
    'SEN-BAN-PP-DM3-2022-059': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_059.bam',
    'SEN-SLP-2022-012': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_012.bam',
    'SEN-DBL-2022-089': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_DBL_089.bam',
    'SEN-BAN-PP-DM2-2022-135': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_135.bam',
    'SEN-BAN-PP-DM3-2022-057': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_057.bam',
    'SEN-BAN-PP-DM3-2022-069': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_069.bam',
    'SEN-BAN-PP-DM3-2022-084': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_084.bam',
    'SEN-BAN-PP-DM3-2022-097': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_097.bam',
    'SEN-BAN-PP-DM3-2022-138': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_138.bam',
    'SEN-BAN-PP-DM3-2022-157': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM3_157.bam',
    'SEN-SLP-2022-003': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_003.bam',
    'SEN-SLP-2022-004': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_SLP_004.bam',
    'SEN-BAN-PP-DM2-2022-124': 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN_PP_DM2_124.bam',
}

In [9]:
# Generate BAI paths:
name_to_bai_map = {k: v + ".bai" for k,v in name_to_bam_map.items()}

In [11]:
# Create a dataframe containing only the samples we need to update:
df_for_upload = t[t["sample"].isin(name_to_bam_map.keys())]

In [12]:
# Create lists in the order the samples appear in the dataframe
# so that we can set the files correctly:
samples = list(df_for_upload["sample"])
input_bam_paths = [name_to_bam_map[s] for s in samples]
input_bai_paths = [name_to_bai_map[s] for s in samples]

In [13]:
# Update the dataframe with the new input bams / input bais:
df_for_upload = df_for_upload.assign(input_bam=input_bam_paths)
df_for_upload = df_for_upload.assign(input_bai=input_bai_paths)
df_for_upload = df_for_upload.rename(columns={"sample": "entity:sample_id"})

In [14]:
# pd.get_option("display.max_columns")  # == 20
pd.set_option("display.max_columns", None)
df_for_upload

,entity:sample_id,ENA,additional_metadata,aligned_bai,aligned_bam,aligned_est_fold_cov,aligned_frac_bases,aligned_num_bases,aligned_num_reads,aligned_read_length,average_identity,collection_site,collection_site_latitude,collection_site_longitude,country,country_latitude,country_longitude,dataset_provenance,fastqc_report,fq1,fq2,fq_unpaired,input_bai,input_bam,input_fq_end1,input_fq_end2,insert_size_average,insert_size_standard_deviation,num_bases,num_reads,num_reads_Q10,num_reads_Q12,num_reads_Q15,num_reads_Q5,num_reads_Q7,participant,pct_callable,pct_properly_paired_reads,population,qc_pass,raw_est_fold_cov,read_length,read_qual_mean,read_qual_median,sample_type,study,unaligned_bai,unaligned_bam,year
21940,SEN-SLP-2022-019,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,218.284266,0.968661,5093191646.0,38319388.0,151.0,98.124559,Thies,14.791054,-16.925124,Senegal,14.359109,-14.470363,Broad 2022 Senegal Dataset,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,,,0.0,0.0,5786227588.0,38319388.0,38319388.0,38319380.0,38314792.0,38319388.0,38319388.0,SEN-SLP-2022-019,,0.0,,,247.986436,151.0,27.98184,28.807947,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,2022.0
21941,SEN-SLP-2022-020,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,86.791733,0.965008,2025097542.0,15696819.0,151.0,98.221838,Thies,14.791054,-16.925124,Senegal,14.359109,-14.470363,Broad 2022 Senegal Dataset,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,,,0.0,0.0,2370219669.0,15696819.0,15696817.0,15696809.0,15694511.0,15696819.0,15696819.0,SEN-SLP-2022-020,,0.0,,,101.582995,151.0,27.468452,28.397351,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,2022.0
21942,SEN-SLP-2022-021,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,152.283072,0.977283,3553196402.0,26122762.0,151.0,98.450575,Thies,14.791054,-16.925124,Senegal,14.359109,-14.470363,Broad 2022 Senegal Dataset,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,,,0.0,0.0,3944537062.0,26122762.0,26122762.0,26122758.0,26120511.0,26122762.0,26122762.0,SEN-SLP-2022-021,,0.0,,,169.05517,151.0,27.970379,28.913907,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,2022.0
21943,SEN-SLP-2022-022,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,172.545182,0.972976,4025968946.0,29657099.0,151.0,98.472391,Thies,14.791054,-16.925124,Senegal,14.359109,-14.470363,Broad 2022 Senegal Dataset,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,,,0.0,0.0,4478221949.0,29657099.0,29657099.0,29657095.0,29654515.0,29657099.0,29657099.0,SEN-SLP-2022-022,,0.0,,,191.927864,151.0,27.901675,28.807947,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5

In [15]:
# Upload the new data:
# ltu.upload_root_table(namespace, workspace, df_for_upload)

In [ ]:
raise Exception("DO NOT RUN ALL CELLS!")

***
## Fix the input bams for the 2019 data:


In [16]:
name_to_bam_map = {
        "SEN-CSS-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_001-SM-K5VAX.bam",
        "SEN-CSS-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_002-SM-K5VAX.bam",
        "SEN-CSS-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_003-SM-K5VAX.bam",
        "SEN-CSS-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_004-SM-K5VAX.bam",
        "SEN-CSS-2019-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_005-SM-K5VAX.bam",
        "SEN-CSS-2019-007": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_007-SM-K5VAX.bam",
        "SEN-CSS-2019-009": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_009-SM-K5VAX.bam",
        "SEN-CSS-2019-014": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_014-SM-K5VAX.bam",
        "SEN-CSS-2019-015": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_015-SM-K5VAX.bam",
        "SEN-CSS-2019-016": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_016-SM-K5VAX.bam",
        "SEN-CSS-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_018-SM-K5VAX.bam",
        "SEN-CSS-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_020-SM-K5VAX.bam",
        "SEN-CSS-2019-024": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_024-SM-K5VAX.bam",
        "SEN-CSS-2019-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_025-SM-K5VAX.bam",
        "SEN-CSS-2019-030": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_CSS_2019_030-SM-K5VAX.bam",
        "SEN-DEG-2016-059": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DE_2016_059-SM-K5VAY.bam",
        "SEN-DEG-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_DE1.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_DE3.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_DE4.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_DE005.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_DE006.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-007": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_DE007.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-008": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_DE008.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_DE010.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-011": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_DE011.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_DE012.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-013": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_DE013.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-015": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_DE015.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_DE018.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-019": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_DE019.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_DE020.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-021": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_DE21.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-023": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_DE23.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-024": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_DE24.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-027": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_DE27.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-029": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_DE29.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-031": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_DE31.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-033": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_DE33.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-034": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_DE34.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-DEG-2019-035": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_035-SM-K5VAX.bam",
        "SEN-DEG-2019-036": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_036-SM-K5VAX.bam",
        "SEN-DEG-2019-037": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_037-SM-K5VAX.bam",
        "SEN-DEG-2019-041": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_041-SM-K5VAX.bam",
        "SEN-DEG-2019-042": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_042-SM-K5VAX.bam",
        "SEN-DEG-2019-043": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_043-SM-K5VAX.bam",
        "SEN-DEG-2019-044": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_044-SM-K5VAX.bam",
        "SEN-DEG-2019-045": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_045-SM-K5VAX.bam",
        "SEN-DEG-2019-046": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_046-SM-K5VAX.bam",
        "SEN-DEG-2019-048": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_048-SM-K5VAX.bam",
        "SEN-DEG-2019-051": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_051-SM-K5VAX.bam",
        "SEN-DEG-2019-052": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_052-SM-K5VAX.bam",
        "SEN-DEG-2019-053": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_053-SM-K5VAX.bam",
        "SEN-DEG-2019-054": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DEG_2019_054-SM-K5VAX.bam",
        "SEN-DWR-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_DWR_2019_001-SM-K5VAX.bam",
        "SEN-GAB-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA2.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA3.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-007": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA7.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-009": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA9.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA10.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-011": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA11.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-013": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA13.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-014": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GAB_2019_014-SM-K5VAX.bam",
        "SEN-GAB-2019-016": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA16.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA17.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_GA18.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-019": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_GA19.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_GA20.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA22.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-023": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_GA23.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-024": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_GA24.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-027": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_GA27.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-028": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_GA28.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-030": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_GA30.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-033": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA33.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-034": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GAB_2019_034-SM-K5VAX.bam",
        "SEN-GAB-2019-037": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GAB_2019_037-SM-K5VAX.bam",
        "SEN-GAB-2019-039": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GAB_2019_039-SM-K5VAX.bam",
        "SEN-GAB-2019-040": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA040.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-045": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA045.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-046": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA046.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA047.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-048": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GAB_2019_048-SM-K5VAX.bam",
        "SEN-GAB-2019-049": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA049.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-050": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GAB_2019_050-SM-K5VAY.bam",
        "SEN-GAB-2019-051": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GAB_2019_051-SM-K5VAY.bam",
        "SEN-GAB-2019-052": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA052.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-054": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA054.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-055": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA055.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-056": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA056.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-057": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_GA057.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-058": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA058.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-059": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA059.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-060": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA060.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-061": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA061.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-063": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA063.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-064": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GAB_2019_064-SM-K5VAY.bam",
        "SEN-GAB-2019-066": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA066.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-067": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA067.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-068": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA068.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-070": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA070.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-071": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA071.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-072": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA072.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-076": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA076.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-079": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA079.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-080": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_GA080.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-084": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA084.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-085": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA085.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-086": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_GA086.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-088": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GAB_2019_088-SM-K5VAY.bam",
        "SEN-GAB-2019-089": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GAB_2019_089-SM-K5VAY.bam",
        "SEN-GAB-2019-091": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_GA091.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-092": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GAB_2019_092-SM-K5VAY.bam",
        "SEN-GAB-2019-093": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GAB_2019_093-SM-K5VAY.bam",
        "SEN-GAB-2019-096": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA096.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-097": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA097.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GAB-2019-098": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_GA098.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-GMK-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GMK_2019_001-SM-K5VAX.bam",
        "SEN-GMK-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GMK_2019_002-SM-K5VAX.bam",
        "SEN-GMK-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GMK_2019_003-SM-K5VAX.bam",
        "SEN-GMK-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_GMK_2019_004-SM-K5VAX.bam",
        "SEN-KAF-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF1.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF2.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF3.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF6.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-014": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF14.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-015": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF15.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-016": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF16.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF17.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF18.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-019": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF19.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-021": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF21.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF22.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-023": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF23.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-024": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF24.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KF25.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-027": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF27.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-029": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KF29.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-030": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KF30.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-031": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KF31.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-032": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KF32.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-035": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KF35.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-038": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KF38.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-039": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KF39.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-040": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF40.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-042": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF42.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-043": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF43.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF47.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-048": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF48.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-049": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF49.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-052": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF52.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-056": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF56.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-057": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF57.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-058": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KF58.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KAF-2019-059": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KF59.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG004.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG005.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG006.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG010.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG012.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-013": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG013.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-016": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG016.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KG017.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG018.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-019": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG019.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG020.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG022.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG025.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-026": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG026.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-027": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KG027.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-028": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG028.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-030": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG030.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-031": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG031.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-032": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG032.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-034": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KG034.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-035": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG035.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-041": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG041.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-042": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG042.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-044": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG044.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG047.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-048": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG048.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-050": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG050.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-051": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG051.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-052": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG052.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-054": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG054.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-056": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG056.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-057": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG057.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-058": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG058.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-059": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG059.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-061": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KG061.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-062": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG062.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-063": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KG063.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-065": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG065.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-067": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG067.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-068": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG068.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-074": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KG074.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-075": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KG075.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-078": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KG078.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-079": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KG079.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-080": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KG080.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-081": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KG081.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-082": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KG082.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-085": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KG085.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-086": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KG086.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-087": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KG087.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-088": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG088.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-091": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KG091.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-094": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KG094.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-096": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG096.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-100": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG100.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-102": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG102.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-104": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG104.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-105": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG105.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KDG-2019-109": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KG109.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-252": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD252.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-253": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KD253.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-254": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD254.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-255": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KD255.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-256": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD256.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-257": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD257.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-258": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD258.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-259": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KD259.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-261": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KD261.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-264": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KD264.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-265": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD265.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-266": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KD266.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-267": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KD267.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-268": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KD268.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-269": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KD269.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-270": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KD270.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-271": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD271.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-272": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD272.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-273": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD273.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-274": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD274.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-275": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD275.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-276": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD276.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-277": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD277.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-278": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD278.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-280": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD280.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-281": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD281.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-284": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD284.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-285": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD285.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-286": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD286.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-288": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD288.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-289": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD289.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-290": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD290.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-291": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD291.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-293": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD293.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-294": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD294.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-295": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KD295.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-296": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD296.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-297": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD297.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-299": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD299.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-300": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD300.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-301": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD301.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-302": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD302.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-304": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD304.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-305": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD305.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-306": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD306.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-307": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_307-SM-K5VAX.bam",
        "SEN-KLD-2019-309": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD309.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-310": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD310.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-311": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD311.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-312": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD312.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-313": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KD313.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-371": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD371.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-372": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_372-SM-K5VAX.bam",
        "SEN-KLD-2019-373": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD373.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-374": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD374.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-375": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD375.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-376": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD376.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-377": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD377.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-378": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_378-SM-K5VAX.bam",
        "SEN-KLD-2019-379": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD379.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-380": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD380.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-381": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD381.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-382": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KD382.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-383": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD383.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-384": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD384.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-390": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD390.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-391": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD391.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-392": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KD392.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KLD-2019-394": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_394-SM-K5VAX.bam",
        "SEN-KLD-2019-395": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_395-SM-K5VAX.bam",
        "SEN-KLD-2019-396": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_396-SM-K5VAX.bam",
        "SEN-KLD-2019-397": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_397-SM-K5VAX.bam",
        "SEN-KLD-2019-399": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_399-SM-K5VAX.bam",
        "SEN-KLD-2019-401": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_401-SM-K5VAX.bam",
        "SEN-KLD-2019-402": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_402-SM-K5VAX.bam",
        "SEN-KLD-2019-404": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_404-SM-K5VAX.bam",
        "SEN-KLD-2019-405": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_405-SM-K5VAX.bam",
        "SEN-KLD-2019-406": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_406-SM-K5VAX.bam",
        "SEN-KLD-2019-410": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_410-SM-K5VAX.bam",
        "SEN-KLD-2019-411": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_411-SM-K5VAX.bam",
        "SEN-KLD-2019-412": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_412-SM-K5VAX.bam",
        "SEN-KLD-2019-413": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_413-SM-K5VAX.bam",
        "SEN-KLD-2019-414": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_414-SM-K5VAX.bam",
        "SEN-KLD-2019-416": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_416-SM-K5VAX.bam",
        "SEN-KLD-2019-417": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_417-SM-K5VAX.bam",
        "SEN-KLD-2019-418": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_418-SM-K5VAX.bam",
        "SEN-KLD-2019-419": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_419-SM-K5VAX.bam",
        "SEN-KLD-2019-420": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_420-SM-K5VAX.bam",
        "SEN-KLD-2019-421": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_421-SM-K5VAX.bam",
        "SEN-KLD-2019-422": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_422-SM-K5VAX.bam",
        "SEN-KLD-2019-423": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_423-SM-K5VAX.bam",
        "SEN-KLD-2019-427": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_427-SM-K5VAX.bam",
        "SEN-KLD-2019-428": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_428-SM-K5VAY.bam",
        "SEN-KLD-2019-429": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_429-SM-K5VAY.bam",
        "SEN-KLD-2019-430": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_430-SM-K5VAY.bam",
        "SEN-KLD-2019-431": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_431-SM-K5VAY.bam",
        "SEN-KLD-2019-433": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_433-SM-K5VAY.bam",
        "SEN-KLD-2019-435": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_435-SM-K5VAY.bam",
        "SEN-KLD-2019-436": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_436-SM-K5VAY.bam",
        "SEN-KLD-2019-464": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_464-SM-K5VAY.bam",
        "SEN-KLD-2019-468": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_KLD_2019_468-SM-K5VAY.bam",
        "SEN-KML-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK2.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK3.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK5.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK6.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-007": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK7.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-008": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK8.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_MK10.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KML-2019-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK12.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK17.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK18.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK20.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_MK22.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-KML-2019-023": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK23.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK25.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-027": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK27.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KML-2019-029": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MK29.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO3.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO4.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO6.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-008": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO8.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-009": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO9.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO10.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-011": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO11.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO12.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-014": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO14.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-015": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO15.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-016": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO16.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO17.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KO18.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-KTL-2019-019": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KO19.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK3.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK4.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_NK6.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-008": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_NK8.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-009": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_NK9.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_NK10.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_NK12.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-013": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_NK13.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_NK20.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_NK22.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK25.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-026": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK26.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-031": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK31.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-034": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_NK34.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-035": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK35.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-036": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK36.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-037": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK37.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-038": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK38.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-040": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK40.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-044": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK44.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-045": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_MAK_2019_045-SM-K5VAY.bam",
        "SEN-MAK-2019-046": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK46.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-050": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK50.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-051": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK51.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-052": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK52.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-055": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK55.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-056": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_NK56.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-057": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK57.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-059": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_NK59.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-065": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK65.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-066": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK66.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-068": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_NK68.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-069": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_NK69.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-070": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK70.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-072": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK72.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-074": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK74.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-076": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK76.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-081": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_NK81.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-082": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK82.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-084": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK84.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-085": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK85.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-086": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK86.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-087": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK87.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-091": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK91.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-097": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK97.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-098": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK98.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MAK-2019-100": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NK100.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_MO1.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_MO2.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_MO3.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_MO4.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_MO6.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-007": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_MO7.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_MO10.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-011": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_MO11.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_MO12.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-014": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_MO14.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-015": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_MO15.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-016": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_MO16.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_MO17.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_MO18.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_MO20.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_MO22.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-MDR-2019-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_MO25.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-NDT-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_NDT_2019_001-SM-K5VAY.bam",
        "SEN-NJC-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NC1.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-NJC-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NC2.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-NJC-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NC3.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-NJC-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NC4.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-NJC-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NC6.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-NJC-2019-009": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_NC9.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-NJC-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_NC10.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-NSN-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_NSN_2019_001-SM-K5VAY.bam",
        "SEN-ROB-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_001-SM-K5VAX.bam",
        "SEN-ROB-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_002-SM-K5VAX.bam",
        "SEN-ROB-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_003-SM-K5VAX.bam",
        "SEN-ROB-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_006-SM-K5VAX.bam",
        "SEN-ROB-2019-007": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_007-SM-K5VAX.bam",
        "SEN-ROB-2019-008": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_008-SM-K5VAX.bam",
        "SEN-ROB-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_010-SM-K5VAX.bam",
        "SEN-ROB-2019-011": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_011-SM-K5VAX.bam",
        "SEN-ROB-2019-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_012-SM-K5VAX.bam",
        "SEN-ROB-2019-196": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_196-SM-K5VAX.bam",
        "SEN-ROB-2019-197": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_197-SM-K5VAX.bam",
        "SEN-ROB-2019-198": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_198-SM-K5VAX.bam",
        "SEN-ROB-2019-199": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_199-SM-K5VAX.bam",
        "SEN-ROB-2019-200": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROB_2019_200-SM-K5VAX.bam",
        "SEN-ROS-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROS_2019_001-SM-K5VAX.bam",
        "SEN-ROS-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROS_2019_002-SM-K5VAX.bam",
        "SEN-ROS-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROS_2019_003-SM-K5VAX.bam",
        "SEN-ROS-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROS_2019_004-SM-K5VAX.bam",
        "SEN-ROS-2019-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROS_2019_005-SM-K5VAX.bam",
        "SEN-ROS-2019-007": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_ROS_2019_007-SM-K5VAX.bam",
        "SEN-SED-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SG1.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SED-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SG2.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SED-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SG3.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SED-2019-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SG5.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SED-2019-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SG22.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SED-2019-023": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SG23.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SED-2019-024": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SG24.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SED-2019-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SG25.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SED-2019-026": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SG26.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SED-2019-027": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SG27.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SED-2019-029": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SG29.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SED-2019-030": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SG30.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SED-2019-031": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SG31.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SED-2019-035": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SG35.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SED-2019-037": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SG37.2019_HVWJMDSXX.3.unmapped.bam",
        "SEN-SED-2019-039": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SG39.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SED-2019-041": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SG41.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SED-2019-042": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SG42.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SED-2019-043": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SG43.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SED-2019-044": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SG44.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SED-2019-046": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SG46.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SED-2019-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SG47.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SED-2019-048": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SG48.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SED-2019-049": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SG49.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SED-2019-051": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SG51.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SES-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SE2.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SES-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SE4.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SES-2019-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SE5.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SES-2019-008": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SE8.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SES-2019-009": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SE9.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SES-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SE10.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SES-2019-011": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SE11.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SES-2019-013": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SE13.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SES-2019-015": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SE15.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SES-2019-016": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SE16.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SES-2019-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SE17.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SES-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_SE18.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SES-2019-019": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_SE19.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SES-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_SE20.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SES-2019-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_SE22.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SES-2019-023": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SE23.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SES-2019-024": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_SE24.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SES-2019-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_SE25.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SES-2019-026": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_SE26.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SES-2019-027": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_SE27.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SES-2019-030": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SE30.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SES-2019-031": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SE31.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SES-2019-032": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SE32.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SES-2019-034": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SE34.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SES-2019-037": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SE37.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SES-2019-038": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SE38.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SES-2019-039": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SE39.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SES-2019-041": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SE41.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SES-2019-042": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_SE42.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SES-2019-043": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_043-SM-K5VAX.bam",
        "SEN-SES-2019-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SE47.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SES-2019-049": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SE49.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SES-2019-050": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_SE50.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SES-2019-051": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_051-SM-K5VAY.bam",
        "SEN-SES-2019-052": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_052-SM-K5VAY.bam",
        "SEN-SES-2019-053": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_053-SM-K5VAY.bam",
        "SEN-SES-2019-054": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_054-SM-K5VAY.bam",
        "SEN-SES-2019-055": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_055-SM-K5VAY.bam",
        "SEN-SES-2019-056": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_056-SM-K5VAY.bam",
        "SEN-SES-2019-057": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_057-SM-K5VAY.bam",
        "SEN-SES-2019-058": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_058-SM-K5VAY.bam",
        "SEN-SES-2019-059": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_059-SM-K5VAY.bam",
        "SEN-SES-2019-062": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_062-SM-K5VAX.bam",
        "SEN-SES-2019-063": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_063-SM-K5VAX.bam",
        "SEN-SES-2019-064": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_064-SM-K5VAX.bam",
        "SEN-SES-2019-065": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_065-SM-K5VAX.bam",
        "SEN-SES-2019-066": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_066-SM-K5VAX.bam",
        "SEN-SES-2019-067": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_067-SM-K5VAX.bam",
        "SEN-SES-2019-068": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_068-SM-K5VAX.bam",
        "SEN-SES-2019-069": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_069-SM-K5VAX.bam",
        "SEN-SES-2019-070": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_070-SM-K5VAX.bam",
        "SEN-SES-2019-071": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_071-SM-K5VAX.bam",
        "SEN-SES-2019-072": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_072-SM-K5VAX.bam",
        "SEN-SES-2019-073": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_073-SM-K5VAX.bam",
        "SEN-SES-2019-074": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_074-SM-K5VAX.bam",
        "SEN-SES-2019-075": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_075-SM-K5VAY.bam",
        "SEN-SES-2019-076": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_076-SM-K5VAY.bam",
        "SEN-SES-2019-077": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_077-SM-K5VAY.bam",
        "SEN-SES-2019-078": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_078-SM-K5VAY.bam",
        "SEN-SES-2019-079": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_079-SM-K5VAY.bam",
        "SEN-SES-2019-080": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_080-SM-K5VAY.bam",
        "SEN-SES-2019-081": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_081-SM-K5VAY.bam",
        "SEN-SES-2019-082": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_082-SM-K5VAY.bam",
        "SEN-SES-2019-083": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_083-SM-K5VAY.bam",
        "SEN-SES-2019-084": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_084-SM-K5VAY.bam",
        "SEN-SES-2019-085": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_085-SM-K5VAY.bam",
        "SEN-SES-2019-086": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_086-SM-K5VAY.bam",
        "SEN-SES-2019-087": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_087-SM-K5VAY.bam",
        "SEN-SES-2019-088": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_088-SM-K5VAY.bam",
        "SEN-SES-2019-089": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_089-SM-K5VAY.bam",
        "SEN-SES-2019-090": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_090-SM-K5VAX.bam",
        "SEN-SES-2019-091": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_091-SM-K5VAY.bam",
        "SEN-SES-2019-092": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_092-SM-K5VAY.bam",
        "SEN-SES-2019-093": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_093-SM-K5VAY.bam",
        "SEN-SES-2019-094": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_094-SM-K5VAY.bam",
        "SEN-SES-2019-095": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_095-SM-K5VAY.bam",
        "SEN-SES-2019-096": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_096-SM-K5VAY.bam",
        "SEN-SES-2019-097": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_097-SM-K5VAY.bam",
        "SEN-SES-2019-098": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_098-SM-K5VAY.bam",
        "SEN-SES-2019-099": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_099-SM-K5VAY.bam",
        "SEN-SES-2019-100": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_100-SM-K5VAY.bam",
        "SEN-SES-2019-101": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_101-SM-K5VAY.bam",
        "SEN-SES-2019-102": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_102-SM-K5VAY.bam",
        "SEN-SES-2019-103": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_103-SM-K5VAY.bam",
        "SEN-SES-2019-105": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_105-SM-K5VAY.bam",
        "SEN-SES-2019-106": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_106-SM-K5VAY.bam",
        "SEN-SES-2019-107": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_107-SM-K5VAY.bam",
        "SEN-SES-2019-108": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_108-SM-K5VAY.bam",
        "SEN-SES-2019-109": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_109-SM-K5VAY.bam",
        "SEN-SES-2019-110": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_110-SM-K5VAY.bam",
        "SEN-SES-2019-111": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_111-SM-K5VAY.bam",
        "SEN-SES-2019-115": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_115-SM-K5VAY.bam",
        "SEN-SES-2019-116": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_116-SM-K5VAY.bam",
        "SEN-SES-2019-118": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_118-SM-K5VAY.bam",
        "SEN-SES-2019-119": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SES_2019_119-SM-K5VAY.bam",
        "SEN-SLP-2007-086": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2007_086-SM-K5VAX.bam",
        "SEN-SLP-2009-062": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2009_062-SM-K5VAY.bam",
        "SEN-SLP-2009-106": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2009_106-SM-K5VAX.bam",
        "SEN-SLP-2010-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2010_010-SM-K5VAY.bam",
        "SEN-SLP-2010-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2010_012-SM-K5VAY.bam",
        "SEN-SLP-2010-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2010_025-SM-K5VAY.bam",
        "SEN-SLP-2010-125": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2010_125-SM-K5VAY.bam",
        "SEN-SLP-2010-166": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2010_166-SM-K5VAY.bam",
        "SEN-SLP-2010-199": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2010_199-SM-K5VAY.bam",
        "SEN-SLP-2010-210": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2010_210-SM-K5VAY.bam",
        "SEN-SLP-2011-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2011_005-SM-K5VAY.bam",
        "SEN-SLP-2011-078": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2011_078-SM-K5VAY.bam",
        "SEN-SLP-2011-104": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2011_104-SM-K5VAY.bam",
        "SEN-SLP-2011-110": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2011_110-SM-K5VAY.bam",
        "SEN-SLP-2011-146": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2011_146-SM-K5VAY.bam",
        "SEN-SLP-2012-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2012_025-SM-K5VAY.bam",
        "SEN-SLP-2012-071": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2012_071-SM-K5VAX.bam",
        "SEN-SLP-2012-106": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2012_106-SM-K5VAX.bam",
        "SEN-SLP-2012-170": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2012_170-SM-K5VAY.bam",
        "SEN-SLP-2012-214": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2012_214-SM-K5VAY.bam",
        "SEN-SLP-2012-221": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2012_221-SM-K5VAY.bam",
        "SEN-SLP-2012-258": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2012_258-SM-K5VAY.bam",
        "SEN-SLP-2012-264": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2012_264-SM-K5VAX.bam",
        "SEN-SLP-2013-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2013_017-SM-K5VAY.bam",
        "SEN-SLP-2013-036": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2013_036-SM-K5VAX.bam",
        "SEN-SLP-2013-050": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2013_050-SM-K5VAY.bam",
        "SEN-SLP-2013-074": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2013_074-SM-K5VAX.bam",
        "SEN-SLP-2013-099": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2013_099-SM-K5VAY.bam",
        "SEN-SLP-2013-194": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2013_194-SM-K5VAY.bam",
        "SEN-SLP-2013-201": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2013_201-SM-K5VAY.bam",
        "SEN-SLP-2014-066": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2014_066-SM-K5VAX.bam",
        "SEN-SLP-2014-193": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2014_193-SM-K5VAY.bam",
        "SEN-SLP-2014-194": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2014_194-SM-K5VAY.bam",
        "SEN-SLP-2014-198": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2014_198-SM-K5VAX.bam",
        "SEN-SLP-2014-200": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2014_200-SM-K5VAX.bam",
        "SEN-SLP-2015-066": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2015_066-SM-K5VAX.bam",
        "SEN-SLP-2015-074": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TH_2015_074-SM-K5VAY.bam",
        "SEN-SLP-2015-082": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2015_082-SM-K5VAX.bam",
        "SEN-SLP-2015-096": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2015_096-SM-K5VAX.bam",
        "SEN-SLP-2015-107": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2015_107-SM-K5VAX.bam",
        "SEN-SLP-2015-115": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2015_115-SM-K5VAX.bam",
        "SEN-SLP-2015-121": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2015_121-SM-K5VAX.bam",
        "SEN-SLP-2016-014": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.014.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-015": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.015.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-016": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.016.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.017.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.022.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-024": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2016_024-SM-K5VAX.bam",
        "SEN-SLP-2016-029": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2016_029-SM-K5VAY.bam",
        "SEN-SLP-2016-031": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.031.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-040": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.040.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-041": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.041.16_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-042": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2016_042-SM-K5VAX.bam",
        "SEN-SLP-2016-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.047.16_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-052": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.052.16_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-056": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.056.16_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-059": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.059.16_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-061": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2016_061-SM-K5VAX.bam",
        "SEN-SLP-2016-064": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.064.16_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-066": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.066.16_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-069": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.069.16_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-083": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2016_083-SM-K5VAY.bam",
        "SEN-SLP-2016-089": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_Sen.Th.089.16_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-097": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_Sen.Th.097.16_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-100": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_Sen.Th.100.16_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-107": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_Sen.Th.107.16_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-114": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2016_114-SM-K5VAY.bam",
        "SEN-SLP-2016-121": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_Sen.Th.121.16_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-122": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_Sen.Th.122.16_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-124": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.124.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-127": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.127.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-128": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2016_128-SM-K5VAY.bam",
        "SEN-SLP-2016-130": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.130.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-131": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.131.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-134": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.134.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-138": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.138.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-139": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.139.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2016-142": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.142.16_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2017-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TH_2017_020-SM-K5VAY.bam",
        "SEN-SLP-2017-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.025.17_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2017-052": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2017_052-SM-K5VAY.bam",
        "SEN-SLP-2017-056": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2017_056-SM-K5VAY.bam",
        "SEN-SLP-2017-060": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2017_060-SM-K5VAY.bam",
        "SEN-SLP-2017-114": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.114.17_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2017-115": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.115.17_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.002.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.003.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.004.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.005.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.006.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-007": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.007.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.010.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.012.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-013": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.013.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-014": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.014.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-015": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.015.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-016": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.016.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.020.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.022.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-023": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.023.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-024": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.024.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-034": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_Sen.Th.034.18_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-040": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.040.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-042": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.042.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-043": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.043.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-046": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.046.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.047.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-048": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.048.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-049": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.049.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-051": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_Sen.Th.051.18_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-055": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_Sen.Th.055.18_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-056": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_Sen.Th.056.18_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-058": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_Sen.Th.058.18_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-060": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_Sen.Th.060.18_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-061": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_Sen.Th.061.18_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-063": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_Sen.Th.063.18_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-064": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_Sen.Th.064.18_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-065": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_Sen.Th.065.18_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-066": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.066.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-070": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.070.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-071": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.071.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-072": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.072.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2018-073": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_Sen.Th.073.18_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH001.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH002.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH003.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH005.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH006.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-008": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH008.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-009": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH009.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH010.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-011": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH011.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-013": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH013.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-014": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH014.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-015": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH015.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-016": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH016.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH018.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-019": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH019.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH020.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-021": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH021.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-023": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH023.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-024": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH024.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH025.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-026": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH026.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-027": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH027.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-028": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH028.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-029": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH029.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-030": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH030.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-033": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TH033.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-034": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TH034.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-035": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TH035.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-036": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TH036.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-039": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TH039.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-040": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TH040.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-041": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH041.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-042": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TH042.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-043": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TH043.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-044": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH044.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-045": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH045.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-046": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH046.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH047.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-048": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH048.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-049": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH049.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-050": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH050.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-051": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH051.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-056": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH056.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-057": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH057.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-059": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH059.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-060": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH060.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-061": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH061.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-062": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH062.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-063": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH063.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-064": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TH064.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-068": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH068.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-069": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TH069.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SLP-2019-078": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SLP_2019_078-SM-K5VAY.bam",
        "SEN-SMS-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_KM002.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KM003.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KM005.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KM006.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-007": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KM007.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KM012.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-016": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KM016.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_KM017.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KM018.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-019": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KM019.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KM020.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-021": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_KM021.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KM022.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-023": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_KM023.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-SMS-2019-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_025-SM-K5VAX.bam",
        "SEN-SMS-2019-027": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_027-SM-K5VAX.bam",
        "SEN-SMS-2019-028": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_028-SM-K5VAX.bam",
        "SEN-SMS-2019-029": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_029-SM-K5VAX.bam",
        "SEN-SMS-2019-030": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_030-SM-K5VAX.bam",
        "SEN-SMS-2019-032": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_032-SM-K5VAX.bam",
        "SEN-SMS-2019-033": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_033-SM-K5VAX.bam",
        "SEN-SMS-2019-034": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_034-SM-K5VAX.bam",
        "SEN-SMS-2019-035": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_035-SM-K5VAX.bam",
        "SEN-SMS-2019-036": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_036-SM-K5VAX.bam",
        "SEN-SMS-2019-037": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_037-SM-K5VAX.bam",
        "SEN-SMS-2019-038": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_038-SM-K5VAX.bam",
        "SEN-SMS-2019-039": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_039-SM-K5VAX.bam",
        "SEN-SMS-2019-040": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_040-SM-K5VAX.bam",
        "SEN-SMS-2019-041": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_041-SM-K5VAX.bam",
        "SEN-SMS-2019-042": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_042-SM-K5VAX.bam",
        "SEN-SMS-2019-043": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_043-SM-K5VAX.bam",
        "SEN-SMS-2019-044": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_044-SM-K5VAX.bam",
        "SEN-SMS-2019-045": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_045-SM-K5VAX.bam",
        "SEN-SMS-2019-046": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_046-SM-K5VAX.bam",
        "SEN-SMS-2019-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_047-SM-K5VAX.bam",
        "SEN-SMS-2019-048": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_048-SM-K5VAX.bam",
        "SEN-SMS-2019-049": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_049-SM-K5VAY.bam",
        "SEN-SMS-2019-050": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_050-SM-K5VAY.bam",
        "SEN-SMS-2019-051": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_051-SM-K5VAY.bam",
        "SEN-SMS-2019-052": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_052-SM-K5VAY.bam",
        "SEN-SMS-2019-053": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_053-SM-K5VAY.bam",
        "SEN-SMS-2019-054": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_054-SM-K5VAY.bam",
        "SEN-SMS-2019-055": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_055-SM-K5VAY.bam",
        "SEN-SMS-2019-057": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_057-SM-K5VAY.bam",
        "SEN-SMS-2019-058": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_058-SM-K5VAY.bam",
        "SEN-SMS-2019-059": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_059-SM-K5VAY.bam",
        "SEN-SMS-2019-060": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_060-SM-K5VAY.bam",
        "SEN-SMS-2019-062": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_062-SM-K5VAY.bam",
        "SEN-SMS-2019-064": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_SMS_2019_064-SM-K5VAY.bam",
        "SEN-TAM-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA1.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TAM_2019_002-SM-K5VAX.bam",
        "SEN-TAM-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA4.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA5.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA6.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA17.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA18.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-021": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA21.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-029": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA29.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-030": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA30.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-033": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA33.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-037": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TA37.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-040": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TA40.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-046": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TA46.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TA47.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-053": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TA53.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-054": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TAM_2019_054-SM-K5VAX.bam",
        "SEN-TAM-2019-055": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TA55.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-056": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TA56.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-058": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TA58.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-059": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA59.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-060": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA60.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-061": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA61.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-062": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA62.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-063": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA63.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-065": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA65.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-071": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA71.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-074": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA74.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-075": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_TA75.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-077": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TA77.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-080": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_TA80.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-082": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_TA82.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-083": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_TA83.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-086": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TAM_2019_086-SM-K5VAX.bam",
        "SEN-TAM-2019-088": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_TA88.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-090": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_TA90.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-091": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_TA91.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-092": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA92.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-093": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA93.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-094": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA94.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-095": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA95.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-097": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA97.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-098": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA98.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAM-2019-099": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TA99.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TAO-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TAO_2019_001-SM-K5VAY.bam",
        "SEN-TAO-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TAO_2019_002-SM-K5VAY.bam",
        "SEN-TAO-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TAO_2019_003-SM-K5VAY.bam",
        "SEN-TAO-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TAO_2019_004-SM-K5VAY.bam",
        "SEN-TAO-2019-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TAO_2019_005-SM-K5VAY.bam",
        "SEN-TAO-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TAO_2019_006-SM-K5VAY.bam",
        "SEN-TAO-2019-008": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TAO_2019_008-SM-K5VAY.bam",
        "SEN-TAO-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TAO_2019_010-SM-K5VAY.bam",
        "SEN-TBA-2019-001": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TBA_2019_001-SM-K5VAX.bam",
        "SEN-TBA-2019-003": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB3.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB4.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB6.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-008": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB8.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-009": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TBA_2019_009-SM-K5VAX.bam",
        "SEN-TBA-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB10.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-011": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB11.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB12.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-013": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB13.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-014": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB14.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB17.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB18.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB20.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB22.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-023": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB23.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-025": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB25.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-026": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB26.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-027": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB27.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-028": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB28.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-029": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB29.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-030": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB30.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-031": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB31.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-032": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB32.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-035": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB35.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-036": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB36.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-039": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB39.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-040": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB40.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-041": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB41.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-042": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB42.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-043": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TBA_2019_043-SM-K5VAX.bam",
        "SEN-TBA-2019-045": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB45.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB47.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-048": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB48.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-049": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB49.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-051": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB51.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-052": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB52.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-053": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB53.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-054": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB54.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-055": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/1_TB55.2019_HVWJMDSXX.1.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-056": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB56.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-057": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB57.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-058": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB58.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-059": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB59.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-060": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB60.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-061": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB61.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-062": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB62.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-063": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB63.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-065": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB65.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-066": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB66.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-067": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB67.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-068": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB68.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-069": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB69.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-070": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB70.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-071": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB71.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-072": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB72.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-075": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB75.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-077": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB77.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-079": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB79.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-081": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TB81.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-083": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB83.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-084": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB84.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-086": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB86.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-087": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB87.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBA-2019-089": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TB89.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-002": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TBK_2019_002-SM-K5VAX.bam",
        "SEN-TBK-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TO4.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-009": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TBK_2019_009-SM-K5VAX.bam",
        "SEN-TBK-2019-010": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TO10.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-011": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TBK_2019_011-SM-K5VAY.bam",
        "SEN-TBK-2019-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TO12.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-016": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TO16.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-018": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TO18.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-019": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_TO19.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TBK_2019_020-SM-K5VAY.bam",
        "SEN-TBK-2019-022": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO22.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-024": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO24.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-027": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TBK_2019_027-SM-K5VAY.bam",
        "SEN-TBK-2019-028": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO28.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-029": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO29.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-030": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_TBK_2019_030-SM-K5VAY.bam",
        "SEN-TBK-2019-032": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO32.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-034": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO34.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-036": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO36.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-040": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO40.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-041": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO41.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-043": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO43.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-046": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO46.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO47.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-TBK-2019-049": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_TO49.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-004": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_VG4.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-005": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_VG5.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-006": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_VG6.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-008": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG8.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-009": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG9.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-011": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG11.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-012": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG12.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-014": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG14.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-015": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG15.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-017": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG17.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-020": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG20.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-024": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG24.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-026": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG26.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-027": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG27.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-028": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG28.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-029": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG29.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-031": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG31.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-032": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG32.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-034": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG34.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-036": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG36.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-037": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG37.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-038": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG38.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-039": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG39.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-040": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG40.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-045": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG45.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-046": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG46.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-047": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG47.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-048": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_VLG_2019_048-SM-K5VAX.bam",
        "SEN-VLG-2019-050": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_VG50.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-051": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_VG51.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-052": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_VG52.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-057": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_VLG_2019_057-SM-K5VAX.bam",
        "SEN-VLG-2019-061": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_VG61.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-062": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_VG62.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-064": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_VG64.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-071": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_VLG_2019_071-SM-K5VAX.bam",
        "SEN-VLG-2019-078": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_VG78.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-079": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/3_VG79.2019_HVWJMDSXX.3.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-080": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG80.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-082": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG82.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-083": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/PDO_22020_03Nov2020/RP-2140.SEN_VLG_2019_083-SM-K5VAX.bam",
        "SEN-VLG-2019-084": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG84.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-086": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG86.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-088": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG88.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-090": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG90.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-093": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG93.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-096": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/4_VG96.2019_HVWJMDSXX.4.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-097": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_VG97.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-099": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_VG99.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-100": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_VG100.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-101": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_VG101.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-103": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_VG103.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-104": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_VG104.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
        "SEN-VLG-2019-105": "gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/HVWJMDSXX_2020FEB/2_VG105.2019_HVWJMDSXX.2.aligned.duplicates_marked.bam",
}

In [17]:
# Generate BAI paths:
name_to_bai_map = {k: v + ".bai" for k,v in name_to_bam_map.items()}

# Create a dataframe containing only the samples we need to update:
df_for_upload = t[t["sample"].isin(name_to_bam_map.keys())]

# Create lists in the order the samples appear in the dataframe
# so that we can set the files correctly:
samples = list(df_for_upload["sample"])
input_bam_paths = [name_to_bam_map[s] for s in samples]
input_bai_paths = [name_to_bai_map[s] for s in samples]

# Update the dataframe with the new input bams / input bais:
df_for_upload = df_for_upload.assign(input_bam=input_bam_paths)
df_for_upload = df_for_upload.assign(input_bai=input_bai_paths)
df_for_upload = df_for_upload.rename(columns={"sample": "entity:sample_id"})

# pd.get_option("display.max_columns")  # == 20
pd.set_option("display.max_columns", None)
df_for_upload

,entity:sample_id,ENA,additional_metadata,aligned_bai,aligned_bam,aligned_est_fold_cov,aligned_frac_bases,aligned_num_bases,aligned_num_reads,aligned_read_length,average_identity,collection_site,collection_site_latitude,collection_site_longitude,country,country_latitude,country_longitude,dataset_provenance,fastqc_report,fq1,fq2,fq_unpaired,input_bai,input_bam,input_fq_end1,input_fq_end2,insert_size_average,insert_size_standard_deviation,num_bases,num_reads,num_reads_Q10,num_reads_Q12,num_reads_Q15,num_reads_Q5,num_reads_Q7,participant,pct_callable,pct_properly_paired_reads,population,qc_pass,raw_est_fold_cov,read_length,read_qual_mean,read_qual_median,sample_type,study,unaligned_bai,unaligned_bam,year
20962,SEN-DEG-2019-011,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,58.810537,0.821964,1372216800.0,11772116.0,150.0,99.38176,Deggo,14.7559,-17.3961,Senegal,14.359109,-14.470363,Broad 2019 Senegal Dataset,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,,,0.0,0.0,1765817400.0,11772116.0,11772116.0,11772116.0,11771940.0,11772116.0,11772116.0,SEN-DEG-2019-011,,0.0,,,75.679492,150.0,29.059776,29.48,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,2019.0
20963,SEN-DEG-2019-012,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,118.585329,0.87192,2766932400.0,22125214.0,150.0,99.515348,Deggo,14.7559,-17.3961,Senegal,14.359109,-14.470363,Broad 2019 Senegal Dataset,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,,,0.0,0.0,3318782100.0,22125214.0,22125214.0,22125214.0,22125108.0,22125214.0,22125214.0,SEN-DEG-2019-012,,0.0,,,142.236532,150.0,29.062754,29.466667,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,2019.0
20964,SEN-DEG-2019-013,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,69.584087,0.838745,1623594300.0,13486730.0,150.0,99.473934,Deggo,14.7559,-17.3961,Senegal,14.359109,-14.470363,Broad 2019 Senegal Dataset,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,,,0.0,0.0,2023009500.0,13486730.0,13486730.0,13486730.0,13486642.0,13486730.0,13486730.0,SEN-DEG-2019-013,,0.0,,,86.702244,150.0,29.039447,29.466667,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,2019.0
20965,SEN-DEG-2019-015,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,82.752227,0.833548,1930844400.0,16332234.0,150.0,99.354416,Deggo,14.7559,-17.3961,Senegal,14.359109,-14.470363,Broad 2019 Senegal Dataset,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,,,0.0,0.0,2449835100.0,16332234.0,16332234.0,16332234.0,16331960.0,16332234.0,16332234.0,SEN-DEG-2019-015,,0.0,,,104.995157,150.0,28.958763,29.413333,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,2019.0
20966,SE

In [18]:
# Upload the new data:
# ltu.upload_root_table(namespace, workspace, df_for_upload)

In [ ]:
raise Exception("DO NOT RUN ALL CELLS!")

***
## Fix the input bams for the MalariaGEN Crosses:

In [43]:
ena_ids = t[t["dataset_provenance"] == "MalariaGEN Crosses"].ENA

input_bam_paths = [f"gs://broad-dsp-lrma-pfcrosses/bam/{e}.realigned.bam" for e in list(ena_ids)]
input_bai_paths = [f"gs://broad-dsp-lrma-pfcrosses/bam/{e}.realigned.bam.bai" for e in list(ena_ids)]

In [47]:
t.loc[t["dataset_provenance"] == "MalariaGEN Crosses", "input_bam"] = input_bam_paths
t.loc[t["dataset_provenance"] == "MalariaGEN Crosses", "input_bai"] = input_bai_paths

df_for_upload = t[t["dataset_provenance"] == "MalariaGEN Crosses"]
df_for_upload = df_for_upload.rename(columns={"sample": "entity:sample_id"})

In [48]:
df_for_upload

,entity:sample_id,ENA,additional_metadata,admin_level_1,admin_level_1_latitude,admin_level_1_longitude,aligned_bai,aligned_bam,aligned_est_fold_cov,aligned_frac_bases,...,qc_pass,raw_est_fold_cov,read_length,read_qual_mean,read_qual_median,sample_type,study,unaligned_bai,unaligned_bam,year
20864,PG0004-CW,ERR012788,,,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,83.758438,0.946003,...,,89.013203,76.0,27.765349,29.342105,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
20865,PG0008-CW,ERR012840,,,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,127.677054,0.966613,...,,132.791368,76.0,27.741893,29.342105,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
20866,PG0015-C,ERR019044,,,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,99.532675,0.819414,...,,124.110469,76.0,27.521692,29.605263,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
20867,PG0016-C,ERR012895,,,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,73.46958,0.963161,...,,76.925449,76.0,28.837532,29.868421,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
20868,PG0017-C,ERR019050,,,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,123.541922,0.945904,...,,132.315021,76.0,28.553442,29.868421,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20957,PG0109-C,ERR029405,,,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,64.239442,0.969169,...,,66.607905,76.0,28.863103,29.605263,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
20958,PG0111-C,ERR029409,,,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,129.093221,0.975613,...,,132.926201,76.0,29.133317,29.736842,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
20959,PG0111-CW,ERR045634,,,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,124.589832,0.981351,...,,127.525151,75.0,28.851663,29.6,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
20960,PG0112-C,ERR029406,,,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,135.852079,0.971934,...,,140.417618,76.0,28.776166,29.605263,,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,


In [49]:
# Upload the new data:
# ltu.upload_root_table(namespace, workspace, df_for_upload)

## Fix bam paths for 2020 samples with spaces in their names:


In [73]:
t = ltu.fetch_existing_root_table(namespace, workspace, "sample")
t = fix_nans(t)
t

,sample,ENA,additional_metadata,aligned_bai,aligned_bam,aligned_est_fold_cov,aligned_frac_bases,aligned_num_bases,aligned_num_reads,aligned_read_length,...,qc_pass,raw_est_fold_cov,read_length,read_qual_mean,read_qual_median,sample_type,study,unaligned_bai,unaligned_bam,year
0,FP0008-C,ERR1081237,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,33.385324,0.884333,778974400.0,8916670.0,100.0,...,True,38.21511,100.0,28.465702,29.2,gDNA,1147-PF-MR-CONWAY,,,2014
1,FP0009-C,ERR1081238,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,115.771497,0.971838,2701277700.0,28198096.0,100.0,...,True,120.851543,100.0,28.680673,29.3,gDNA,1147-PF-MR-CONWAY,,,2014
2,FP0010-CW,ERR2889621,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,267.359424,0.982429,6238254393.0,47021604.0,151.0,...,True,304.303398,151.0,26.407521,27.350993,sWGA,1147-PF-MR-CONWAY,,,2014
3,FP0011-CW,ERR2889624,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,229.001606,0.973595,5343257612.0,38554147.0,151.0,...,True,249.505694,151.0,26.63096,27.668874,sWGA,1147-PF-MR-CONWAY,,,2014
4,FP0012-CW,ERR2889627,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,262.616685,0.977577,6127592835.0,43682989.0,151.0,...,True,282.697332,151.0,26.77012,27.821192,sWGA,1147-PF-MR-CONWAY,,,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23296,SEN_2020_NDO.PD2_0054,,,,,,,,,,...,,,,,,,,,,2020
23297,SEN_2116_SES.PPS_0198,,,,,,,,,,...,,,,,,,,,,2116
23298,SEN_2020_NDO.PD3_0141,,,,,,,,,,...,,,,,,,,,,2020
23299,SEN_2020_NDO.PD3_0130,,,,,,,,,,...,,,,,,,,,,2020


In [109]:
df = t[t['dataset_provenance']=="Broad 2020 Senegal Dataset"]
input_bams_to_change = df[df['input_bam'].str.contains(" ")]["input_bam"]
fixed_input_bam_names = input_bams_to_change.str.replace(" ", "_")
df.loc[df['input_bam'].str.contains(" "), "input_bam"] = fixed_input_bam_names
df.loc[df['input_bai'].str.contains(" "), "input_bai"] = fixed_input_bam_names + ".bai"

# print(df[df["input_bam"].str.contains("__")]["input_bam"].iloc[0])
# print(df[df["input_bai"].str.contains("__")]["input_bai"].iloc[0])

df_for_upload = df_for_upload.rename(columns={"sample": "entity:sample_id"})
df_for_upload

,entity:sample_id,ENA,additional_metadata,aligned_bai,aligned_bam,aligned_est_fold_cov,aligned_frac_bases,aligned_num_bases,aligned_num_reads,aligned_read_length,...,qc_pass,raw_est_fold_cov,read_length,read_qual_mean,read_qual_median,sample_type,study,unaligned_bai,unaligned_bam,year
22685,SEN_2020_SES.PPS_0188,,,,,,,,,,...,,,,,,,,,,2020
22686,SEN_2020_SES.PPS_0111,,,,,,,,,,...,,,,,,,,,,2020
22687,SEN_2020_SES.PPS_0143,,,,,,,,,,...,,,,,,,,,,2020
22688,SEN_2020_SES.PPS_0151,,,,,,,,,,...,,,,,,,,,,2020
22689,SEN_2020_SES.PPS_0159,,,,,,,,,,...,,,,,,,,,,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23296,SEN_2020_NDO.PD2_0054,,,,,,,,,,...,,,,,,,,,,2020
23297,SEN_2116_SES.PPS_0198,,,,,,,,,,...,,,,,,,,,,2116
23298,SEN_2020_NDO.PD3_0141,,,,,,,,,,...,,,,,,,,,,2020
23299,SEN_2020_NDO.PD3_0130,,,,,,,,,,...,,,,,,,,,,2020


In [110]:
# Upload the new data:
# ltu.upload_root_table(namespace, workspace, df_for_upload)

## Looking at data in the `sample_set` table:

In [114]:
t = ltu.fetch_existing_root_table(namespace, workspace, "sample_set")
t = fix_nans(t)

In [128]:
df = t[t.drug_res_report != ""]
print(len(df))
print(len(df[df["annotated_vcf"] != ""]))
print(len(df[df["func_annotated_vcf"] != ""]))

1384
1101
357


In [122]:
t

,sample_set,ENA,additional_metadata,aligned_bai,aligned_bam,aligned_est_fold_cov,aligned_frac_bases,aligned_num_bases,aligned_num_reads,aligned_read_length_mean,...,participant,pct_properly_paired_reads,population,report,sample_type,samples,snpEff_genes,snpEff_summary,study,year
0,FP0008-C,ERR1081237,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,33.385324,0.884333,778974400.0,8916670.0,100.0,...,FP0008-C,83.6,AF-W,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gDNA,"{'itemsType': 'EntityReference', 'items': [{'e...",gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,1147-PF-MR-CONWAY,2014.0
1,FP0009-C,ERR1081238,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,115.771497,0.971838,2701277700.0,28198096.0,100.0,...,FP0009-C,91.4,AF-W,,gDNA,"{'itemsType': 'EntityReference', 'items': [{'e...",gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,1147-PF-MR-CONWAY,2014.0
2,FP0010-CW,ERR2889621,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,267.359424,0.982429,6238254393.0,47021604.0,151.0,...,FP0010-CW,89.2,AF-W,,sWGA,"{'itemsType': 'EntityReference', 'items': [{'e...",gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,1147-PF-MR-CONWAY,2014.0
3,FP0011-CW,ERR2889624,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,229.001606,0.973595,5343257612.0,38554147.0,151.0,...,FP0011-CW,91.1,AF-W,,sWGA,"{'itemsType': 'EntityReference', 'items': [{'e...",gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,1147-PF-MR-CONWAY,2014.0
4,FP0012-CW,ERR2889627,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,262.616685,0.977577,6127592835.0,43682989.0,151.0,...,FP0012-CW,86.3,AF-W,,sWGA,"{'itemsType': 'EntityReference', 'items': [{'e...",gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,1147-PF-MR-CONWAY,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21032,Pf7_qc_passed_chunk_9,,,,,,,,,,...,,,,,,"{'itemsType': 'EntityReference', 'items': [{'e...",,,,
21033,Pf7_qc_passed_chunk_10,,,,,,,,,,...,,,,,,"{'itemsType': 'EntityReference', 'items': [{'e...",,,,
21034,HQ_Assemblies_2018_Otto_et_al_Not_in_PF7,,,,,,,,,,...,,,,,,"{'itemsType': 'EntityReference', 'items': [{'e...",,,,
21035,Broad_2022_Senegal_Dataset_20231205,,,,,,,,,,...,,,,,,"{'itemsType': 'EntityReference', 'items': [{'e...",,,,


## Collect `sample_set` entries that have `func_annotated_vcf` but not `annotated_vcf`

In [132]:
t = ltu.fetch_existing_root_table(namespace, workspace, "sample_set")
t = fix_nans(t)

In [139]:
df = t[t["func_annotated_vcf"] != ""]
func_annotated_samples = set(df["sample_set"])

In [141]:
# ltu.add_one_set(ns=namespace, 
#             ws=workspace, 
#             etype="sample_set_set",
#             ename=f"2022_2019_func_annotated_samples",
#             member_type="sample_set",
#             members=func_annotated_samples,
#             attributes=None
#                )

## Create a set of all `sample_set`s that have the new drug resistance stuff defined:

In [145]:
t = ltu.fetch_existing_root_table(namespace, workspace, "sample_set")
t = fix_nans(t)
t

,sample_set,ENA,additional_metadata,aligned_bai,aligned_bam,aligned_est_fold_cov,aligned_frac_bases,aligned_num_bases,aligned_num_reads,aligned_read_length_mean,...,participant,pct_properly_paired_reads,population,report,sample_type,samples,snpEff_genes,snpEff_summary,study,year
0,FP0008-C,ERR1081237,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,33.385324,0.884333,778974400.0,8916670.0,100.0,...,FP0008-C,83.6,AF-W,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/s...,gDNA,"{'itemsType': 'EntityReference', 'items': [{'e...",gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,1147-PF-MR-CONWAY,2014.0
1,FP0009-C,ERR1081238,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,115.771497,0.971838,2701277700.0,28198096.0,100.0,...,FP0009-C,91.4,AF-W,,gDNA,"{'itemsType': 'EntityReference', 'items': [{'e...",gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,1147-PF-MR-CONWAY,2014.0
2,FP0010-CW,ERR2889621,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,267.359424,0.982429,6238254393.0,47021604.0,151.0,...,FP0010-CW,89.2,AF-W,,sWGA,"{'itemsType': 'EntityReference', 'items': [{'e...",gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,1147-PF-MR-CONWAY,2014.0
3,FP0011-CW,ERR2889624,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,229.001606,0.973595,5343257612.0,38554147.0,151.0,...,FP0011-CW,91.1,AF-W,,sWGA,"{'itemsType': 'EntityReference', 'items': [{'e...",gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,1147-PF-MR-CONWAY,2014.0
4,FP0012-CW,ERR2889627,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,262.616685,0.977577,6127592835.0,43682989.0,151.0,...,FP0012-CW,86.3,AF-W,,sWGA,"{'itemsType': 'EntityReference', 'items': [{'e...",gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,1147-PF-MR-CONWAY,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22001,SEN_2020_NDO.PD2_0054,,,,,,,,,,...,,,,,,"{'itemsType': 'EntityReference', 'items': [{'e...",,,,2020.0
22002,SEN_2116_SES.PPS_0198,,,,,,,,,,...,,,,,,"{'itemsType': 'EntityReference', 'items': [{'e...",,,,2116.0
22003,SEN_2020_NDO.PD3_0141,,,,,,,,,,...,,,,,,"{'itemsType': 'EntityReference', 'items': [{'e...",,,,2020.0
22004,SEN_2020_NDO.PD3_0130,,,,,,,,,,...,,,,,,"{'itemsType': 'EntityReference', 'items': [{'e...",,,,2020.0


In [151]:
sample_sets_with_expanded_reports = set(t[t['drug_res_report_expanded'] != ""]["sample_set"])

In [152]:
# Add the pf7 samples that pass QC:
ltu.add_one_set(ns=namespace, 
                ws=workspace, 
                etype="sample_set_set",
                ename="2022_2019_samples_with_expanded_drug_reports",
                member_type="sample_set",
                members=sample_sets_with_expanded_reports,
                attributes=None
               )

## Populate the QC Pass PF7 samples

In [6]:
qc_pass_pf7_samples = {"FP0008-C","FP0009-C","FP0010-CW","FP0011-CW","FP0012-CW","FP0013-CW","FP0014-CW","FP0015-C","FP0016-C","FP0017-C","FP0018-C","FP0019-CW","FP0020-C","FP0021-C","FP0022-C","FP0023-C","FP0024-C","FP0025-C","FP0028-C","FP0029-C","FP0030-C","FP0031-C","FP0032-C","FP0033-C","FP0034-C","FP0035-C","FP0037-CW","FP0038-C","FP0039-C","FP0040-C","FP0041-C","FP0042-C","FP0043-C","FP0044-C","FP0045-CW","FP0046-C","FP0047-C","FP0048-C","FP0050-CW","FP0051-C","FP0052-C","FP0054-C","FP0055-C","FP0056-C","FP0057-C","FP0058-C","FP0059-C","FP0061-C","FP0062-C","FP0063-C","FP0065-C","FP0066-C","FP0067-C","FP0068-C","FP0069-C","FP0073-C","FP0075-CW","FP0076-CW","FP0077-C","FP0078-C","FP0081-C","FP0082-C","FP0083-CW","FP0084-C","FP0089-C","FP0090-C","FP0092-C","FP0093-C","FP0094-CW","FP0095-C","FP0096-C","FP0097-C","FP0098-C","FP0099-CW","FP0100-CW","FP0101-C","FP0102-C","FP0103-C","FP0105-C","FP0112-C","FP0113-C","FP0114-C","FP0116-C","FP0117-C","FP0119-C","FP0120-C","FP0123-C","FP0127-C","FP0129-C","FP0132-CW","FP0133-CW","FP0136-C","PA0007-C","PA0008-C","PA0011-C","PA0012-C","PA0013-C","PA0014-C","PA0015-C","PA0016-C","PA0017-C","PA0018-C","PA0020-C","PA0021-C","PA0022-C","PA0026-C","PA0027-C","PA0028-C","PA0029-C","PA0030-C","PA0032-C","PA0034-C","PA0035-C","PA0036-C","PA0037-C","PA0038-C","PA0039-C","PA0040-C","PA0041-C","PA0042-C","PA0044-C","PA0045-C","PA0046-C","PA0047-C","PA0049-C","PA0050-C","PA0051-C","PA0052-C","PA0053-C","PA0054-C","PA0057-C","PA0060-C","PA0061-C","PA0063-C","PA0064-C","PA0065-C","PA0066-C","PA0067-C","PA0068-C","PA0069-C","PA0071-C","PA0072-C","PA0073-C","PA0074-C","PA0075-C","PA0078-C","PA0081-C","PA0084-C","PA0085-C","PA0087-C","PA0089-C","PA0091-C","PA0092-C","PA0093-C","PA0094-C","PA0095-C","PA0097-C","PA0098-C","PA0099-C","PA0100-C","PA0101-C","PA0102-C","PA0103-C","PA0104-C","PA0105-C","PA0134-C","PA0135-C","PA0136-C","PA0137-C","PA0138-C","PA0140-C","PA0141-C","PA0142-C","PA0144-C","PA0145-C","PA0146-C","PA0147-C","PA0148-C","PA0149-C","PA0150-C","PA0151-C","PA0152-C","PA0153-C","PA0155-C","PA0156-C","PA0157-C","PA0158-C","PA0159-C","PA0160-C","PA0161-C","PA0162-C","PA0163-C","PA0164-C","PA0165-C","PA0167-C","PA0168-C","PA0169-C","PA0171-C","PA0172-C","PA0173-C","PA0175-C","PA0176-C","PA0177-C","PA0178-C","PA0179-C","PA0180-C","PA0181-C","PA0182-C","PA0183-C","PA0184-C","PA0185-C","PA0186-C","PA0187-C","PA0188-C","PA0189-C","PA0191-C","PA0192-C","PA0193-C","PA0195-C","PA0196-C","PA0197-C","PA0198-C","PA0199-C","PA0200-C","PA0201-C","PA0202-C","PA0203-C","PA0204-C","PA0205-C","PA0206-C","PA0207-C","PA0208-C","PA0209-C","PA0210-C","PA0211-C","PA0212-C","PA0213-C","PA0214-C","PA0215-C","PA0216-C","PA0217-C","PA0218-C","PA0219-C","PA0220-C","PA0221-C","PA0222-C","PA0223-C","PA0224-C","PA0225-C","PA0226-C","PA0227-C","PA0229-C","PA0230-C","PA0231-C","PA0232-C","PA0233-C","PA0234-C","PA0237-C","PA0238-C","PA0239-C","PA0240-C","PA0243-C","PA0245-C","PA0246-C","PA0249-C","PA0251-C","PA0253-C","PA0254-C","PA0255-C","PA0256-C","PA0257-C","PA0260-C","PA0261-C","PA0262-C","PA0263-C","PA0266-C","PA0267-C","PA0268-C","PA0270-C","PA0281-CW","PA0283-CW","PA0284-CW","PA0290-C","PA0291-CW","PA0294-CW","PA0295-CW","PA0297-CW","PA0298-C","PA0299-CW","PA0300-C","PA0302-C","PA0303-CW","PA0305-CW","PA0306-CW","PA0307-CW","PA0312-CW","PA0313-CW","PA0318-CW","PA0321-CW","PA0325-CW","PA0329-CW","PA0330-CW","PA0331-CW","PA0332-CW","PA0334-CW","PA0337-C","PA0341-C","PA0342-CW","PA0344-CW","PA0345-CW","PA0346-CW","PA0347-CW","PA0349-C","PA0350-CW","PA0357-CW","PA0363-CW","PA0370-C","PA0371-C","PA0372-C","PA0373-C","PA0374-C","PA0375-C","PA0376-C","PA0377-C","PA0378-C","PA0379-C","PA0382-C","PA0383-C","PA0384-C","PA0387-C","PA0388-C","PA0389-C","PA0391-C","PA0392-C","PA0393-C","PA0394-C","PA0395-C","PA0396-C","PA0397-C","PA0398-C","PA0399-C","PA0400-C","PA0401-C","PA0402-C","PA0403-C","PA0404-C","PA0405-C","PA0406-C","PA0407-C","PA0409-C","PA0411-C","PA0413-C","PA0414-C","PA0417-C","PA0419-C","PA0423-C","PA0424-C","PA0425-C","PA0431-C","PA0438-C","PA0439-C","PA0440-C","PA0441-C","PA0442-C","PA0443-C","PA0444-C","PA0445-C","PA0446-C","PA0447-C","PA0448-C","PA0449-C","PA0450-C","PA0451-C","PA0453-C","PA0454-C","PA0455-C","PA0456-C","PA0457-C","PA0458-C","PA0459-C","PA0460-C","PA0461-C","PA0462-C","PA0463-C","PA0464-C","PA0465-C","PA0467-C","PA0468-C","PA0469-C","PA0470-C","PA0471-C","PA0472-C","PA0473-C","PA0474-C","PA0475-C","PA0478-C","PA0479-C","PA0480-C","PA0481-C","PA0482-C","PA0483-C","PA0484-C","PA0485-C","PA0486-C","PA0487-C","PA0488-C","PA0489-C","PA0490-C","PA0491-C","PA0492-C","PA0494-C","PA0496-C","PA0497-C","PA0498-C","PA0500-C","PA0501-C","PA0502-C","PA0503-C","PA0504-C","PA0505-C","PA0506-C","PA0507-C","PA0508-C","PA0513-C","PA0514-C","PA0516-C","PA0520-C","PA0521-C","PA0522-C","PA0523-C","PA0524-C","PA0525-C","PA0532-C","PA0534-C","PA0537-C","PA0540-C","PA0541-C","PA0542-C","PA0544-C","PA0550-C","PA0551-C","PA0554-C","PA0558-C","PA0561-C","PA0562-C","PA0563-C","PA0564-C","PA0572-C","PA0575-C","PA0576-C","PA0577-C","PA0586-C","PA0597-C","PA0599-C","PA0604-C","PA0611-C","PA0617-C","PA0618-C","PA0619-C","PA0620-C","PA0621-C","PA0622-C","PA0623-C","PA0625-C","PA0626-C","PA0627-C","PA0628-C","PA0629-C","PA0630-C","PA0631-C","PA0633-C","PA0634-C","PA0636-C","PA0637-C","PA0638-C","PA0639-C","PA0641-C","PA0642-C","PA0643-C","PA0645-C","PA0649-C","PA0654-C","PA0655-C","PA0677-CW","PA0699-CW","PA0809-CW","PA0895-CW","PA0899-CW","PA0900-CW","PA0916-CW","PA1029-CW","PA1038-CW","PA1048-CW","PA1050-CW","PA1051-CW","PA1052-CW","PA1061-CW","PA1062-CW","PA1313-Cw","PA1315-Cw","PA1316-Cw","PA1317-Cw","PA1318-Cw","PA1319-Cw","PA1320-Cw","PA1321-Cw","PA1323-Cw","PA1324-Cw","PA1326-Cw","PA1328-Cw","PA1330-Cw","PA1331-Cw","PA1332-Cw","PA1333-Cw","PA1336-Cw","PA1337-Cw","PA1338-Cw","PA1339-Cw","PA1340-Cw","PA1349-Cw","PA1362-Cw","PA1369-Cw","PA1376-Cw","PA1391-Cw","PA1406-Cw","PA1411-Cw","PA1426-Cw","PA1437-Cw","PA1446-Cw","PA1447-Cw","PA1448-Cw","PA1450-Cw","PA1452-Cw","PA1453-Cw","PA1456-Cw","PC0001-C","PC0003-02","PC0004-01","PC0005-Cx","PC0007-C","PC0008-01","PC0009-C","PC0011-C","PC0012-C","PC0013-C","PC0019-C","PC0023-C","PC0026-C","PC0030-C","PC0031-C","PC0032-C","PC0033-C","PC0044-C","PC0046-C","PC0047-C","PC0051-C","PC0053-C","PC0054-C","PC0057-C","PC0058-C","PC0061-C","PC0063-C","PC0064-C","PC0065-C","PC0066-C","PC0067-C","PC0068-C","PC0069-C","PC0070-C","PC0071-C","PC0073-C","PC0074-C","PC0075-C","PC0077-C","PC0078-C","PC0079-C","PC0085-C","PC0097-C","PC0103-C","PC0104-C","PC0107-C","PC0113-C","PC0118-C","PC0122-C","PC0167-C","PC0169-C","PC0170-C","PC0172-C","PC0173-C","PC0174-C","PC0178-C","PC0180-C","PC0183-C","PC0184-C","PC0186-C","PC0187-C","PC0188-C","PC0189-C","PC0191-C","PC0193-C","PC0197-C","PC0198-C","PC0199-C","PC0200-C","PC0201-C","PC0202-C","PC0206-C","PC0208-C","PC0209-C","PC0213-C","PC0220-C","PC0229-C","PC0231-C","PC0232-C","PC0233-C","PC0236-C","PC0245-C","PC0246-C","PC0252-C","PC0254-C","PC0255-C","PC0256-C","PC0257-C","PC0258-C","PC0259-C","PC0262-C","PC0263-C","PC0265-C","PC0268-C","PC0269-C","PC0273-C","PC0276-C","PC0277-C","PC0278-C","PC0282-C","PC0287-C","PC0288-C","PC0293-C","PC0295-C","PC0296-C","PC0301-C","PC0302-C","PC0305-C","PC0311-C","PC0314-C","PC0337-C","PC0352-C","PC4008-C","PC4009-C","PC4010-C","PC4011-C","PC4013-C","PC4014-C","PC4015-C","PC4016-C","PC4018-C","PC4019-C","PC4021-C","PC4024-C","PC4025-C","PC4026-C","PC4027-C","PC4028-C","PC4030-C","PC4031-C","PC4032-C","PC4033-C","PC4034-C","PC4035-C","PC4036-C","PC4037-C","PC4038-C","PC4039-C","PC4040-C","PC4041-C","PC4042-C","PC4043-C","PC4044-C","PC4045-C","PC4046-C","PC4047-C","PC4048-C","PC4049-C","PC4050-C","PC4051-C","PC4052-C","PC4054-C","PC4055-C","PC4056-C","PC4060-C","PC4061-C","PC4062-C","PC4066-C","PC4068-C","PC4074-C","PC4079-C","PC4083-C","PC4084-C","PC4085-C","PC4086-C","PC4088-C","PC4095-C","PC4098-C","PC4104-C","PC4114-C","PC4116-C","PC4117-C","PC4118-C","PC4126-C","PC4128-C","PC4129-C","PC4130-C","PD0003-C","PD0004-03","PD0004-C","PD0005-C","PD0006-01","PD0007-01","PD0008-02","PD0009-01","PD0011-01","PD0015-02","PD0016-C","PD0017-C","PD0018-C","PD0019-C","PD0020-C","PD0021-C","PD0022-C","PD0023-C","PD0025-C","PD0026-C","PD0027-C","PD0028-C","PD0029-C","PD0030-C","PD0031-C","PD0032-C","PD0033-C","PD0034-C","PD0037-C","PD0038-C","PD0040-C","PD0041-C","PD0042-C","PD0043-C","PD0045-C","PD0046-C","PD0048-C","PD0050-C","PD0052-C","PD0053-C","PD0056-C","PD0057-C","PD0058-C","PD0060-C","PD0061-C","PD0062-C","PD0063-C","PD0067-C","PD0068-C","PD0070-C","PD0072-C","PD0073-C","PD0074-C","PD0075-C","PD0076-C","PD0077-C","PD0078-C","PD0079-C","PD0080-C","PD0081-C","PD0082-C","PD0083-C","PD0084-C","PD0085-C","PD0086-C","PD0087-C","PD0088-C","PD0089-C","PD0090-C","PD0091-C","PD0092-C","PD0093-C","PD0094-C","PD0095-C","PD0096-C","PD0097-C","PD0098-C","PD0101-C","PD0102-C","PD0103-C","PD0105-C","PD0106-C","PD0109-C","PD0110-C","PD0111-C","PD0112-C","PD0113-C","PD0115-Cx","PD0116-Cx","PD0117-C","PD0118-C","PD0119-C","PD0121-C","PD0122-C","PD0123-C","PD0124-C","PD0125-C","PD0126-C","PD0127-C","PD0128-C","PD0129-C","PD0130-C","PD0131-C","PD0132-C","PD0133-C","PD0134-C","PD0135-C","PD0136-C","PD0137-C","PD0138-C","PD0139-C","PD0140-C","PD0141-C","PD0142-C","PD0143-C","PD0144-C","PD0145-C","PD0459-Cx","PD0460-C","PD0461-C","PD0462-C","PD0464-C","PD0466-C","PD0467-C","PD0469-C","PD0470-C","PD0471-C","PD0472-C","PD0473-C","PD0474-C","PD0475-C","PD0476-C","PD0477-C","PD0478-C","PD0480-C","PD0481-C","PD0482-C","PD0483-C","PD0484-C","PD0485-C","PD0487-C","PD0488-C","PD0489-C","PD0490-C","PD0491-C","PD0492-C","PD0493-C","PD0494-C","PD0495-C","PD0496-C","PD0497-C","PD0498-C","PD0499-C","PD0500-C","PD0501-C","PD0502-C","PD0503-C","PD0504-C","PD0505-C","PD0506-C","PD0507-C","PD0508-C","PD0509-Cx2","PD0510-C","PD0511-C","PD0512-C","PD0513-C","PD0514-C","PD0515-C","PD0516-C","PD0517-C","PD0518-C","PD0519-C","PD0520-C","PD0521-C","PD0522-C","PD0523-C","PD0524-C","PD0525-C","PD0526-C","PD0527-C","PD0528-C","PD0529-C","PD0530-C","PD0531-C","PD0532-C","PD0533-C","PD0534-C","PD0535-C","PD0536-C","PD0537-C","PD0538-C","PD0539-C","PD0540-C","PD0541-C","PD0543-C","PD0544-C","PD0545-C","PD0546-C","PD0547-C","PD0548-C","PD0549-C","PD0550-C","PD0552-C","PD0553-C","PD0554-C","PD0555-C","PD0556-C","PD0557-C","PD0558-C","PD0559-C","PD0561-C","PD0562-C","PD0563-C","PD0564-C","PD0565-C","PD0566-C","PD0567-C","PD0569-C","PD0570-C","PD0571-C","PD0572-C","PD0573-C","PD0574-C","PD0575-C","PD0576-C","PD0577-C","PD0578-C","PD0579-C","PD0580-C","PD0581-C","PD0582-C","PD0583-C","PD0584-C","PD0585-C","PD0586-C","PD0587-C","PD0589-C","PD0590-C","PD0591-C","PD0592-C","PD0626-C","PD0627-C","PD0629-C","PD0630-C","PD0632-C","PD0635-C","PD0636-C","PD0637-C","PD0638-C","PD0648-C","PD0651-C","PD0658-C","PD0659-C","PD0662-C","PD0663-C","PD0664-C","PD0667-C","PD0668-C","PD0670-C","PD0716-C","PD0717-C","PD0718-C","PD0720-C","PD0721-C","PD0722-C","PD0724-C","PD0725-C","PD0726-C","PD0727-C","PD0728-C","PD0729-C","PD0730-C","PD0731-C","PD0733-C","PD0734-C","PD0735-C","PD0740-C","PD0741-C","PD0742-C","PD0744-C","PD0745-C","PD0746-C","PD0747-C","PD0748-C","PD0749-C","PD0750-C","PD0751-C","PD0754-C","PD0755-C","PD0758-C","PD0760-C","PD0761-C","PD0762-C","PD0765-C","PD0766-C","PD0767-C","PD0771-C","PD0772-C","PD0774-C","PD0775-C","PD0776-C","PD0777-C","PD0781-C","PD0782-C","PD0785-C","PD0786-C","PD0787-C","PD0789-C","PD0790-C","PD0791-C","PD0792-C","PD0793-C","PD0794-C","PD0795-C","PD0796-C","PD0798-C","PD0799-C","PD0800-C","PD0801-C","PD0802-C","PD0803-C","PD0804-C","PD0806-C","PD0808-C","PD0809-C","PD0810-C","PD0813-C","PD0814-C","PD0816-C","PD0817-C","PD0818-C","PD0819-C","PD0820-C","PD0821-C","PD0822-C","PD0823-C","PD0824-C","PD0825-C","PD0826-C","PD0827-C","PD0828-C","PD0829-C","PD0830-C","PD0831-C","PD0832-C","PD0833-C","PD0834-C","PD0835-C","PD0836-C","PD0837-C","PD0838-C","PD0839-C","PD0840-C","PD0841-C","PD0842-C","PD0843-C","PD0845-C","PD0846-C","PD0847-C","PD0848-C","PD0849-C","PD0851-C","PD0852-C","PD0853-C","PD0854-C","PD0855-C","PD0856-C","PD0857-C","PD0858-C","PD0859-C","PD0860-C","PD0861-C","PD0862-C","PD0863-C","PD0864-C","PD0865-C","PD0867-C","PD0869-C","PD0870-C","PD0871-C","PD0872-C","PD0873-C","PD0874-C","PD0875-C","PD0876-C","PD0877-C","PD0878-C","PD0879-C","PD0881-C","PD0882-C","PD0883-C","PD0884-C","PD0885-C","PD0886-C","PD0887-C","PD0889-C","PD0890-C","PD0891-C","PD0892-C","PD0894-C","PD0895-C","PD0898-C","PD0900-C","PD0901-C","PD0904-C","PD0905-C","PD0906-C","PD0907-C","PD0908-C","PD0909-C","PD0910-C","PD0912-C","PD0913-C","PD0914-C","PD0915-C","PD0916-C","PD0918-C","PD0919-C","PD0920-C","PD0921-C","PD0923-C","PD0924-C","PD0925-C","PD0926-C","PD0927-C","PD0928-C","PD0929-C","PD0930-C","PD0931-C","PD0932-C","PD0933-C","PD0934-C","PD0935-C","PD0936-C","PD0937-C","PD0938-C","PD0939-C","PD0940-C","PD0941-C","PD0942-C","PD0943-C","PD0944-C","PD0945-C","PD0948-C","PD0949-C","PD0950-C","PD0951-C","PD0952-C","PD0953-C","PD0954-C","PD0955-C","PD0956-C","PD0957-C","PD0958-C","PD0959-C","PD0960-C","PD0961-C","PD0962-C","PD0963-C","PD0964-C","PD0966-C","PD0967-C","PD0968-C","PD0969-C","PD0971-C","PD0972-C","PD0973-C","PD0974-C","PD0976-C","PD0977-C","PD0978-C","PD0979-C","PD0980-C","PD0982-C","PD0983-C","PD0984-C","PD0985-C","PD0986-C","PD0987-C","PD0988-C","PD0989-C","PD0990-C","PD0992-C","PD0994-C","PD0995-C","PD0996-C","PD0997-C","PD0998-C","PD1000-C","PD1001-C","PD1003-C","PD1004-C","PD1005-C","PD1006-C","PD1007-C","PD1008-C","PD1009-C","PD1010-C","PD1011-C","PD1012-C","PD1013-C","PD1014-C","PD1015-C","PD1016-C","PD1017-C","PD1018-C","PD1019-C","PD1020-C","PD1021-C","PD1023-C","PD1024-C","PD1026-C","PD1027-C","PD1028-C","PD1029-C","PD1030-C","PD1031-C","PD1032-C","PD1033-C","PD1034-C","PD1036-C","PD1037-C","PD1038-C","PD1039-C","PD1040-C","PD1041-C","PD1042-C","PD1043-C","PD1044-C","PD1045-C","PD1046-C","PD1047-C","PD1048-C","PD1049-C","PD1050-C","PD1051-C","PD1052-C","PD1053-C","PD1054-C","PD1055-C","PD1056-C","PD1059-C","PD1060-C","PD1061-C","PD1062-C","PD1063-C","PD1064-C","PD1065-C","PD1066-C","PD1068-C","PD1069-C","PD1072-C","PD1073-C","PD1074-C","PD1075-C","PD1076-C","PD1077-C","PD1078-C","PD1079-C","PD1081-C","PD1082-C","PD1083-C","PD1084-C","PD1086-C","PD1087-C","PD1088-C","PD1090-C","PD1091-C","PD1092-C","PD1093-C","PD1095-C","PD1096-C","PD1097-C","PD1098-C","PD1099-C","PD1100-C","PD1101-C","PD1102-C","PD1103-C","PD1104-C","PD1105-C","PD1106-C","PD1107-C","PD1108-C","PD1109-C","PD1110-C","PD1111-C","PD1112-C","PD1113-C","PD1114-C","PD1116-C","PD1117-C","PD1118-C","PD1119-C","PD1120-C","PD1121-C","PD1122-C","PD1124-C","PD1125-C","PD1126-C","PD1127-C","PD1128-C","PD1129-C","PD1130-C","PD1131-C","PD1133-C","PD1134-C","PD1135-C","PD1136-C","PD1137-C","PD1138-C","PD1139-C","PD1140-C","PD1141-C","PD1142-C","PD1143-C","PD1144-C","PD1145-C","PD1146-C","PD1147-C","PD1148-C","PD1149-C","PD1150-C","PD1151-C","PD1153-C","PD1155-C","PD1156-C","PD1157-C","PD1158-C","PD1159-C","PD1160-C","PD1161-C","PD1162-C","PD1163-C","PD1164-C","PD1165-C","PD1166-C","PD1167-C","PD1168-C","PD1169-C","PD1170-C","PD1171-C","PD1172-C","PD1173-C","PD1174-C","PD1175-C","PD1176-C","PD1177-C","PD1178-C","PD1179-C","PD1180-C","PD1181-C","PD1182-C","PD1183-C","PD1184-C","PD1185-C","PD1187-C","PD1188-C","PD1189-C","PD1190-C","PD1191-C","PD1192-C","PD1193-C","PD1194-C","PD1195-C","PD1196-C","PD1197-C","PD1198-C","PD1199-C","PD1200-C","PD1201-C","PD1202-C","PD1203-C","PD1204-C","PD1205-C","PD1206-C","PD1208-C","PD1209-C","PD1210-C","PD1211-C","PD1212-C","PD1213-C","PD1214-C","PD1215-C","PD1216-C","PD1217-C","PD1218-C","PD1219-C","PD1220-C","PD1221-C","PD1222-C","PD1223-C","PD1224-C","PD1225-C","PD1226-C","PD1227-C","PD1228-C","PD1229-C","PD1230-C","PD1231-C","PD1232-C","PD1233-C","PD1234-C","PD1236-C","PD1237-C","PD1239-C","PD1240-C","PD1241-C","PD1243-C","PD1244-C","PD1246-C","PD1247-C","PD1249-C","PD1250-C","PD1251-C","PD1252-C","PD1253-C","PD1254-C","PD1255-C","PD1256-C","PD1257-C","PD1258-C","PD1259-C","PD1260-C","PD1261-C","PD1262-C","PD1263-C","PD1264-C","PD1265-C","PD1266-C","PD1267-C","PD1269-C","PD1270-C","PD1271-C","PD1273-C","PD1274-C","PD1276-C","PD1277-C","PD1278-C","PD1279-C","PD1280-C","PD1281-C","PD1282-C","PD1283-C","PD1284-C","PD1285-C","PD1286-C","PD1288-C","PD1289-C","PD1290-C","PD1291-C","PD1292-C","PD1293-C","PD1294-C","PD1295-C","PD1296-C","PD1297-C","PD1299-C","PD1300-C","PD1301-C","PD1302-C","PD1304-C","PD1305-C","PD1306-C","PD1307-C","PD1308-C","PD1309-C","PD1310-C","PD1311-C","PD1312-C","PD1313-C","PD1314-C","PD1318-C","PD1319-C","PD1322-C","PD1326-C","PD1328-C","PD1329-C","PD1330-C","PD1331-C","PD1332-C","PD1333-C","PD1335-C","PD1337-C","PD1338-C","PD1339-C","PD1340-C","PD1341-C","PD1343-C","PD1345-C","PD1347-C","PD1348-C","PD1350-C","PD1353-C","PD1354-C","PD1355-C","PD1357-C","PD1358-C","PD1360-C","PD1361-C","PD1364-C","PD1365-C","PD1366-C","PD1368-C","PD1373-C","PD1376-C","PD1379-C","PD1380-C","PD1382-C","PD1386-C","PD1387-C","PD1388-C","PD1390-C","PD1391-C","PD1392-C","PD1394-C","PD1397-C","PD1399-C","PD1400-C","PD1402-C","PD1403-C","PD1404-C","PD1405-C","PD1406-C","PD1407-C","PD1409-C","PD1411-C","PD1412-C","PD1413-C","PD1414-C","PD1415-C","PD1418-C","PD1419-C","PD1420-C","PD1421-C","PD1422-C","PD1426-C","PD1427-C","PD1428-C","PD1430-C","PD1434-C","PD1436-C","PD1437-C","PD1440-C","PD1443-C","PD1451-C","PD1457-C","PD1462-C","PD1464-C","PD1465-C","PD1466-C","PD1470-C","PD1472-C","PD1474-C","PD1475-C","PD1476-C","PD1478-C","PD1479-C","PD1480-C","PD1482-C","PD1483-C","PD1484-C","PD1485-C","PD1486-C","PD1487-C","PD1489-C","PD1490-C","PD1491-C","PD1493-C","PD1494-C","PD1496-C","PD1497-C","PD1498-C","PD1499-C","PD1501-C","PD1502-C","PD1503-C","PD1504-C","PD1505-C","PD1507-C","PD1508-C","PD1510-C","PD1511-C","PD1512-C","PD1513-C","PD1514-C","PD1515-C","PD1516-C","PD1518-C","PD1582-C","PD1589-C","PE0002-CW","PE0009-CW","PE0010-C","PE0011-C","PE0012-C","PE0013-C","PE0016-C","PE0017-CW","PE0018-C","PE0019-C","PE0020-C","PE0021-C","PE0022-C","PE0023-C","PE0024-C","PE0025-C","PE0027-C","PE0028-C","PE0030-C","PE0087-C","PE0088-C","PE0089-C","PE0090-C","PE0091-C","PE0092-C","PE0093-C","PE0096-C","PE0098-C","PE0102-C","PE0103-C","PE0105-C","PE0108-C","PE0109-C","PE0111-C","PE0113-C","PE0114-C","PE0115-C","PE0116-C","PE0117-C","PE0119-C","PE0121-C","PE0122-C","PE0123-C","PE0124-C","PE0125-C","PE0126-C","PE0127-C","PE0128-C","PE0129-C","PE0130-C","PE0131-C","PE0132-C","PE0133-C","PE0134-C","PE0135-C","PE0136-C","PE0137-C","PE0138-C","PE0139-C","PE0140-C","PE0141-C","PE0142-C","PE0143-C","PE0144-C","PE0145-C","PE0146-C","PE0147-C","PE0148-C","PE0151-C","PE0156-C","PE0157-C","PE0158-C","PE0159-C","PE0160-C","PE0161-C","PE0162-C","PE0163-C","PE0164-C","PE0166-C","PE0170-C","PE0172-C","PE0173-C","PE0177-C","PE0178-C","PE0179-C","PE0180-C","PE0181-C","PE0182-C","PE0184-C","PE0185-C","PE0186-C","PE0187-C","PE0189-C","PE0190-C","PE0191-C","PE0195-C","PE0197-C","PE0198-C","PE0199-C","PE0200-C","PE0201-C","PE0202-C","PE0203-C","PE0205-C","PE0206-C","PE0207-C","PE0208-C","PE0210-C","PE0212-C","PE0213-C","PE0215-C","PE0216-C","PE0218-C","PE0219-C","PE0221-C","PE0223-C","PE0224-C","PE0225-C","PE0226-C","PE0227-C","PE0228-C","PE0229-C","PE0230-C","PE0231-C","PE0232-C","PE0234-C","PE0237-C","PE0242-C","PE0244-C","PE0245-C","PE0247-C","PE0251-C","PE0252-C","PE0253-C","PE0254-C","PE0255-C","PE0256-C","PE0258-C","PE0259-C","PE0261-C","PE0262-C","PE0264-C","PE0265-C","PE0266-C","PE0267-C","PE0268-C","PE0269-C","PE0270-C","PE0271-C","PE0272-C","PE0273-C","PE0274-C","PE0275-C","PE0276-C","PE0278-C","PE0279-C","PE0281-C","PE0282-C","PE0283-C","PE0284-C","PE0285-C","PE0286-C","PE0288-C","PE0290-C","PE0291-C","PE0292-C","PE0294-C","PE0296-C","PE0297-C","PE0298-C","PE0299-C","PE0300-C","PE0301-C","PE0302-C","PE0303-C","PE0305-C","PE0307-C","PE0308-C","PE0309-C","PE0310-C","PE0311-C","PE0312-C","PE0314-C","PE0315-C","PE0317-C","PE0319-C","PE0327-C","PE0329-C","PE0330-C","PE0333-C","PE0335-C","PE0336-C","PE0337-C","PE0338-C","PE0339-C","PE0340-C","PE0341-C","PE0342-C","PE0343-C","PE0347-C","PE0348-C","PE0349-C","PE0350-C","PE0351-C","PE0354-C","PE0361-C","PE0362-C","PE0366-C","PE0368-C","PE0369-C","PE0370-C","PE0372-C","PE0376-C","PE0377-C","PE0378-C","PE0379-C","PE0382-C","PE0383-C","PE0385-C","PE0386-C","PE0387-C","PE0390-C","PE0391-C","PE0393-C","PE0395-C","PE0397-C","PE0399-C","PE0400-C","PE0401-C","PE0403-C","PE0405-C","PE0406-C","PE0409-C","PE0411-C","PE0412-C","PE0415-C","PE0416-C","PE0419-C","PE0420-C","PE0421-C","PE0423-C","PE0424-C","PE0425-C","PE0426-C","PE0429-C","PE0430-C","PE0431-C","PE0432-C","PE0434-C","PE0435-C","PE0436-C","PE0437-C","PE0438-C","PE0439-C","PE0440-C","PE0441-C","PE0442-C","PE0444-C","PE0445-C","PE0447-C","PE0448-C","PE0452-C","PE0453-C","PE0454-C","PE0456-C","PE0462-C","PE0465-C","PE0466-C","PE0467-C","PE0468-C","PE0469-C","PE0470-C","PE0471-C","PE0473-C","PE0476-C","PE0477-C","PE0479-C","PE0480-C","PE0482-C","PE0483-C","PE0485-C","PE0487-C","PE0490-C","PE0492-C","PE0493-C","PE0494-C","PE0496-C","PE0499-C","PE0500-C","PE0501-C","PE0504-C","PE0505-C","PE0508-C","PE0510-C","PE0512-C","PE0515-C","PE0516-C","PE0521-C","PE0529-C","PE0530-C","PE0532-C","PE0535-C","PE0537-C","PE0538-C","PE0540-C","PE0541-C","PE0542-C","PE0543-C","PE0544-C","PE0547-C","PE0548-C","PE0550-C","PE0553-C","PE0554-C","PE0555-C","PE0558-C","PF0004-C","PF0006-C","PF0007-C","PF0008-C","PF0009-C","PF0010-C","PF0011-C","PF0012-C","PF0016-C","PF0020-C","PF0021-C","PF0022-C","PF0024-C","PF0025-C","PF0026-C","PF0027-C","PF0028-C","PF0029-C","PF0030-C","PF0031-C","PF0033-C","PF0035-C","PF0036-C","PF0037-C","PF0038-C","PF0039-C","PF0040-C","PF0041-C","PF0042-C","PF0043-C","PF0044-C","PF0045-C","PF0046-C","PF0047-C","PF0048-C","PF0049-C","PF0050-C","PF0051-C","PF0052-C","PF0053-C","PF0054-C","PF0057-C","PF0058-C","PF0062-C","PF0063-C","PF0064-C","PF0065-C","PF0066-C","PF0069-C","PF0073-C","PF0074-C","PF0075-C","PF0078-C","PF0079-C","PF0080-C","PF0081-C","PF0084-C","PF0085-C","PF0089-C","PF0090-C","PF0092-C","PF0095-C","PF0096-C","PF0097-C","PF0098-C","PF0100-C","PF0101-C","PF0102-C","PF0103-C","PF0104-C","PF0105-C","PF0106-C","PF0107-C","PF0108-C","PF0109-C","PF0111-C","PF0112-C","PF0113-C","PF0114-C","PF0115-C","PF0116-C","PF0117-C","PF0118-C","PF0119-C","PF0121-C","PF0122-C","PF0123-C","PF0124-C","PF0125-C","PF0127-C","PF0128-C","PF0129-C","PF0131-C","PF0132-C","PF0133-C","PF0134-C","PF0135-C","PF0136-C","PF0137-C","PF0138-C","PF0140-C","PF0141-C","PF0142-C","PF0143-C","PF0144-C","PF0145-C","PF0146-C","PF0148-C","PF0149-C","PF0150-C","PF0151-C","PF0152-C","PF0153-C","PF0154-C","PF0155-C","PF0156-C","PF0157-C","PF0158-C","PF0159-C","PF0160-C","PF0161-C","PF0162-C","PF0163-C","PF0164-C","PF0167-C","PF0168-C","PF0169-C","PF0170-C","PF0172-C","PF0175-C","PF0176-C","PF0177-C","PF0178-C","PF0180-C","PF0184-C","PF0185-C","PF0186-C","PF0188-C","PF0189-C","PF0190-C","PF0191-C","PF0192-C","PF0193-C","PF0195-C","PF0196-C","PF0197-C","PF0198-C","PF0199-C","PF0202-C","PF0203-C","PF0204-C","PF0205-C","PF0206-C","PF0207-C","PF0208-C","PF0211-C","PF0212-C","PF0213-C","PF0214-C","PF0217-C","PF0218-C","PF0219-C","PF0221-C","PF0222-C","PF0223-C","PF0225-C","PF0226-C","PF0228-C","PF0229-C","PF0230-C","PF0231-C","PF0232-C","PF0233-C","PF0234-C","PF0235-C","PF0238-C","PF0239-C","PF0240-C","PF0241-C","PF0242-C","PF0245-C","PF0246-C","PF0248-C","PF0249-C","PF0250-C","PF0252-C","PF0253-C","PF0254-C","PF0255-C","PF0256-C","PF0258-C","PF0259-C","PF0260-C","PF0261-C","PF0262-C","PF0263-C","PF0264-C","PF0265-C","PF0266-C","PF0267-C","PF0268-C","PF0269-C","PF0270-C","PF0272-C","PF0274-C","PF0275-C","PF0276-C","PF0277-C","PF0278-C","PF0279-C","PF0280-C","PF0281-C","PF0282-C","PF0285-C","PF0286-C","PF0287-C","PF0288-C","PF0289-C","PF0290-C","PF0294-C","PF0298-C","PF0299-C","PF0301-C","PF0304-C","PF0305-C","PF0306-C","PF0308-C","PF0309-C","PF0310-C","PF0311-C","PF0314-C","PF0318-C","PF0321-C","PF0323-C","PF0325-C","PF0326-C","PF0327-C","PF0328-C","PF0329-C","PF0330-C","PF0331-C","PF0332-C","PF0333-C","PF0334-C","PF0335-CW","PF0336-CW","PF0337-C","PF0339-C","PF0341-C","PF0342-C","PF0344-C","PF0347-C","PF0349-C","PF0350-C","PF0353-C","PF0354-C","PF0355-C","PF0356-C","PF0357-C","PF0359-C","PF0361-C","PF0363-C","PF0364-C","PF0365-C","PF0366-C","PF0367-C","PF0368-C","PF0369-C","PF0370-C","PF0371-C","PF0372-C","PF0373-C","PF0374-C","PF0375-C","PF0377-C","PF0378-C","PF0380-C","PF0381-C","PF0383-C","PF0385-C","PF0386-C","PF0388-C","PF0389-C","PF0390-C","PF0392-C","PF0393-CW","PF0395-C","PF0396-C","PF0397-C","PF0398-C","PF0399-C","PF0400-C","PF0401-C","PF0404-C","PF0405-C","PF0406-C","PF0407-C","PF0408-C","PF0409-C","PF0410-CW","PF0411-C","PF0412-C","PF0413-C","PF0414-C","PF0415-C","PF0417-C","PF0418-C","PF0419-C","PF0421-C","PF0422-C","PF0423-C","PF0424-C","PF0425-C","PF0427-C","PF0428-CW","PF0430-C","PF0431-C","PF0432-C","PF0433-C","PF0434-C","PF0437-C","PF0438-CW","PF0439-C","PF0440-C","PF0441-CW","PF0442-C","PF0443-C","PF0444-CW","PF0445-C","PF0446-CW","PF0447-C","PF0448-C","PF0449-C","PF0450-C","PF0451-C","PF0452-C","PF0453-C","PF0454-C","PF0456-C","PF0458-C","PF0461-C","PF0463-C","PF0464-C","PF0465-C","PF0467-C","PF0469-C","PF0471-C","PF0472-C","PF0473-C","PF0476-C","PF0477-C","PF0478-C","PF0479-C","PF0480-C","PF0482-C","PF0484-C","PF0485-C","PF0486-C","PF0487-C","PF0488-C","PF0489-C","PF0491-C","PF0492-C","PF0493-C","PF0494-C","PF0499-C","PF0502-C","PF0504-C","PF0507-C","PF0510-C","PF0511-C","PF0512-C","PF0513-C","PF0514-C","PF0515-C","PF0518-C","PF0519-C","PF0522-C","PF0524-C","PF0525-C","PF0529-C","PF0531-C","PF0539-C","PF0542-C","PF0550-C","PF0556-C","PF0557-C","PF0559-C","PF0560-C","PF0562-C","PF0578-C","PF0579-C","PF0583-C","PF0584-C","PF0587-C","PF0590-C","PF0596-C","PF0597-C","PF0599-C","PF0603-C","PF0604-C","PF0606-C","PF0608-C","PF0609-C","PF0610-C","PF0612-C","PF0613-C","PF0614-C","PF0616-C","PF0617-C","PF0619-C","PF0620-C","PF0621-C","PF0624-C","PF0625-C","PF0627-C","PF0628-C","PF0629-C","PF0632-C","PF0633-C","PF0634-C","PF0635-C","PF0638-C","PF0641-C","PF0643-C","PF0647-C","PF0650-C","PF0651-C","PF0653-C","PF0655-C","PF0661-C","PF0663-C","PF0664-C","PF0666-C","PF0667-C","PF0668-C","PF0669-C","PF0670-C","PF0671-C","PF0672-C","PF0673-C","PF0674-C","PF0675-C","PF0676-C","PF0678-C","PF0679-C","PF0680-C","PF0681-C","PF0682-C","PF0683-C","PF0684-C","PF0685-C","PF0686-C","PF0687-C","PF0689-C","PF0690-C","PF0691-C","PF0692-C","PF0693-C","PF0694-C","PF0695-C","PF0696-C","PF0697-C","PF0698-C","PF0699-C","PF0700-C","PF0701-C","PF0703-C","PF0704-C","PF0706-C","PF0707-C","PF0708-C","PF0710-C","PF0711-C","PF0712-C","PF0715-C","PF0718-C","PF0720-C","PF0722-C","PF0723-C","PF0724-C","PF0725-C","PF0727-C","PF0728-C","PF0729-C","PF0730-C","PF0731-C","PF0732-C","PF0733-C","PF0734-C","PF0735-C","PF0736-C","PF0737-C","PF0738-C","PF0739-C","PF0740-C","PF0741-C","PF0742-C","PF0743-C","PF0744-C","PF0745-C","PF0747-C","PF0748-C","PF0749-C","PF0750-C","PF0751-C","PF0752-C","PF0753-C","PF0754-C","PF0755-C","PF0756-C","PF0757-C","PF0758-C","PF0759-C","PF0760-C","PF0761-C","PF0762-C","PF0763-C","PF0764-C","PF0765-C","PF0766-C","PF0767-C","PF0768-C","PF0769-C","PF0771-C","PF0772-C","PF0773-C","PF0775-C","PF0776-C","PF0777-C","PF0778-C","PF0780-C","PF0781-C","PF0782-C","PF0783-C","PF0784-C","PF0785-C","PF0786-C","PF0787-C","PF0789-C","PF0790-C","PF0791-C","PF0792-C","PF0793-C","PF0796-C","PF0798-C","PF0799-C","PF0800-C","PF0801-C","PF0802-C","PF0803-C","PF0804-C","PF0805-C","PF0806-C","PF0807-C","PF0808-C","PF0809-C","PF0811-C","PF0812-C","PF0814-C","PF0815-C","PF0816-C","PF0817-C","PF0818-C","PF0819-C","PF0820-C","PF0821-C","PF0822-C","PF0823-C","PF0825-C","PF0826-C","PF0827-C","PF0828-C","PF0829-C","PF0831-C","PF0832-C","PF0833-C","PF0834-C","PF0835-C","PF0836-C","PF0837-C","PF0838-C","PF0839-C","PF0840-C","PF0841-C","PF0842-C","PF0843-C","PF0844-C","PF0845-C","PF0846-C","PF0847-C","PF0848-C","PF0849-C","PF0850-C","PF0851-C","PF0852-C","PF0853-C","PF0854-C","PF0855-C","PF0856-C","PF0857-C","PF0858-C","PF0859-C","PF0860-C","PF0861-C","PF0862-C","PF0863-C","PF0865-C","PF0866-C","PF0867-C","PF0868-C","PF0869-C","PF0870-C","PF0872-C","PF0873-C","PF0874-C","PF0875-C","PF0876-C","PF0877-C","PF0878-C","PF0879-C","PF0881-C","PF0882-C","PF0883-C","PF0884-C","PF0885-C","PF0886-C","PF0887-C","PF0888-C","PF0889-C","PF0890-C","PF0891-C","PF0892-C","PF0893-C","PF0894-C","PF0895-C","PF0896-C","PF0897-C","PF0898-C","PF0899-C","PF0900-C","PF0901-C","PF0902-C","PF0903-C","PF0904-C","PF0905-C","PF0906-C","PF0907-C","PF0910-C","PF0911-C","PF0912-C","PF0913-C","PF0914-C","PF0915-C","PF0916-C","PF0917-C","PF0918-C","PF0919-C","PF0920-C","PF0921-C","PF0922-C","PF0923-C","PF0925-C","PF0926-C","PF0928-C","PF0929-C","PF0930-C","PF0932-C","PF0933-C","PF0934-C","PF0935-C","PF0936-C","PF0938-C","PF0939-C","PF0942-C","PF0943-C","PF0944-C","PF0946-C","PF0947-C","PF0948-C","PF0950-C","PF0951-C","PF0952-C","PF0953-C","PF0955-C","PF0956-C","PF0959-C","PF0960-C","PF0961-C","PF0962-C","PF0963-C","PF0964-C","PF0966-C","PF0967-C","PF0968-C","PF0969-C","PF0970-C","PF0972-C","PF0973-C","PF0974-C","PF0975-C","PF0976-C","PF0977-C","PF0978-C","PF0979-C","PF0980-C","PF0981-C","PF0982-C","PF0983-C","PF0984-C","PF0985-C","PF0986-C","PF0987-C","PF0988-C","PF0992-C","PF0997-C","PF0998-C","PF0999-C","PF1001-C","PF1004-C","PF1005-C","PF1006-C","PF1009-C","PF1010-C","PF1011-C","PF1012-C","PF1013-C","PF1014-C","PF1015-C","PF1016-C","PF1017-C","PF1018-C","PF1019-C","PF1020-C","PF1021-C","PF1022-C","PF1023-C","PF1024-C","PF1025-C","PF1026-C","PF1027-C","PF1028-C","PF1029-C","PF1030-C","PF1031-C","PF1032-Cx","PF1033-C","PF1034-C","PF1035-C","PF1036-C","PF1037-C","PF1038-C","PF1039-C","PF1040-C","PF1041-C","PF1042-C","PF1043-C","PF1044-C","PF1045-C","PF1046-C","PF1047-C","PF1048-C","PF1049-C","PF1050-C","PF1051-C","PF1052-C","PF1053-C","PF1054-C","PF1055-C","PF1056-C","PF1057-C","PF1058-C","PF1059-C","PF1060-C","PF1061-C","PF1062-C","PF1063-C","PF1064-C","PF1065-C","PF1066-Cx","PF1067-C","PF1068-C","PF1069-C","PF1070-C","PF1071-C","PF1072-C","PF1073-C","PF1074-Cx","PF1075-C","PF1076-C","PF1077-C","PF1078-C","PF1079-C","PF1080-C","PF1081-C","PF1082-C","PF1083-C","PF1084-C","PF1085-C","PF1086-C","PF1087-C","PF1088-C","PF1089-C","PF1090-C","PF1091-C","PF1092-C","PF1094-C","PF1095-C","PF1096-C","PF1097-C","PF1098-C","PF1099-C","PF1100-C","PF1101-C","PF1102-C","PF1103-C","PF1104-C","PF1105-C","PF1106-Cx","PF1107-C","PF1108-C","PF1110-C","PF1111-C","PF1112-C","PF1113-C","PF1114-C","PF1118-Cx","PF1119-Cx8","PF1120-Cx6","PF1121-Cx9","PF1123-Cx","PF1124-Cx","PF1125-Cx","PF1126-Cx","PF1127-Cx7","PF1128-Cx","PF1129-Cx9","PF1130-Cx6","PF1131-Cx9","PF1132-Cx","PF1134-Cx","PF1135-Cx","PF1136-Cx","PF1137-Cx","PF1139-Cx","PF1140-Cx","PF1141-Cx","PF1142-Cx","PF1143-Cx8","PF1148-Cx8","PF1150-Cx9","PF1151-Cx6","PF1152-Cx9","PF1153-Cx9","PF1154-Cx","PF1155-Cx","PF1157-Cx6","PF1157-Cx9","PF1160-Cx9","PF1161-Cx8","PF1165-Cx","PF1167-Cx","PF1169-Cx","PF1170-Cx","PF1171-Cx","PF1172-Cx","PF1173-Cx","PF1174-Cx","PF1175-Cx","PF1176-Cx","PF1177-Cx","PF1178-Cx","PF1179-Cx8","PF1180-Cx","PF1182-Cx","PF1184-Cx","PF1185-Cx","PF1187-Cx","PF1188-Cx","PF1189-Cx","PF1190-Cx","PF1194-Cx","PF1195-Cx","PF1196-Cx","PF1197-Cx8","PF1198-Cx8","PF1200-Cx","PF1220-CW","PF1226-CW","PF1227-CW","PF1228-CW","PF1229-CW","PF1230-CW","PF1233-CW","PF1235-CW","PF1236-CW","PF1239-CW","PF1240-CW","PF1246-CW","PF1247-CW","PF1251-CW","PF1252-CW","PG0607-C","PG0608-C","PG0609-C","PG0610-C","PG0611-C","PH0001-C","PH0002-C","PH0003-C","PH0004-C","PH0005-C","PH0006-C","PH0007-C","PH0008-C","PH0010-C","PH0011-C","PH0012-C","PH0013-C","PH0014-C","PH0015-C","PH0016-C","PH0017-C","PH0018-C","PH0019-C","PH0020-C","PH0021-C","PH0022-C","PH0023-C","PH0024-C","PH0026-C","PH0027-Cx","PH0028-Cx","PH0031-Cx","PH0042-C","PH0043-C","PH0044-C","PH0045-C","PH0046-C","PH0047-Cx","PH0051-Cx","PH0052-C","PH0054-C","PH0055-C","PH0056-C","PH0057-C","PH0058-C","PH0059-C","PH0060-C","PH0063-C","PH0064-C","PH0105-C","PH0108-CW","PH0109-C","PH0110-C","PH0111-C","PH0112-C","PH0114-C","PH0116-C","PH0117-C","PH0118-C","PH0119-C","PH0121-CW","PH0122-CW","PH0123-CW","PH0124-CW","PH0125-CW","PH0126-CW","PH0128-CW","PH0130-CW","PH0131-CW","PH0132-CW","PH0133-CW","PH0134-C","PH0136-C","PH0137-C","PH0138-CW","PH0139-CW","PH0140-CW","PH0141-CW","PH0142-CW","PH0143-CW","PH0144-CW","PH0145-CW","PH0146-CW","PH0147-CW","PH0148-CW","PH0149-CW","PH0150-CW","PH0151-CW","PH0152-CW","PH0153-CW","PH0155-C","PH0156-C","PH0157-C","PH0161-C","PH0163-C","PH0164-C","PH0165-C","PH0167-C","PH0169-C","PH0171-C","PH0172-C","PH0173-C","PH0191-C","PH0192-C","PH0194-C","PH0197-C","PH0198-C","PH0201-C","PH0203-C","PH0204-C","PH0205-C","PH0206-C","PH0207-C","PH0208-C","PH0209-C","PH0210-C","PH0212-C","PH0213-C","PH0217-C","PH0218-C","PH0221-C","PH0222-C","PH0223-CW","PH0224-C","PH0225-C","PH0227-C","PH0228-C","PH0229-C","PH0234-C","PH0236-C","PH0237-C","PH0238-C","PH0239-C","PH0240-C","PH0241-C","PH0242-C","PH0243-C","PH0244-C","PH0245-C","PH0246-C","PH0247-C","PH0248-C","PH0249-C","PH0250-C","PH0251-C","PH0252-C","PH0253-C","PH0256-C","PH0274-C","PH0278-CW","PH0279-C","PH0282-C","PH0283-C","PH0284-C","PH0285-C","PH0286-CW","PH0292-C","PH0294-C","PH0299-CW","PH0300-C","PH0302-C","PH0303-CW","PH0304-C","PH0305-C","PH0306-C","PH0307-C","PH0308-CW","PH0321-C","PH0322-C","PH0324-C","PH0325-C","PH0326-C","PH0327-C","PH0328-C","PH0329-C","PH0330-C","PH0332-C","PH0333-C","PH0334-C","PH0335-C","PH0336-C","PH0337-C","PH0338-C","PH0339-C","PH0340-C","PH0341-C","PH0342-C","PH0343-C","PH0344-C","PH0345-C","PH0346-C","PH0347-C","PH0348-C","PH0349-C","PH0350-C","PH0351-C","PH0352-C","PH0353-C","PH0354-C","PH0355-C","PH0356-C","PH0357-C","PH0358-C","PH0359-C","PH0360-C","PH0361-C","PH0362-C","PH0363-C","PH0364-C","PH0365-C","PH0366-C","PH0367-C","PH0368-C","PH0369-C","PH0370-C","PH0371-C","PH0372-C","PH0373-C","PH0374-C","PH0375-C","PH0376-C","PH0377-C","PH0378-C","PH0379-C","PH0380-C","PH0381-C","PH0382-C","PH0383-C","PH0384-C","PH0386-C","PH0387-C","PH0388-C","PH0389-C","PH0390-C","PH0391-C","PH0392-C","PH0393-C","PH0394-C","PH0395-C","PH0396-C","PH0397-C","PH0398-C","PH0400-C","PH0403-C","PH0404-C","PH0407-C","PH0408-C","PH0409-CW","PH0410-C","PH0411-C","PH0412-C","PH0413-C","PH0414-C","PH0415-C","PH0416-C","PH0418-C","PH0420-CW","PH0421-C","PH0422-CW","PH0424-C","PH0425-C","PH0431-CW","PH0437-C","PH0438-C","PH0444-C","PH0445-C","PH0447-C","PH0450-C","PH0451-C","PH0453-C","PH0454-C","PH0455-C","PH0456-C","PH0457-CW","PH0458-CW","PH0459-CW","PH0460-C","PH0461-C","PH0462-C","PH0463-C","PH0464-CW","PH0465-CW","PH0466-C","PH0467-CW","PH0468-C","PH0469-CW","PH0470-C","PH0471-C","PH0472-C","PH0473-C","PH0474-CW","PH0475-C","PH0476-C","PH0478-C","PH0479-CW","PH0480-C","PH0481-C","PH0482-C","PH0483-CW","PH0484-C","PH0485-C","PH0486-C","PH0487-C","PH0488-C","PH0489-C","PH0490-C","PH0493-C","PH0494-CW","PH0495-CW","PH0496-CW","PH0497-C","PH0498-C","PH0499-CW","PH0500-CW","PH0501-CW","PH0502-C","PH0503-C","PH0504-C","PH0505-C","PH0506-C","PH0507-C","PH0508-C","PH0510-C","PH0511-C","PH0513-C","PH0514-C","PH0515-C","PH0516-C","PH0517-C","PH0518-C","PH0519-C","PH0520-C","PH0521-C","PH0522-C","PH0523-C","PH0524-C","PH0525-C","PH0526-C","PH0527-C","PH0528-C","PH0529-C","PH0530-C","PH0531-C","PH0532-C","PH0533-C","PH0534-C","PH0535-C","PH0536-C","PH0537-C","PH0538-C","PH0539-C","PH0540-C","PH0541-C","PH0542-C","PH0543-C","PH0544-C","PH0545-C","PH0546-C","PH0547-C","PH0548-C","PH0549-C","PH0551-C","PH0552-C","PH0553-C","PH0554-C","PH0555-C","PH0556-C","PH0557-C","PH0558-C","PH0559-C","PH0560-C","PH0561-C","PH0562-C","PH0563-C","PH0564-C","PH0565-C","PH0566-C","PH0568-C","PH0569-C","PH0570-C","PH0571-C","PH0572-C","PH0573-C","PH0574-C","PH0576-C","PH0578-C","PH0579-C","PH0580-C","PH0582-C","PH0583-C","PH0584-C","PH0585-C","PH0586-C","PH0587-C","PH0588-C","PH0589-C","PH0590-C","PH0592-C","PH0593-C","PH0594-C","PH0595-C","PH0596-C","PH0597-C","PH0598-C","PH0599-C","PH0600-C","PH0601-C","PH0602-C","PH0603-C","PH0604-C","PH0605-C","PH0608-C","PH0609-C","PH0610-C","PH0611-C","PH0613-C","PH0614-C","PH0617-C","PH0619-C","PH0621-C","PH0623-C","PH0624-C","PH0625-C","PH0626-C","PH0627-C","PH0628-C","PH0629-C","PH0630-C","PH0631-C","PH0632-C","PH0633-C","PH0634-C","PH0635-C","PH0636-C","PH0637-C","PH0638-C","PH0639-C","PH0640-C","PH0641-C","PH0642-C","PH0643-C","PH0644-C","PH0645-C","PH0646-C","PH0647-C","PH0648-C","PH0650-C","PH0651-C","PH0676-C","PH0677-C","PH0678-C","PH0679-C","PH0680-C","PH0681-C","PH0682-C","PH0683-C","PH0684-C","PH0685-C","PH0686-C","PH0687-C","PH0692-C","PH0694-C","PH0695-C","PH0696-C","PH0697-C","PH0698-C","PH0700-C","PH0701-C","PH0702-C","PH0703-C","PH0704-C","PH0706-C","PH0707-C","PH0708-C","PH0709-C","PH0712-C","PH0713-C","PH0714-C","PH0715-C","PH0716-C","PH0717-C","PH0718-C","PH0719-C","PH0722-C","PH0723-C","PH0724-C","PH0725-C","PH0726-Cx","PH0727-C","PH0728-C","PH0729-C","PH0730-C","PH0731-C","PH0732-C","PH0733-C","PH0734-C","PH0735-C","PH0736-C","PH0737-C","PH0738-C","PH0739-C","PH0741-C","PH0742-C","PH0743-C","PH0744-C","PH0745-C","PH0746-C","PH0747-Cx","PH0749-C","PH0750-Cx","PH0752-C","PH0753-Cx","PH0754-C","PH0755-C","PH0756-Cx","PH0757-C","PH0758-C","PH0759-C","PH0760-C","PH0761-C","PH0762-Cx","PH0763-C","PH0764-C","PH0765-C","PH0766-Cx","PH0767-C","PH0768-C","PH0769-C","PH0770-C","PH0772-C","PH0773-Cx","PH0774-C","PH0775-Cx","PH0776-C","PH0777-C","PH0778-C","PH0779-Cx","PH0780-C","PH0782-C","PH0783-C","PH0784-C","PH0785-Cx","PH0786-Cx","PH0788-C","PH0789-C","PH0790-C","PH0793-C","PH0794-C","PH0795-C","PH0797-C","PH0800-C","PH0801-C","PH0802-C","PH0805-C","PH0807-C","PH0810-C","PH0811-C","PH0813-C","PH0814-Cx","PH0815-C","PH0817-C","PH0818-C","PH0819-C","PH0820-C","PH0821-C","PH0822-C","PH0823-C","PH0824-C","PH0825-C","PH0826-C","PH0828-C","PH0829-C","PH0830-C","PH0831-C","PH0832-C","PH0833-C","PH0834-C","PH0835-C","PH0836-C","PH0837-C","PH0838-C","PH0839-C","PH0840-C","PH0841-C","PH0842-C","PH0843-C","PH0844-C","PH0845-C","PH0846-C","PH0847-C","PH0848-C","PH0849-C","PH0850-C","PH0851-C","PH0852-C","PH0853-C","PH0854-C","PH0855-C","PH0856-C","PH0858-C","PH0860-Cx","PH0863-Cx","PH0865-C","PH0866-C","PH0868-Cx","PH0870-C","PH0871-C","PH0872-C","PH0873-Cx","PH0874-Cx","PH0875-C","PH0876-C","PH0877-C","PH0878-C","PH0879-C","PH0880-C","PH0882-C","PH0885-Cx","PH0886-C","PH0887-C","PH0888-C","PH0889-C","PH0890-C","PH0891-Cx","PH0892-C","PH0894-C","PH0895-C","PH0896-C","PH0898-C","PH0899-C","PH0900-C","PH0901-C","PH0902-C","PH0903-C","PH0904-C","PH0905-C","PH0906-C","PH0907-C","PH0908-C","PH0909-C","PH0910-C","PH0911-Cx","PH0912-C","PH0915-C","PH0916-C","PH0917-Cx","PH0918-Cx","PH0919-C","PH0920-Cx","PH0921-C","PH0923-Cx","PH0924-C","PH0925-C","PH0926-C","PH0927-C","PH0928-C","PH0929-C","PH0930-Cx","PH0931-C","PH0932-C","PH0933-C","PH0935-C","PH0936-C","PH0937-C","PH0938-C","PH0939-C","PH0940-Cx","PH0941-C","PH0942-Cx","PH0945-C","PH0952-CW","PH0959-Cx","PH0961-Cx","PH0962-C","PH0964-Cx","PH0966-Cx","PH0968-C","PH0970-C","PH0971-C","PH0973-C","PH0974-Cx","PH0976-C","PH0979-C","PH0980-C","PH0981-C","PH0982-C","PH0983-C","PH0984-C","PH0985-C","PH0986-C","PH0987-C","PH0991-C","PH0994-C","PH0996-C","PH0997-C","PH0999-C","PH1000-C","PH1001-C","PH1002-C","PH1003-C","PH1007-C","PH1015-C","PH1023-C","PH1025-C","PH1027-C","PH1032-C","PH1037-C","PH1043-C","PH1045-C","PH1051-C","PH1058-C","PH1068-C","PH1069-C","PH1071-C","PH1073-C","PH1085-C","PH1086-C","PH1087-C","PH1088-C","PH1089-C","PH1092-C","PH1097-C","PH1105-C","PH1106-C","PH1136-C","PH1137-C","PH1145-C","PH1147-C","PH1148-C","PH1150-C","PH1152-C","PH1153-C","PH1155-C","PH1158-C","PH1159-C","PH1160-C","PH1162-C","PH1163-C","PH1164-C","PH1165-C","PH1168-C","PH1169-C","PH1170-C","PH1172-C","PH1173-C","PH1175-C","PH1183-C","PH1186-C","PH1187-C","PH1188-C","PH1189-C","PH1190-C","PH1191-C","PH1192-C","PH1195-C","PH1197-C","PH1198-C","PH1200-C","PH1201-C","PH1202-C","PH1203-C","PH1205-C","PH1206-C","PH1208-C","PH1209-C","PH1211-C","PH1215-C","PH1216-C","PH1217-C","PH1218-C","PH1220-C","PH1223-C","PH1224-C","PH1229-C","PH1234-C","PH1236-C","PH1250-C","PH1251-C","PH1252-C","PH1253-C","PH1289-C","PH1297-C","PH1299-C","PH1300-C","PH1304-C","PH1305-C","PH1309-C","PH1310-C","PH1313-C","PH1316-C","PH1317-C","PH1318-C","PH1319-C","PH1328-C","PH1336-C","PH1337-C","PH1338-C","PH1339-C","PH1341-C","PH1343-C","PH1351-C","PH1354-C","PH1355-C","PH1357-C","PH1358-C","PH1360-C","PH1368-C","PH1373-C","PH1377-C","PH1378-C","PH1379-C","PH1384-C","PH1387-C","PH1390-C","PH1391-C","PH1393-C","PH1394-C","PH1395-C","PH1396-C","PH1397-C","PH1398-C","PH1399-C","PH1402-C","PH1407-C","PH1411-C","PH1419-C","PH1422-C","PH1439-C","PH1443-C","PH1449-C","PH1453-C","PH1454-C","PH1477-C","PH1480-C","PH1482-C","PH1484-C","PH1486-C","PH1487-C","PH1488-C","PH1489-C","PH1490-C","PH1491-C","PH1492-C","PH1493-C","PH1494-C","PH1495-C","PH1497-C","PH1498-C","PH1499-C","PH1500-C","PH1501-C","PH1502-C","PH1504-C","PH1506-C","PH1519-C","PH1529-C","PH1562-C","PH1564-C","PH1565-C","PH1566-C","PH1567-C","PH1568-C","PH1570-C","PH1571-C","PH1575-C","PH1577-C","PH1578-C","PH1580-C","PH1587-C","PH1589-C","PH1594-C","PH1596-C","PH1597-C","PH1602-C","PH1604-C","PH1611-C","PH1614-C","PH1615-C","PH1616-C","PH1622-C","PH1623-C","PH1624-C","PH1626-C","PH1628-C","PH1630-C","PH1634-C","PH1636-C","PH1638-C","PH1639-C","PH1643-C","PH1668-CW2","PH1669-CW2","PH1670-CW2","PH1671-CW2","PH1674-CW2","PH1675-CW2","PH1676-CW2","PH1677-CW2","PH1678-CW2","PH1679-CW2","PH1692-CW","PH1708-C","PH1709-C","PH1710-C","PH1711-C","PH1712-C","PH1713-C","PH1714-C","PH1715-C","PH1716-C","PH1717-C","PH1718-C","PH1719-C","PH1720-C","PH1721-C","PH1723-C","PH1729-C","PH1730-C","PH1731-C","PH1732-C","PH1733-C","PH1734-C","PH1735-C","PH1737-C","PH1738-C","PH1741-C","PH1742-C","PH1743-C","PH1744-C","PH1745-C","PH1746-C","PH1747-C","PH1748-C","PH1749-C","PH1750-C","PH1752-C","PH1753-C","PH1754-C","PH1756-C","PH1759-C","PH1760-C","PH1764-C","PH1765-C","PH1766-C","PH1767-C","PH1768-C","PJ0129-Cx","PJ0130-Cx","PJ0133-C","PJ0134-C","PJ0135-Cx","PJ0136-Cx","PJ0137-Cx","PJ0138-Cx","PJ0140-C","PJ0144-Cx","PJ0146-Cx","PJ0152-Cx","PJ0156-C","PJ0158-C","PJ0160-C","PJ0162-C","PJ0163-C","PJ0164-C","PJ0168-C","PJ0170-C","PJ0171-C","PJ0172-C","PJ0173-C","PJ0174-C","PJ0175-C","PJ0176-C","PJ0177-C","PJ0178-C","PJ0179-C","PJ0180-C","PJ0181-C","PJ0183-C","PJ0186-C","PJ0189-C","PJ0190-C","PJ0192-C","PJ0193-C","PJ0194-C","PJ0195-C","PJ0196-C","PJ0197-C","PJ0198-C","PJ0199-C","PJ0200-C","PJ0205-C","PJ0206-C","PJ0208-C","PJ0209-C","PJ0210-C","PJ0222-C","PJ0223-C","PJ0225-C","PJ0230-C","PJ0233-C","PJ0234-C","PJ0237-C","PJ0238-C","PJ0239-C","PJ0240-C","PJ0241-C","PJ0242-C","PJ0243-C","PJ0244-C","PJ0245-C","PJ0246-C","PJ0247-C","PJ0251-C","PJ0252-C","PJ0258-C","PJ0259-C","PJ0262-C","PJ0263-C","PJ0264-C","PJ0265-C","PJ0266-C","PJ0267-C","PJ0268-C","PJ0269-C","PJ0270-C","PJ0271-C","PJ0272-C","PK0002-C","PK0007-C","PK0009-C","PK0011-C","PK0012-C","PK0017-C","PK0018-C","PK0019-C","PK0020-C","PK0021-C","PK0023-C","PK0025-C","PK0026-C","PK0027-C","PK0028-C","PK0029-C","PK0030-C","PK0031-C","PK0034-C","PK0035-C","PK0036-C","PK0037-C","PK0038-C","PK0040-C","PK0041-C","PK0042-C","PK0043-C","PK0044-C","PK0045-C","PK0046-C","PK0047-C","PK0048-C","PK0049-C","PK0050-C","PK0051-C","PK0052-C","PK0053-C","PK0054-C","PK0055-C","PK0056-C","PK0059-C","PK0060-C","PK0061-C","PK0062-C","PK0063-C","PK0064-C","PK0065-C","PK0066-C","PK0067-C","PK0068-C","PK0069-C","PK0070-C","PK0072-C","PK0073-C","PK0074-C","PK0075-C","PK0076-C","PM0001-C","PM0002-C","PM0003-C","PM0004-C","PM0005-C","PM0006-C","PM0007-C","PM0008-C","PM0009-C","PM0014-C","PM0015-C","PM0017-C","PM0020-C","PM0021-C","PM0022-C","PM0023-C","PM0026-C","PM0033-C","PM0034-C","PM0036-C","PM0037-C","PM0038-C","PM0039-C","PM0040-C","PM0042-C","PM0047-C","PM0048-C","PM0049-C","PM0051-C","PM0054-C","PM0058-C","PM0060-C","PM0062-C","PM0063-C","PM0064-C","PM0068-C","PM0071-C","PM0072-C","PM0073-C","PM0074-C","PM0075-C","PM0076-C","PM0077-C","PM0078-C","PM0083-C","PM0084-C","PM0085-C","PM0086-C","PM0087-C","PM0089-C","PM0090-C","PM0093-C","PM0094-C","PM0096-C","PM0097-C","PM0098-C","PM0100-C","PM0103-C","PM0113-C","PM0114-C","PM0115-C","PM0117-C","PM0123-C","PM0124-C","PM0127-C","PM0129-C","PM0132-C","PM0133-C","PM0134-C","PM0136-C","PM0138-C","PM0139-C","PM0142-C","PM0144-C","PM0145-C","PM0146-C","PM0147-C","PM0151-C","PM0152-C","PM0153-C","PM0156-C","PM0157-C","PM0159-C","PM0160-C","PM0161-C","PM0182-C","PM0183-C","PM0184-C","PM0185-C","PM0186-C","PM0187-C","PM0188-C","PM0190-C","PM0191-C","PM0192-C","PM0193-C","PM0194-C","PM0195-C","PM0196-C","PM0197-C","PM0198-C","PM0199-C","PM0200-C","PM0201-C","PM0202-C","PM0203-C","PM0204-C","PM0205-C","PM0206-C","PM0207-C","PM0208-C","PM0209-C","PM0211-C","PM0212-C","PM0213-C","PM0214-C","PM0215-C","PM0216-C","PM0217-C","PM0218-C","PM0219-C","PM0220-C","PM0221-C","PM0222-C","PM0223-C","PM0224-C","PM0227-C","PM0228-C","PM0229-C","PM0230-C","PM0231-C","PM0232-C","PM0233-C","PM0234-C","PM0235-C","PM0236-C","PM0237-C","PM0238-C","PM0240-C","PM0241-C","PM0242-C","PM0243-C","PM0244-C","PM0246-C","PM0247-C","PM0248-C","PM0249-C","PM0250-C","PM0251-C","PM0252-C","PM0253-C","PM0254-C","PM0255-C","PM0256-C","PM0257-C","PM0258-C","PM0259-C","PM0260-C","PM0261-C","PM0262-C","PM0263-C","PM0264-C","PM0265-C","PM0266-C","PM0267-C","PM0268-C","PM0269-C","PM0270-C","PM0271-C","PM0272-C","PM0273-C","PM0274-C","PM0275-C","PM0276-C","PM0277-C","PM0278-C","PM0279-C","PM0280-C","PM0281-C","PM0282-C","PM0283-C","PM0290-C","PM0291-C","PM0293-C","PM0295-C","PM0296-C","PM0297-C","PM0299-C","PM0300-C","PM0301-C","PM0302-C","PM0303-C","PM0305-C","PM0306-C","PM0307-C","PM0308-C","PM0309-C","PM0310-C","PM0311-C","PM0312-C","PM0313-C","PM0314-C","PM0315-C","PM0316-C","PM0317-C","PM0318-C","PM0319-C","PM0320-C","PM0321-C","PM0322-C","PM0323-C","PM0324-C","PM0325-C","PM0326-C","PM0327-C","PM0328-C","PM0331-C","PM0332-C","PM0333-C","PM0334-C","PM0335-C","PM0336-C","PM0338-C","PM0339-C","PM0340-C","PM0341-C","PM0342-C","PM0343-C","PM0344-C","PM0345-C","PM0346-C","PM0347-C","PM0349-C","PM0350-C","PM0351-C","PM0352-C","PM0355-C","PM0356-C","PM0357-C","PM0358-C","PM0359-C","PM0360-C","PM0361-C","PM0363-C","PM0364-C","PM0365-C","PM0366-C","PM0367-C","PM0368-C","PM0369-C","PM0370-C","PM0372-C","PM0374-C","PM0375-C","PM0377-C","PM0378-C","PM0379-C","PM0380-C","PM0381-C","PM0382-C","PM0383-C","PM0384-C","PM0385-C","PM0386-C","PM0387-C","PM0390-C","PM0391-C","PM0392-C","PM0393-C","PM0394-C","PM0395-C","PM0396-C","PM0397-C","PM0398-C","PM0399-C","PM0400-C","PM0403-C","PM0405-C","PM0406-C","PM0408-C","PM0410-C","PM0411-C","PM0412-C","PM0413-C","PM0414-C","PM0415-C","PM0416-C","PM0417-C","PM0418-C","PM0419-C","PM0420-C","PM0421-C","PM0422-C","PM0423-C","PM0424-C","PM0425-C","PM0426-C","PM0427-C","PM0428-C","PM0429-C","PM0432-C","PM0433-C","PM0434-C","PM0435-C","PM0436-C","PM0437-C","PM0438-C","PM0440-C","PM0441-C","PM0442-C","PM0444-C","PM0445-C","PM0446-C","PM0447-C","PM0448-C","PM0449-C","PM0451-C","PM0452-C","PM0453-C","PM0454-C","PM0455-C","PM0456-C","PM0457-C","PM0458-C","PM0459-C","PM0460-C","PM0461-C","PM0463-C","PM0464-C","PM0466-C","PM0467-C","PM0468-C","PM0469-C","PM0470-C","PM0471-C","PM0472-C","PM0473-C","PM0475-C","PM0476-C","PM0478-C","PM0479-Cx","PM0480-C","PM0481-C","PM0482-C","PM0483-C","PM0484-C","PM0485-C","PM0486-C","PM0487-C","PM0488-C","PM0489-C","PM0490-C","PM0491-C","PM0492-C","PM0493-C","PM0494-C","PM0495-C","PM0496-C","PM0497-C","PM0498-C","PM0499-C","PM0500-C","PM0501-C","PM0502-C","PM0503-C","PM0504-C","PM0505-C","PM0506-C","PM0510-C","PM0511-C","PM0512-C","PM0513-C","PM0514-C","PM0517-C","PM0518-C","PM0519-CW","PM0520-CW","PM0521-CW","PM0522-CW","PM0523-CW","PM0524-CW","PM0525-CW","PM0526-CW","PM0527-CW","PM0528-CW","PM0529-C","PM0530-CW","PM0531-CW","PM0532-CW","PM0533-CW","PM0534-CW","PM0535-C","PM0536-CW","PM0537-CW","PM0538-CW","PM0539-CW","PM0540-CW","PM0541-CW","PM0542-CW","PM0543-CW","PM0544-C","PM0545-CW","PM0546-CW","PM0547-CW","PM0548-CW","PM0549-CW","PM0550-CW","PM0551-CW","PM0552-CW","PM0554-CW","PM0555-CW","PM0556-CW","PM0557-CW","PM0558-CW","PM0559-CW","PM0560-CW","PM0561-CW","PM0562-CW","PM0563-CW","PM0564-CW","PM0565-C","PM0566-C","PM0567-C","PM0568-C","PM0569-C","PM0570-C","PM0571-C","PM0572-C","PM0573-C","PM0574-C","PM0575-C","PM0576-C","PM0577-C","PM0578-C","PM0579-C","PM0580-C","PM0581-C","PM0582-C","PM0583-C","PM0584-C","PM0585-C","PM0586-C","PM0587-C","PM0588-C","PM0589-C","PM0590-C","PM0591-C","PM0592-C","PM0593-C","PM0594-C","PM0595-C","PM0596-C","PM0597-C","PM0598-C","PM0599-C","PM0600-C","PM0601-C","PM0602-C","PM0603-C","PM0604-C","PM0605-C","PM0606-C","PM0607-C","PM0608-C","PM0609-C","PM0610-C","PM0611-C","PM0612-C","PM0613-C","PM0695-CW","PM0696-CW","PM0700-CW","PM0704-CW","PM0705-CW","PM0711-CW","PM0713-CW","PM0714-CW","PM0716-CW","PM0718-CW","PM0719-CW","PM0723-CW","PM0724-CW","PM0725-CW","PM0727-CW","PM0733-CW","PM0735-CW","PM0738-CW","PM0739-CW","PM0743-CW","PM0744-CW","PM0745-CW","PM0746-CW","PM0748-CW","PM0752-CW","PM0753-CW","PM0755-CW","PM0758-CW","PM0759-CW","PM0763-CW","PM0765-CW","PM0766-CW","PM0769-CW","PM0772-CW","PM0774-CW","PM0775-CW","PM0776-CW","PM0777-CW","PM0782-CW","PM0784-CW","PM0788-CW","PM0791-CW","PM0793-CW","PM0794-CW","PM0801-CW","PM0802-CW","PM0803-CW","PM0804-C","PM0805-C","PM0806-C","PM0807-C","PM0808-C","PM0809-C","PM0810-C","PM0811-C","PN0003-C","PN0004-C","PN0005-C","PN0006-C","PN0007-C","PN0008-C","PN0010-C","PN0011-C","PN0012-C","PN0014-C","PN0015-C","PN0019-C","PN0020-C","PN0022-C","PN0023-C","PN0025-C","PN0031-C","PN0032-C","PN0037-C","PN0040-C","PN0041-C","PN0042-C","PN0043-C","PN0044-C","PN0054-C","PN0055-Cx","PN0056-C","PN0057-C","PN0058-Cx","PN0059-C","PN0063-C","PN0073-C","PN0074-C","PN0075-C","PN0077-C","PN0078-C","PN0079-C","PN0082-C","PN0083-C","PN0084-C","PN0085-C","PN0086-C","PN0087-C","PN0090-C","PN0097-C","PN0098-C","PN0099-C","PN0101-C","PN0102-C","PN0103-C","PN0104-C","PN0105-C","PN0107-C","PN0108-C","PN0109-C","PN0110-C","PN0111-C","PN0112-C","PN0113-C","PN0114-C","PN0115-C","PN0116-C","PN0117-C","PN0118-C","PN0119-C","PN0120-C","PN0121-C","PN0122-C","PN0124-C","PN0125-C","PN0126-C","PN0127-C","PN0128-C","PN0129-C","PN0130-C","PN0131-C","PN0133-C","PN0134-C","PN0135-C","PN0138-C","PN0140-C","PN0141-C","PN0142-C","PN0143-C","PN0145-C","PN0148-C","PN0149-C","PN0150-C","PN0151-C","PN0153-C","PN0154-C","PN0155-C","PN0156-C","PN0159-C","PN0161-C","PN0162-C","PN0163-C","PN0164-C","PN0165-C","PN0166-C","PN0168-C","PN0169-C","PN0170-C","PN0171-C","PN0172-C","PN0173-C","PN0176-C","PN0177-C","PN0178-C","PN0179-C","PN0180-C","PN0181-C","PN0182-C","PN0184-C","PN0185-C","PN0186-C","PN0187-C","PN0190-C","PN0191-C","PN0192-C","PP0002-C","PP0004-C","PP0005-C","PP0010-C","PP0011-C","PP0012-C","PP0013-C","PP0014-C","PP0015-C","PP0016-C","PP0017-C","PP0019-C","PP0020-C","PP0021-C","PP0023-C","PP0024-C","PP0025-C","PP0026-C","PP0027-C","PP0028-C","PP0029-C","PR0001-CW","PR0002-CW","PR0003-CW","PR0004-CW","PR0005-CW","PR0006-CW","PR0007-CW","PR0008-CW","PR0009-CW","PR0010-C","PR0012-C","PR0013-C","PR0014-CW","PR0015-C","PR0016-C","PR0017-C","PR0018-C","PR0019-C","PR0020-C","PR0022-CW","PR0024-CW","PR0032-C","PR0033-C","PR0039-C","PR0040-CW","PR0042-C","PR0043-C","PR0044-C","PR0045-C","PR0114-C","PR0115-C","PR0116-C","PR0117-C","PR0118-C","PR0119-C","PR0120-C","PR0121-C","PR0122-C","PR0123-C","PR0124-C","PR0125-C","PR0126-C","PR0127-C","PR0128-C","PR0129-C","PR0130-C","PR0131-C","PR0132-C","PR0133-C","PR0134-C","PR0135-C","PR0136-C","PR0137-C","PR0138-C","PR0141-C","PR0142-C","PR0143-C","PR0144-C","PR0145-C","PR0146-C","PR0147-C","PR0148-C","PR0149-C","PR0150-C","PR0151-C","PR0153-C","PR0154-C","PR0155-C","PR0156-C","PR0157-C","PR0158-C","PR0159-C","PR0160-C","PR0162-C","PR0163-C","PR0164-C","PR0165-C","PR0166-C","PR0262-C","PR0263-C","PR0264-C","PR0265-C","PR0266-C","PR0267-C","PR0268-C","PR0270-C","PR0271-C","PR0272-C","PR0273-C","PR0274-C","PR0275-C","PR0276-C","PR0277-C","PR0278-C","PR0279-C","PR0280-C","PR0281-C","PR0282-C","PR0283-C","PR0284-C","PR0285-C","PR0286-C","PR0287-C","PR0288-C","PR0289-C","PR0290-C","PR0291-C","PR0292-C","PT0001-CW","PT0002-CW","PT0003-CW","PT0004-CW","PT0005-CW","PT0006-CW","PT0007-CW","PT0008-CW","PT0009-CW","PT0010-CW","PT0013-CW","PT0014-CW","PT0015-CW","PT0016-CW","PT0017-CW","PT0018-CW","PT0020-CW","PT0021-CW","PT0023-CW","PT0027-C","PT0029-C","PT0031-C","PT0032-C","PT0034-C","PT0035-C","PT0038-C","PT0039-C","PT0040-C","PT0041-C","PT0042-C","PT0043-C","PT0044-C","PT0046-C","PT0047-C","PT0048-C","PT0051-C","PT0052-C","PT0053-C","PT0054-C","PT0055-C","PT0056-C","PT0057-C","PT0058-C","PT0059-C","PT0060-C","PT0061-C","PT0062-C","PT0063-C","PT0064-C","PT0065-C","PT0066-C","PT0067-C","PT0068-C","PT0069-C","PT0070-C","PT0071-C","PT0072-C","PT0073-C","PT0074-C","PT0075-C","PT0076-C","PT0077-Cx","PT0078-Cx","PT0079-C","PT0080-C","PT0081-C","PT0082-C","PT0083-C","PT0084-C","PT0085-C","PT0086-C","PT0087-C","PT0088-C","PT0089-C","PT0090-C","PT0091-C","PT0092-C","PT0094-C","PT0095-C","PT0096-C","PT0097-C","PT0098-C","PT0099-C","PT0100-C","PT0101-C","PT0102-C","PT0103-C","PT0104-C","PT0105-C","PT0106-C","PT0107-C","PT0108-C","PT0109-C","PT0111-C","PT0112-C","PT0113-C","PT0114-C","PT0115-C","PT0117-C","PT0118-C","PT0119-C","PT0120-C","PT0121-C","PT0122-C","PT0123-C","PT0124-C","PT0125-C","PT0126-C","PT0127-C","PT0129-C","PT0130-C","PT0131-C","PT0132-C","PT0133-C","PT0134-C","PT0135-C","PT0136-C","PT0137-C","PT0139-C","PT0140-C","PT0141-C","PT0142-C","PT0143-C","PT0144-C","PT0145-C","PT0146-C","PT0147-C","PT0148-C","PT0149-C","PT0150-C","PT0152-C","PT0155-C","PT0156-C","PT0157-C","PT0158-C","PT0160-C","PT0161-C","PT0162-C","PT0165-C","PT0166-C","PT0167-C","PT0168-C","PT0169-C","PT0170-C","PT0171-C","PT0172-C","PT0173-C","PT0174-C","PT0175-C","PT0176-C","PT0177-C","PT0178-C","PT0179-C","PT0180-C","PT0181-C","PT0182-C","PT0183-C","PT0184-C","PT0187-C","PT0188-C","PT0189-C","PT0190-C","PT0191-C","PT0192-C","PT0193-C","PT0194-C","PT0195-C","PT0196-C","PT0197-C","PT0198-C","PT0199-C","PT0200-C","PT0201-C","PT0202-C","PT0203-C","PT0204-C","PT0205-C","PT0206-C","PT0207-C","PT0208-C","PT0209-C","PT0210-C","PT0211-C","PT0212-C","PT0213-C","PT0214-C","PT0215-C","PT0216-C","PT0217-C","PT0218-C","PT0219-C","PT0220-C","PT0221-C","PT0222-C","PT0223-C","PT0224-C","PT0225-C","PT0226-C","PT0227-C","PT0228-C","PT0229-C","PT0230-C","PT0231-C","PT0232-C","PT0233-Cx","PT0234-C","PT0235-C","PT0236-C","PT0237-C","PT0238-C","PT0239-C","PT0240-C","PT0241-C","PT0242-C","PT0243-C","PT0244-C","PT0246-C","PT0247-C","PT0248-C","PT0249-C","PT0250-C","PT0252-C","PT0253-C","PT0254-C","PT0255-C","PT0256-C","PT0257-C","PT0258-C","PT0260-C","PT0261-C","PT0262-C","PT0263-C","PT0264-C","PT0266-C","PT0269-C","PT0270-C","PT0271-C","PT0273-C","PT0274-C","PT0275-C","PT0276-C","PT0277-C","PT0279-C","PT0280-C","PT0281-C","PT0282-C","PT0283-C","PT0285-C","PT0286-C","PT0287-C","PT0289-C","PT0290-C","PT0291-C","PT0292-C","PT0294-C","PT0295-C","PT0296-C","PT0297-C","PT0298-C","PT0299-C","PT0300-C","PT0301-C","PT0302-C","PT0303-C","PT0304-C","PV0002-C","PV0004-C","PV0006-C","PV0014-C","PV0015-C","PV0017-C","PV0018-C","PV0020-C","PV0023-C","PV0028-C","PV0030-C","PV0031-C","PV0035-C","PV0037-C","PV0039-C","PV0040-C","PV0041-C","PV0046-C","PV0048-C","PV0080-C","PV0081-C","PV0082-C","PV0083-C","PV0084-C","PV0085-C","PV0086-C","PV0087-C","PV0088-C","PV0089-C","PV0091-C","PV0092-C","PV0093-C","PV0094-C","PV0097-C","PV0098-C","PV0099-C","PV0100-C","PV0101-C","PV0102-C","PV0105-C","PV0109-C","PV0110-C","PV0112-C","PV0113-C","PV0114-C","PV0115-C","PV0116-C","PV0117-C","PV0118-C","PV0120-C","PV0122-C","PV0124-C","PV0125-C","PV0126-C","PV0127-C","PV0128-C","PV0129-C","PV0130-C","PV0132-C","PV0133-C","PV0134-C","PV0135-C","PV0137-C","PV0138-C","PV0139-C","PV0140-C","PV0141-C","PV0142-C","PV0144-C","PV0147-C","PV0148-C","PV0149-C","PV0151-C","PV0152-C","PV0153-C","PV0155-C","PV0156-C","PV0158-C","PV0160-C","PV0161-C","PV0167-C","PV0169-C","PV0170-C","PV0174-C","PV0176-C","PV0180-C","PV0182-C","PV0183-C","PV0184-C","PV0185-C","PV0186-C","PV0187-C","PV0188-C","PV0189-C","PV0192-C","PV0193-C","PV0194-C","PV0195-C","PV0197-C","PV0202-C","PV0204-C","PV0205-C","PV0206-C","PV0209-C","PV0210-C","PV0211-C","PV0212-C","PV0215-C","PV0216-C","PV0218-C","PV0219-C","PV0220-C","PV0221-C","PV0223-C","PV0224-C","PV0225-C","PV0226-C","PV0227-C","PV0228-C","PV0229-C","PV0230-C","PV0231-C","PV0233-C","PV0234-C","PV0235-C","PV0236-C","PV0237-C","PV0238-C","PV0239-C","PV0240-C","PV0241-C","PV0243-Cx2","PV0244-C","PV0245-Cx2","PV0246-C","PV0247-Cx2","PV0248-C","PV0249-Cx","PV0250-C","PV0251-C","PV0252-C","PV0254-C","PV0256-C","PV0257-C","PV0258-C","PV0259-C","PV0261-C","PV0262-C","PV0263-C","PV0264-C","PV0265-C","PV0266-C","PV0267-C","PV0268-C","PV0269-C","PV0270-C","PV0271-C","PV0272-C","PV0273-C","PV0274-C","PV0276-C","PV0277-C","PV0278-C","PV0279-C","PV0280-C","PV0281-C","PV0282-C","PV0283-C","PV0284-C","PV0285-C","PV0286-C","PV0287-C","PV0288-C","PV0289-C","PV0290-C","PV0291-C","PV0292-C","PV0293-C","PV0294-C","PV0295-C","PV0296-C","PV0297-C","PV0298-C","PV0299-C","PV0300-C","PV0301-C","PV0302-C","PV0303-C","PV0304-C","PV0305-C","PV0306-C","PV0308-Cx","PV0309-C","PV0310-C","PV0311-C","PV0312-C","PV0313-C","PV0314-C","PV0315-Cx","PV0317-C","PV0318-C","PV0319-C","PV0320-C","PV0321-C","PV0322-C","PV0323-C","PV0324-C","PV0325-C","PV0326-C","PV0328-C","PV0329-C","PV0330-C","PV0331-Cx","PV0333-C","PV0335-C","PV0336-C","PV0337-C","PV0338-C","PV0339-C","PV0340-C","PV0343-C","PV0344-C","PV0345-C","PV0347-C","PV0355-C","PV0358-C","PV0360-C","PV0361-C","PV0362-C","PV0363-C","PV0364-C","PV0365-C","PV0366-C","PV0367-C","PV0368-C","PV0369-C","PV0370-C","PV0372-C","PV0373-C","PV0374-C","PV0375-C","PV0376-C","PV0377-C","PV0378-C","PV0379-C","PV0380-C","PV0381-C","PV0382-C","PV0383-C","PV0384-C","PV0385-C","PV0386-C","PV0387-C","PV0388-C","PV0389-C","PV0393-C","PV0395-C","PV0396-C","PV0397-C","PV0398-C","PV0399-C","PV0400-C","PV0401-C","PV0402-C","PV0403-C","PW0001-C","PW0002-C","PW0003-C","PW0004-C","PW0005-C","PW0006-C","PW0007-C","PW0008-C","PW0009-C","PW0010-CW","PW0012-C","PW0013-C","PW0014-C","PW0015-C","PW0016-C","PW0017-C","PW0041-C","PW0042-C","PW0043-C","PW0044-C","PW0045-C","PW0046-C","PW0047-C","PW0048-C","PW0049-C","PW0050-C","PW0052-C","PW0053-C","PW0054-C","PW0055-C","PW0056-C","PW0057-C","PW0058-C","PW0059-C","PW0060-C","PW0061-C","PW0063-C","PW0064-C","PW0066-C","PW0067-C","PW0069-C","PW0073-C","PW0074-C","PW0075-C","PW0076-C","PW0077-C","PW0078-C","PW0079-C","PW0080-C","PW0083-C","PW0084-C","PW0085-C","PW0086-C","PW0087-C","PW0088-C","PW0089-C","PW0090-C","PW0091-C","PW0092-C","PW0093-C","PW0094-C","PW0095-C","PW0097-C","PW0098-C","PW0099-C","PW0100-C","PW0101-C","PW0102-C","PW0104-C","PW0105-C","PW0106-C","PZ0001-CW","PZ0002-CW","PZ0003-CW","PZ0004-CW","PZ0005-C","PZ0006-CW","PZ0007-C","PZ0008-CW","PZ0009-C","PZ0010-CW","PZ0011-CW","PZ0015-C","QC0111-C","QC0112-C","QC0113-C","QC0114-C","QC0115-C","QC0116-C","QC0118-C","QC0119-C","QC0120-C","QC0121-C","QC0122-C","QC0123-C","QC0125-C","QC0126-C","QC0127-C","QC0128-C","QC0129-C","QC0130-C","QC0131-C","QC0132-C","QC0133-C","QC0134-C","QC0135-C","QC0136-C","QC0137-C","QC0138-C","QC0139-C","QC0140-C","QC0141-C","QC0142-C","QC0143-C","QC0144-C","QC0145-C","QC0146-C","QC0147-C","QC0148-C","QC0149-C","QC0150-C","QC0151-C","QC0152-C","QC0153-C","QC0154-C","QC0155-C","QC0156-C","QC0157-C","QC0159-C","QC0160-C","QC0161-C","QC0162-C","QC0163-C","QC0164-C","QC0165-C","QC0166-C","QC0167-C","QC0168-C","QC0169-C","QC0170-C","QC0171-C","QC0172-C","QC0173-C","QC0174-C","QC0175-C","QC0176-C","QC0177-C","QC0178-C","QC0179-C","QC0180-C","QC0181-Cx","QC0182-C","QC0183-C","QC0184-C","QC0185-C","QC0186-C","QC0187-C","QC0188-C","QC0189-C","QC0193-C","QC0194-C","QC0195-C","QC0196-C","QC0198-C","QC0199-C","QC0200-C","QC0201-C","QC0204-C","QC0205-C","QC0206-C","QC0207-C","QC0209-C","QC0210-C","QC0211-C","QC0212-C","QC0213-C","QC0214-C","QC0215-C","QC0219-C","QC0220-C","QC0221-C","QC0222-C","QC0223-C","QC0225-C","QC0227-C","QC0228-C","QC0229-C","QC0230-C","QC0231-C","QC0232-C","QC0233-C","QC0235-C","QC0239-C","QC0242-C","QC0243-C","QC0244-C","QC0245-C","QC0246-C","QC0248-C","QC0250-C","QC0251-C","QC0253-C","QC0254-C","QC0255-C","QC0256-C","QC0257-C","QC0258-C","QC0259-C","QC0260-C","QC0261-C","QC0262-C","QC0263-C","QC0264-C","QC0266-C","QC0267-C","QC0268-C","QC0275-C","QC0276-C","QC0277-C","QC0278-C","QC0279-C","QC0281-C","QC0282-C","QC0283-C","QC0284-C","QC0285-C","QC0287-C","QC0289-C","QC0295-C","QC0296-C","QC0297-C","QC0298-C","QC0299-C","QC0300-C","QC0301-C","QC0302-C","QC0303-C","QC0305-C","QC0306-C","QC0307-C","QC0308-C","QC0309-C","QC0311-C","QC0313-C","QC0314-C","QC0318-C","QC0319-C","QC0320-C","QC0321-C","QC0322-C","QC0323-C","QC0324-C","QC0325-C","QC0326-C","QC0327-C","QC0328-C","QC0329-C","QC0330-C","QC0331-C","QC0332-C","QC0333-C","QC0334-C","QC0335-C","QC0336-C","QC0337-C","QC0338-C","QC0339-C","QC0340-C","QC0341-C","QC0342-C","QC0343-C","QC0344-C","QC0345-C","QC0346-C","QC0347-C","QC0348-C","QC0349-C","QC0350-C","QC0351-C","QC0353-C","QC0354-C","QC0357-C","QC0358-C","QC0359-C","QC0362-C","QC0363-C","QC0364-C","QC0365-C","QC0366-C","QC0367-C","QC0368-C","QC0369-C","QC0370-C","QC0372-C","QC0392-CW","QC0396-CW","QC0399-CW","QC0402-CW","QC0404-CW","QC0409-CW","QC0417-CW","QC0420-CW","QC0421-CW","QC0424-CW","QC0428-CW","QC0436-CW","QC0438-CW","QC0439-CW","QC0442-CW","QC0443-CW","QC0444-CW","QC0445-CW","QC0446-CW","QC0447-CW","QC0448-CW","QC0462-C","QC0471-CW","QC0477-CW","QC0479-CW","QC0481-CW","QC0485-CW","QC0487-CW","QC0488-CW","QC0489-CW","QC0490-CW","QC0491-CW","QC0494-CW","QC0497-CW","QC0504-CW","QC0505-CW","QC0506-CW","QC0507-CW","QC0523-CW","QC0524-CW","QC0530-CW","QC0534-C","QC0535-CW","QC0536-CW","QC0537-CW","QC0542-CW","QC0544-CW","QC0547-CW","QC0548-CW","QC0551-CW","QC0552-C","QC0554-CW","QC0557-CW","QC0558-CW","QC0563-CW","QC0564-CW","QC0572-CW","QC0573-CW","QC0574-CW","QC0576-CW","QC0581-CW","QC0599-CW","QC0603-CW","QC0604-CW","QC0606-CW","QC0607-CW","QC0608-CW","QC0609-C","QC0619-C","QC0620-C","QC0621-C","QC0622-C","QC0623-C","QC0624-C","QC0625-C","QC0626-C","QC0627-C","QC0628-C","QC0629-C","QC0630-C","QC0633-C","QC0634-C","QC0635-C","QC0636-C","QC0637-C","QC0638-C","QC0639-C","QC0640-C","QC0642-C","QC0643-C","QC0644-C","QC0645-C","QC0646-C","QC0647-C","QC0648-C","QC0649-C","QE0347-C","QE0348-C","QE0350-C","QE0352-C","QE0356-C","QE0360-C","QE0361-C","QE0362-C","QE0363-C","QE0364-C","QE0365-C","QE0366-C","QE0367-C","QE0368-C","QE0369-C","QE0370-C","QE0371-C","QE0372-C","QE0373-C","QE0374-C","QE0375-C","QE0376-C","QE0378-C","QE0379-C","QE0380-C","QE0381-C","QE0382-C","QE0383-C","QE0384-C","QE0385-C","QE0386-C","QE0387-C","QE0388-C","QE0389-C","QE0390-C","QE0391-C","QE0392-C","QE0393-Cx","QE0395-C","QE0396-C","QE0397-C","QE0398-C","QE0399-C","QE0400-C","QE0401-C","QE0402-C","QE0403-Cx","QE0404-C","QE0405-Cx","QE0407-C","QE0408-C","QE0409-C","QE0410-C","QE0411-C","QE0412-C","QE0413-C","QE0414-C","QE0415-C","QE0416-C","QE0417-C","QE0418-C","QE0419-C","QE0420-C","QE0421-C","QE0422-C","QE0423-C","QE0424-C","QE0425-C","QE0426-C","QE0427-C","QE0428-C","QE0429-C","QE0430-C","QE0431-C","QE0432-C","QE0433-C","QE0434-C","QE0435-C","QE0436-C","QE0437-C","QE0438-C","QE0439-C","QE0440-C","QE0441-C","QE0442-C","QE0443-C","QE0444-C","QE0445-C","QE0446-C","QE0447-C","QE0448-C","QE0449-C","QE0451-C","QE0452-C","QE0453-C","QE0454-C","QE0455-C","QE0456-C","QE0457-C","QE0458-C","QE0459-C","QE0460-C","QE0461-C","QE0463-C","QE0464-C","QE0465-C","QE0466-C","QE0467-C","QE0468-C","QE0469-C","QE0470-C","QE0471-C","QE0472-C","QE0474-C","QE0475-C","QE0476-C","QE0478-C","QE0479-C","QE0481-C","QE0483-C","QE0484-C","QE0485-C","QE0486-C","QE0487-C","QE0488-C","QE0489-C","QE0490-C","QG0007-C","QG0008-C","QG0009-C","QG0010-C","QG0011-C","QG0012-C","QG0013-C","QG0014-C","QG0015-C","QG0016-C","QG0017-C","QG0018-C","QG0019-C","QG0020-C","QG0021-C","QG0022-C","QG0023-C","QG0024-C","QG0025-C","QG0026-C","QG0027-C","QG0028-C","QG0029-C","QG0030-C","QG0031-C","QG0032-C","QG0033-C","QG0034-C","QG0035-C","QG0036-C","QG0037-C","QG0038-C","QG0039-C","QG0040-C","QG0041-C","QG0042-C","QG0043-C","QG0044-C","QG0045-C","QG0046-C","QG0047-C","QG0048-C","QG0049-C","QG0050-C","QG0051-C","QG0052-C","QG0053-C","QG0054-C","QG0055-C","QG0056-C","QG0057-C","QG0058-C","QG0059-C","QG0060-C","QG0061-C","QG0062-C","QG0063-C","QG0064-C","QG0065-C","QG0066-C","QG0067-C","QG0068-C","QG0069-C","QG0070-C","QG0071-C","QG0072-C","QG0073-C","QG0074-C","QG0075-C","QG0076-C","QG0077-C","QG0078-C","QG0080-C","QG0081-C","QG0082-C","QG0083-C","QG0084-C","QG0085-C","QG0086-C","QG0087-C","QG0088-C","QG0089-C","QG0090-C","QG0091-C","QG0092-C","QG0093-C","QG0094-C","QG0095-C","QG0096-C","QG0097-C","QG0098-C","QG0099-C","QG0100-C","QG0101-C","QG0102-C","QG0103-C","QG0104-C","QG0105-C","QG0106-C","QG0107-C","QG0108-C","QG0109-C","QG0110-C","QG0111-C","QG0112-C","QG0113-C","QG0114-C","QG0115-C","QG0116-C","QG0117-C","QG0118-C","QG0119-C","QG0120-C","QG0121-C","QG0122-C","QG0123-C","QG0124-C","QG0125-C","QG0126-C","QG0127-C","QG0128-C","QG0129-C","QG0130-C","QG0131-C","QG0132-C","QG0133-C","QG0134-C","QG0135-C","QG0136-C","QG0137-C","QG0138-C","QG0139-C","QG0140-C","QG0141-C","QG0142-C","QG0143-C","QG0144-C","QG0145-C","QG0146-C","QG0147-C","QG0148-C","QG0149-C","QG0150-C","QG0151-C","QG0152-C","QG0153-C","QG0154-C","QG0155-C","QG0156-C","QG0157-C","QG0158-C","QG0159-C","QG0160-C","QG0161-C","QG0162-C","QG0163-C","QG0164-C","QG0165-C","QG0166-C","QG0167-C","QG0168-C","QG0169-C","QG0171-C","QG0172-C","QG0173-C","QG0174-C","QG0175-C","QG0176-C","QG0178-C","QG0179-C","QG0180-C","QG0181-C","QG0182-C","QG0183-C","QG0184-C","QG0185-C","QG0186-C","QG0187-C","QG0188-C","QG0189-C","QG0190-C","QG0191-C","QG0192-C","QG0193-C","QG0194-C","QG0195-C","QG0196-C","QG0197-C","QG0198-C","QG0200-C","QG0201-C","QG0202-C","QG0203-C","QG0204-C","QG0205-C","QG0206-C","QG0207-C","QG0208-C","QG0209-C","QG0210-C","QG0211-C","QG0212-C","QG0213-C","QG0214-C","QG0215-C","QG0216-C","QG0217-C","QG0218-C","QG0219-C","QG0224-C","QG0225-C","QG0226-C","QG0228-C","QG0229-C","QG0231-C","QG0232-C","QG0233-C","QG0234-C","QG0235-C","QG0236-C","QG0237-C","QG0238-C","QG0239-C","QG0240-C","QG0241-C","QG0243-C","QG0244-C","QG0245-C","QG0246-C","QG0247-C","QG0248-C","QG0249-C","QG0250-C","QG0251-C","QG0252-C","QG0253-C","QG0254-C","QG0255-C","QG0256-C","QG0257-C","QG0258-C","QG0259-C","QG0260-C","QG0261-C","QG0262-C","QG0263-C","QG0264-C","QG0265-C","QG0266-C","QG0267-C","QG0269-C","QG0270-C","QG0271-C","QG0273-C","QG0274-C","QG0275-C","QG0276-C","QG0277-C","QG0278-C","QG0279-C","QG0280-C","QG0281-C","QG0282-C","QG0284-C","QG0285-C","QG0286-C","QG0287-C","QG0288-C","QG0289-C","QG0290-C","QG0291-C","QG0292-C","QG0293-C","QG0294-C","QG0295-C","QG0296-C","QG0297-C","QG0298-C","QG0299-C","QG0300-C","QG0303-C","QG0307-C","QG0312-C","QG0313-C","QG0314-C","QG0316-C","QG0320-C","QG0321-C","QG0325-C","QG0326-CW","QG0327-C","QG0328-C","QG0330-C","QG0331-CW","QG0332-CW","QG0335-C","QG0336-C","QG0337-C","QG0338-C","QG0340-C","QG0342-C","QG0344-C","QG0345-C","QG0347-C","QG0348-C","QG0350-C","QG0351-C","QG0352-C","QG0353-C","QG0354-C","QG0355-C","QG0356-C","QG0357-C","QG0358-C","QG0359-C","QG0360-C","QG0361-C","QG0362-C","QG0363-C","QG0364-C","QG0365-C","QG0366-C","QG0367-C","QG0368-C","QG0369-C","QG0370-C","QG0372-C","QG0375-C","QG0376-C","QG0377-C","QG0378-CW","QG0379-C","QG0380-C","QG0382-C","QG0383-C","QG0384-C","QG0385-C","QG0386-C","QG0388-C","QG0390-C","QG0393-C","QG0394-C","QG0395-C","QG0396-C","QG0397-C","QG0398-C","QG0399-C","QG0400-C","QG0401-C","QG0402-C","QG0403-C","QG0407-C","QG0408-C","QG0409-C","QG0411-C","QG0412-C","QG0413-C","QG0414-C","QG0415-C","QG0416-C","QG0418-C","QG0419-C","QG0421-C","QG0422-C","QG0423-C","QG0424-C","QG0425-C","QG0426-C","QG0427-C","QG0429-C","QG0430-C","QG0431-C","QG0432-C","QG0433-C","QG0434-C","QG0435-C","QG0436-C","QG0438-C","QG0439-C","QG0442-C","QG0443-C","QG0444-C","QG0445-C","QG0446-C","QG0447-C","QG0448-C","QG0449-C","QG0450-C","QG0451-C","QG0454-C","QG0456-C","QG0458-C","QG0459-C","QG0461-C","QG0462-C","QG0464-C","QG0467-C","QG0468-C","QG0470-C","QG0471-C","QG0476-C","QG0477-C","QG0479-C","QG0481-C","QG0483-C","QG0484-C","QG0485-C","QG0486-C","QG0488-C","QG0489-CW","QG0496-CW","QG0512-CW","QG0513-CW","QJ0001-C","QJ0002-C","QJ0003-C","QJ0004-C","QJ0005-C","QJ0006-C","QJ0007-C","QJ0008-C","QJ0009-C","QJ0010-C","QJ0015-C","QJ0018-C","QJ0019-C","QJ0094-C","QJ0159-C","QJ0160-C","QJ0161-C","QJ0162-C","QJ0163-C","QJ0164-C","QJ0165-C","QJ0167-C","QJ0168-C","QJ0169-C","QJ0170-C","QJ0172-C","QJ0173-C","QJ0174-C","QJ0175-C","QM0005-C","QM0006-C","QM0007-C","QM0008-C","QM0009-C","QM0011-C","QM0012-C","QM0013-C","QM0020-C","QM0027-C","QM0030-C","QM0035-C","QM0036-C","QM0037-C","QM0038-C","QM0041-C","QM0056-C","QM0057-C","QM0077-C","QM0121-C","QM0124-C","QM0125-C","QM0132-C","QM0134-C","QP0001-C","QP0002-C","QP0003-C","QP0004-C","QP0005-C","QP0006-C","QP0007-C","QP0008-C","QP0009-C","QP0010-C","QP0011-C","QP0012-C","QP0013-C","QP0014-C","QP0015-C","QP0016-C","QP0017-C","QP0018-C","QP0019-C","QP0020-C","QP0021-C","QP0022-C","QP0023-C","QP0024-C","QP0025-C","QP0026-C","QP0027-C","QP0028-C","QP0029-C","QP0030-C","QP0031-C","QP0032-C","QP0033-C","QP0034-C","QP0035-C","QP0036-C","QP0037-C","QP0038-C","QP0039-C","QP0040-C","QP0041-C","QP0042-C","QP0043-C","QP0044-C","QP0045-C","QP0047-C","QP0048-C","QP0050-C","QP0051-C","QP0054-C","QP0055-C","QP0056-C","QP0057-C","QP0058-C","QP0060-C","QP0061-C","QP0062-C","QP0063-C","QP0064-C","QP0065-C","QP0066-C","QP0068-C","QP0069-C","QP0071-C","QP0072-C","QP0073-C","QP0074-C","QP0075-C","QP0076-C","QP0077-C","QP0078-C","QP0079-C","QP0080-C","QP0081-C","QP0083-C","QP0084-C","QP0085-C","QP0086-C","QP0087-C","QP0088-C","QP0089-C","QP0090-C","QP0091-C","QP0092-C","QP0093-C","QP0094-C","QP0095-C","QP0096-C","QP0098-C","QP0099-C","QP0100-C","QP0101-C","QP0102-C","QP0103-C","QP0104-C","QP0105-C","QP0106-C","QP0107-C","QP0108-C","QP0110-C","QP0111-C","QP0112-C","QP0113-C","QP0114-C","QP0115-C","QP0116-C","QP0117-C","QP0118-C","QP0119-C","QP0120-C","QP0121-C","QP0122-C","QP0123-C","QP0124-C","QP0125-C","QP0126-C","QP0127-C","QP0128-C","QP0129-C","QP0130-C","QP0131-C","QP0132-C","QP0133-C","QP0134-C","QP0136-C","QP0137-C","QP0138-C","QP0139-C","QP0140-C","QP0142-C","QP0144-C","QP0145-C","QP0146-C","QP0147-C","QP0148-C","QP0149-C","QP0150-C","QP0151-C","QP0152-C","QP0153-C","QP0154-C","QP0155-C","QP0156-C","QP0157-C","QP0158-C","QP0159-C","QP0160-C","QP0161-C","QP0162-C","QP0163-C","QP0164-C","QP0165-C","QP0166-C","QP0168-C","QP0169-C","QP0170-C","QP0171-C","QP0172-C","QP0174-C","QP0175-C","QP0176-C","QP0177-C","QP0178-C","QP0179-C","QP0180-C","QP0181-C","QP0182-C","QP0183-C","QP0185-C","QP0186-C","QP0187-C","QP0188-C","QP0189-C","QP0190-C","QP0191-C","QP0192-C","QP0193-C","QP0194-C","QP0195-C","QP0196-C","QP0197-C","QP0198-C","QP0199-C","QP0200-C","QP0201-C","QP0202-C","QP0203-C","QP0204-C","QP0205-C","QP0206-C","QP0207-C","QP0208-C","QP0209-C","QP0210-C","QP0211-C","QP0212-C","QP0213-C","QP0214-C","QP0215-C","QP0217-C","QP0218-C","QP0219-C","QP0220-C","QP0222-C","QP0223-C","QP0224-C","QP0225-C","QP0226-C","QP0227-C","QP0228-C","QP0229-C","QP0230-C","QP0231-C","QP0232-C","QP0233-C","QP0234-C","QP0235-C","QP0236-C","QP0237-C","QP0238-C","QP0239-C","QP0240-C","QP0241-C","QP0242-C","QP0243-C","QP0244-C","QP0245-C","QP0246-C","QP0248-C","QP0249-C","QP0253-C","QP0254-C","QP0255-C","QP0258-C","QQ0002-C","QQ0003-C","QQ0004-C","QQ0005-C","QQ0006-C","QQ0008-C","QQ0009-C","QQ0010-C","QQ0012-C","QQ0013-C","QQ0014-C","QQ0024-C","QQ0027-C","QQ0031-C","QQ0032-C","QQ0033-C","QQ0039-C","QQ0043-C","QQ0047-C","QQ0048-C","QQ0049-C","QQ0050-C","QQ0051-C","QQ0053-C","QQ0054-C","QQ0055-C","QQ0056-C","QQ0058-C","QQ0059-C","QQ0061-C","QQ0063-C","QQ0065-C","QQ0066-C","QQ0068-C","QQ0070-C","QQ0071-C","QQ0073-C","QQ0074-C","QQ0075-C","QQ0076-C","QQ0078-C","QQ0079-C","QQ0081-C","QQ0082-C","QQ0083-C","QQ0084-C","QQ0088-C","QQ0089-C","QQ0090-C","QQ0092-C","QQ0095-C","QQ0096-C","QQ0097-C","QQ0099-C","QQ0100-C","QQ0102-C","QQ0103-C","QQ0105-C","QQ0109-C","QQ0110-C","QQ0111-C","QQ0112-C","QQ0113-C","QQ0114-C","QQ0115-C","QQ0116-C","QQ0117-C","QQ0118-C","QQ0119-C","QQ0120-C","QQ0121-C","QS0056-C","QS0104-C","QS0109-C","QS0110-C","QS0116-C","QS0126-C","QS0129-C","QS0132-C","QS0133-C","QS0135-C","QS0144-C","QS0154-C","QS0155-C","QS0156-C","QS0157-C","QS0159-C","QS0162-C","QS0163-C","QS0168-C","QS0169-C","QS0170-C","QT0001-C","QT0002-C","QT0004-C","QT0005-C","QT0006-C","QT0007-C","QT0008-C","QT0009-C","QT0010-C","QT0012-CW2","QT0014-C","QT0015-CW2","QT0016-C","QT0017-C","QT0018-C","QT0019-CW4","QT0020-CW2","QT0021-CW","QT0022-CW","QT0023-CW4","QT0024-C","QT0025-C","QT0026-CW4","QT0027-CW3","QT0028-CW5","QT0029-CW5","QT0030-CW3","QT0031-CW2","QT0032-CW2","QT0033-CW","QT0035-CW","QT0036-CW","QT0045-CW","QT0047-CW","QT0052-C","QT0054-C","QV0001-CW","QV0002-C","QV0003-CW","QV0004-C","QV0005-C","QV0006-CW","QV0008-C","QV0009-C","QV0010-C","QV0012-CW","QV0013-C","QV0014-C","QV0015-C","QV0016-C","QV0018-C","QV0019-C","QV0020-C","QV0021-C","QV0023-CW","QV0026-C","QV0028-C","QV0029-C","QV0030-C","QV0031-C","QV0032-C","QV0033-C","QV0034-CW","QV0035-C","QV0037-C","QV0038-C","QV0039-C","QV0040-C","QV0041-C","QV0042-C","QV0044-C","QV0045-C","QV0046-C","QV0047-C","QV0049-C","QV0051-C","QV0052-CW","QV0053-C","QV0056-C","QV0058-C","QV0059-C","QV0060-C","QV0061-C","QV0062-C","QV0064-C","QV0065-C","QV0067-C","QV0069-C","QV0070-C","QV0071-C","QV0072-C","QV0073-C","QV0074-C","QV0075-C","QV0076-C","QV0077-C","QV0078-C","QV0080-C","QV0081-C","QV0082-C","QV0083-C","QV0084-C","QV0085-C","QV0086-C","QV0088-C","QV0089-C","QV0090-C","QV0091-C","QV0092-C","QV0093-C","QV0094-C","QV0095-C","QV0096-C","QV0097-C","QV0099-C","QV0100-C","QV0101-C","QV0102-C","QV0103-C","QV0104-C","QV0105-C","QV0106-C","QV0107-C","QV0108-C","QV0109-C","QV0110-C","QV0112-C","QW0009-CW","QW0010-CxW","QW0014-CW","QW0016-CW","QW0018-CW","QW0019-CW","QW0020-CW","QW0021-CW","QW0022-CW","QW0023-CW","QW0025-CW","QW0027-CW","QW0031-CW","QW0033-CW","QW0035-CW","QW0038-CW","QW0041-CW","QW0043-CW","QW0046-CW","QW0049-CW","QW0050-CW","QW0057-CW","QW0058-CW","QW0064-CW","QW0067-CW","QW0071-CW","QW0072-CW","QW0074-CW","QW0076-CxW","QW0077-CxW","QW0081-CW","QW0083-CxW","QW0084-CxW","QW0087-CW","QW0088-CW","QW0089-CxW","QW0090-CxW","QW0091-CxW","QW0092-CxW","QW0096-CW","QW0100-CW","QW0101-CW","QW0107-CW","QW0109-CW","QW0115-CW","QW0118-CW","QW0119-CxW","QW0120-CxW","QW0125-CxW","QW0127-CW","QW0129-CW","QW0139-CxW","QW0144-CxW","QW0146-CW","QW0148-CW","RCN00121","RCN00122","RCN00123","RCN00124","RCN00125","RCN00126","RCN00127","RCN00129","RCN00130","RCN00131","RCN00132","RCN00135","RCN00136","RCN00138","RCN00140","RCN00142","RCN00143","RCN00145","RCN00146","RCN00147","RCN00148","RCN00149","RCN00150","RCN00151","RCN00152","RCN00154","RCN00157","RCN00159","RCN00160","RCN00161","RCN00162","RCN00163","RCN00164","RCN00165","RCN00168","RCN00170","RCN00172","RCN00173","RCN00177","RCN00178","RCN00179","RCN00180","RCN00181","RCN00183","RCN00184","RCN00187","RCN00188","RCN00193","RCN00196","RCN00197","RCN00206","RCN00207","RCN00208","RCN00209","RCN00211","RCN00213","RCN00215","RCN00217","RCN00219","RCN00220","RCN00222","RCN00223","RCN00224","RCN00225","RCN00226","RCN00227","RCN00228","RCN00229","RCN00232","RCN00233","RCN00234","RCN00235","RCN00236","RCN00237","RCN00238","RCN00240","RCN00241","RCN00242","RCN00243","RCN00246","RCN00247","RCN00250","RCN00252","RCN00253","RCN00254","RCN00257","RCN00258","RCN00260","RCN00261","RCN00262","RCN00263","RCN00265","RCN00266","RCN00267","RCN00268","RCN00269","RCN00270","RCN00271","RCN00272","RCN00273","RCN00274","RCN00275","RCN00276","RCN00277","RCN00278","RCN00279","RCN00280","RCN00281","RCN00282","RCN00283","RCN00284","RCN00285","RCN00287","RCN00288","RCN00289","RCN00290","RCN00291","RCN00292","RCN00293","RCN00294","RCN00295","RCN00296","RCN00297","RCN00712","RCN00714","RCN00715","RCN00716","RCN00720","RCN00721","RCN00722","RCN00724","RCN00726","RCN00729","RCN00733","RCN00740","RCN00741","RCN00745","RCN00747","RCN00751","RCN00752","RCN00753","RCN00754","RCN00755","RCN00758","RCN00759","RCN00760","RCN00761","RCN00762","RCN00763","RCN00764","RCN00765","RCN00766","RCN00768","RCN00770","RCN00771","RCN00773","RCN00776","RCN00778","RCN00779","RCN00780","RCN00781","RCN00782","RCN00785","RCN00787","RCN00788","RCN00790","RCN00792","RCN00793","RCN00794","RCN00795","RCN00796","RCN00798","RCN00799","RCN00802","RCN00804","RCN00805","RCN00807","RCN00808","RCN00810","RCN00811","RCN00812","RCN00813","RCN00816","RCN00817","RCN00820","RCN00824","RCN00825","RCN00829","RCN00830","RCN00831","RCN00832","RCN00833","RCN00834","RCN00835","RCN00837","RCN00839","RCN00840","RCN00842","RCN00843","RCN00844","RCN00845","RCN00846","RCN00847","RCN00848","RCN00849","RCN00850","RCN00852","RCN00853","RCN00854","RCN00855","RCN00857","RCN00858","RCN00859","RCN00861","RCN00862","RCN00868","RCN00869","RCN00870","RCN00871","RCN00872","RCN00875","RCN00876","RCN00877","RCN00878","RCN00879","RCN00881","RCN00883","RCN00884","RCN00885","RCN00886","RCN00887","RCN00894","RCN00895","RCN00896","RCN00898","RCN00899","RCN00900","RCN00903","RCN00904","RCN00905","RCN00906","RCN00912","RCN00913","RCN00915","RCN00916","RCN00917","RCN00920","RCN00924","RCN00926","RCN00927","RCN00929","RCN00932","RCN00933","RCN00934","RCN00936","RCN00937","RCN00938","RCN00939","RCN00942","RCN00944","RCN00945","RCN00950","RCN00952","RCN00954","RCN00955","RCN00956","RCN00957","RCN00959","RCN00961","RCN00966","RCN00968","RCN00972","RCN00973","RCN00974","RCN00977","RCN00978","RCN00979","RCN00980","RCN00981","RCN00984","RCN00985","RCN00986","RCN00988","RCN00989","RCN00990","RCN00991","RCN00992","RCN00993","RCN00994","RCN00995","RCN00996","RCN00997","RCN00999","RCN01000","RCN01001","RCN01002","RCN01003","RCN01004","RCN01005","RCN01006","RCN01007","RCN01008","RCN01009","RCN01010","RCN01011","RCN01012","RCN01013","RCN01014","RCN01015","RCN01016","RCN01017","RCN01018","RCN01019","RCN01020","RCN01021","RCN01022","RCN01023","RCN01024","RCN01025","RCN01026","RCN01027","RCN01028","RCN01029","RCN01032","RCN01033","RCN01034","RCN01035","RCN01037","RCN01038","RCN01040","RCN01043","RCN01045","RCN01046","RCN01047","RCN01049","RCN01050","RCN01052","RCN01053","RCN01054","RCN01055","RCN01056","RCN01057","RCN01058","RCN01059","RCN01060","RCN01061","RCN01062","RCN01063","RCN01064","RCN01066","RCN01067","RCN01069","RCN01070","RCN01071","RCN01072","RCN01073","RCN01074","RCN01075","RCN01076","RCN01078","RCN01080","RCN01081","RCN01084","RCN01085","RCN01086","RCN01087","RCN01088","RCN01090","RCN01091","RCN01092","RCN01093","RCN01095","RCN01096","RCN01097","RCN01100","RCN01101","RCN01102","RCN01103","RCN01104","RCN01106","RCN01107","RCN01108","RCN01109","RCN01110","RCN01111","RCN01112","RCN01113","RCN01114","RCN01116","RCN01117","RCN01119","RCN01120","RCN01121","RCN01122","RCN01124","RCN01125","RCN01126","RCN01127","RCN01128","RCN01130","RCN01131","RCN01133","RCN01134","RCN01137","RCN01138","RCN01139","RCN01140","RCN01141","RCN01142","RCN01144","RCN01775","RCN01776","RCN01777","RCN01778","RCN01779","RCN01780","RCN01781","RCN01782","RCN01783","RCN01784","RCN01785","RCN01786","RCN01787","RCN01788","RCN01789","RCN01859","RCN01860","RCN01861","RCN01862","RCN01863","RCN01864","RCN01865","RCN01867","RCN01868","RCN01869","RCN01870","RCN01871","RCN01872","RCN01873","RCN01874","RCN01875","RCN01876","RCN01877","RCN01878","RCN01879","RCN02007","RCN02008","RCN02009","RCN02010","RCN02011","RCN02012","RCN02013","RCN02014","RCN02015","RCN02016","RCN02017","RCN02018","RCN02019","RCN02020","RCN02021","RCN02022","RCN02024","RCN02025","RCN02026","RCN02027","RCN02028","RCN02030","RCN02031","RCN02032","RCN02033","RCN02035","RCN02038","RCN02040","RCN02043","RCN02044","RCN02045","RCN02046","RCN02047","RCN02048","RCN02049","RCN02050","RCN02051","RCN02052","RCN02053","RCN02054","RCN02056","RCN02058","RCN02060","RCN02061","RCN02062","RCN02063","RCN02064","RCN02065","RCN02066","RCN02070","RCN02071","RCN02072","RCN02074","RCN02075","RCN02078","RCN02083","RCN02084","RCN02085","RCN02087","RCN02088","RCN02089","RCN02091","RCN02092","RCN02093","RCN02097","RCN02098","RCN02099","RCN02100","RCN02101","RCN02102","RCN02103","RCN02106","RCN02108","RCN02109","RCN02111","RCN02112","RCN02114","RCN02116","RCN02117","RCN02118","RCN02119","RCN02120","RCN02121","RCN02123","RCN02124","RCN02125","RCN02126","RCN02127","RCN02130","RCN02133","RCN02134","RCN02136","RCN02139","RCN02140","RCN02141","RCN02142","RCN02143","RCN02145","RCN02146","RCN02147","RCN02149","RCN02151","RCN02152","RCN02153","RCN02154","RCN02155","RCN02157","RCN02160","RCN02161","RCN02164","RCN02167","RCN02169","RCN02170","RCN02171","RCN02172","RCN02175","RCN02182","RCN02184","RCN02186","RCN02187","RCN02188","RCN02189","RCN02190","RCN02191","RCN02192","RCN02197","RCN02198","RCN02199","RCN02201","RCN02202","RCN02203","RCN02205","RCN02206","RCN02207","RCN02210","RCN02213","RCN02214","RCN02216","RCN02217","RCN02218","RCN02221","RCN02222","RCN02224","RCN02226","RCN02227","RCN02228","RCN02229","RCN02231","RCN02232","RCN02233","RCN02234","RCN02237","RCN02238","RCN02239","RCN02240","RCN02241","RCN02242","RCN02243","RCN02245","RCN02246","RCN02247","RCN02248","RCN02249","RCN02250","RCN02251","RCN02252","RCN02254","RCN02255","RCN02259","RCN02270","RCN02271","RCN02272","RCN02273","RCN02274","RCN02275","RCN02278","RCN02279","RCN02280","RCN02281","RCN02282","RCN02284","RCN02287","RCN02288","RCN02291","RCN02292","RCN02294","RCN02295","RCN02296","RCN02297","RCN02298","RCN02300","RCN02301","RCN02302","RCN02303","RCN02304","RCN02305","RCN02308","RCN02309","RCN02310","RCN02311","RCN02312","RCN02313","RCN02314","RCN02315","RCN02316","RCN02317","RCN02318","RCN02319","RCN02320","RCN02321","RCN02322","RCN02324","RCN02325","RCN02326","RCN02327","RCN02328","RCN02329","RCN02330","RCN02331","RCN02333","RCN02335","RCN02337","RCN02338","RCN02339","RCN02341","RCN02343","RCN02345","RCN02346","RCN02348","RCN02350","RCN02352","RCN02353","RCN02354","RCN02355","RCN02356","RCN02357","RCN02358","RCN02359","RCN02360","RCN02361","RCN02362","RCN02363","RCN02364","RCN02365","RCN02366","RCN02367","RCN02369","RCN02394","RCN02398","RCN02401","RCN02404","RCN02409","RCN02410","RCN02411","RCN02421","RCN02427","RCN02437","RCN02443","RCN02446","RCN02449","RCN02452","RCN02453","RCN02454","RCN02455","RCN02456","RCN02457","RCN02459","RCN02462","RCN02463","RCN02470","RCN02471","RCN02478","RCN02484","RCN02485","RCN02486","RCN02488","RCN02490","RCN02494","RCN02501","RCN02503","RCN02508","RCN02515","RCN02516","RCN02517","RCN02519","RCN02520","RCN02521","RCN02522","RCN02524","RCN02525","RCN02526","RCN02527","RCN02528","RCN02529","RCN02530","RCN02531","RCN02532","RCN02533","RCN02534","RCN02535","RCN02537","RCN02538","RCN02539","RCN02540","RCN02541","RCN02542","RCN02543","RCN02544","RCN02545","RCN02546","RCN02547","RCN02548","RCN02549","RCN02550","RCN02551","RCN02552","RCN02553","RCN02554","RCN02555","RCN02556","RCN02557","RCN02558","RCN02559","RCN02560","RCN02562","RCN02563","RCN02564","RCN02565","RCN02566","RCN02567","RCN02568","RCN02571","RCN02572","RCN02573","RCN02574","RCN02577","RCN02578","RCN02579","RCN02580","RCN02581","RCN02582","RCN02585","RCN02586","RCN02587","RCN02588","RCN02589","RCN02590","RCN02591","RCN02592","RCN02593","RCN02594","RCN02596","RCN02598","RCN02599","RCN02601","RCN02602","RCN02605","RCN02606","RCN02607","RCN02608","RCN02609","RCN02611","RCN02612","RCN02613","RCN02614","RCN02615","RCN02616","RCN02617","RCN02619","RCN02620","RCN02621","RCN02624","RCN02626","RCN02628","RCN02629","RCN02630","RCN02632","RCN02633","RCN02634","RCN02635","RCN02636","RCN02637","RCN02638","RCN02639","RCN02640","RCN02641","RCN02642","RCN02643","RCN02644","RCN02645","RCN02646","RCN02647","RCN02648","RCN02649","RCN02650","RCN02651","RCN02652","RCN02653","RCN02654","RCN02655","RCN02656","RCN02657","RCN02659","RCN02662","RCN02663","RCN02664","RCN02666","RCN02673","RCN02674","RCN02685","RCN02686","RCN02690","RCN02691","RCN02692","RCN02693","RCN02694","RCN02696","RCN02700","RCN02701","RCN02702","RCN02703","RCN02704","RCN02705","RCN02707","RCN02708","RCN02710","RCN02712","RCN02713","RCN02714","RCN02716","RCN02717","RCN02718","RCN02719","RCN02720","RCN02721","RCN02722","RCN02723","RCN02724","RCN02725","RCN02726","RCN02727","RCN02728","RCN02729","RCN02731","RCN02732","RCN02733","RCN02734","RCN02735","RCN02736","RCN02737","RCN02738","RCN02740","RCN02741","RCN02742","RCN02743","RCN02744","RCN02745","RCN02746","RCN02747","RCN02748","RCN02749","RCN02750","RCN02751","RCN02756","RCN02758","RCN02759","RCN02760","RCN02761","RCN02762","RCN02764","RCN02765","RCN02766","RCN02767","RCN02774","RCN02775","RCN02776","RCN02777","RCN02779","RCN02780","RCN02781","RCN02782","RCN02783","RCN02784","RCN02785","RCN02787","RCN02788","RCN02789","RCN02790","RCN02791","RCN02792","RCN02793","RCN02795","RCN02796","RCN02798","RCN02799","RCN02800","RCN02801","RCN02804","RCN02806","RCN02807","RCN02809","RCN02814","RCN02816","RCN02817","RCN02818","RCN02819","RCN02820","RCN02821","RCN02822","RCN02824","RCN02826","RCN02827","RCN02829","RCN02830","RCN02831","RCN02832","RCN02833","RCN02834","RCN02837","RCN02839","RCN02841","RCN02842","RCN02843","RCN02844","RCN02846","RCN02847","RCN02848","RCN02849","RCN02850","RCN02851","RCN02853","RCN02854","RCN02855","RCN02856","RCN02857","RCN02858","RCN02860","RCN02861","RCN02862","RCN02863","RCN02866","RCN02879","RCN02880","RCN02881","RCN02884","RCN02887","RCN02892","RCN02893","RCN02894","RCN02895","RCN02902","RCN02903","RCN02904","RCN02907","RCN02908","RCN02910","RCN02915","RCN02916","RCN02917","RCN02918","RCN02922","RCN02923","RCN02925","RCN02929","RCN02930","RCN02932","RCN02933","RCN02934","RCN02935","RCN02941","RCN02942","RCN02943","RCN02944","RCN02945","RCN02946","RCN02947","RCN02948","RCN02949","RCN02951","RCN02952","RCN02953","RCN02954","RCN02955","RCN02956","RCN02958","RCN02959","RCN02960","RCN02961","RCN02962","RCN02964","RCN02965","RCN02966","RCN02968","RCN02970","RCN02971","RCN02972","RCN02973","RCN02974","RCN02976","RCN02977","RCN02979","RCN02980","RCN02981","RCN02982","RCN02983","RCN02985","RCN02986","RCN02987","RCN02988","RCN02989","RCN02991","RCN02992","RCN02993","RCN02994","RCN02995","RCN02996","RCN02997","RCN02998","RCN02999","RCN03001","RCN03002","RCN03003","RCN03004","RCN03006","RCN03007","RCN03008","RCN03010","RCN03011","RCN03013","RCN03014","RCN03016","RCN03017","RCN03018","RCN03019","RCN03020","RCN03021","RCN03022","RCN03023","RCN03025","RCN03026","RCN03028","RCN03030","RCN03031","RCN03032","RCN03034","RCN03035","RCN03036","RCN03040","RCN03041","RCN03042","RCN03043","RCN03044","RCN03045","RCN03047","RCN03048","RCN03049","RCN03050","RCN03054","RCN03056","RCN03058","RCN03059","RCN03061","RCN03063","RCN03066","RCN03069","RCN03071","RCN03072","RCN03073","RCN03074","RCN03075","RCN03078","RCN03079","RCN03082","RCN03083","RCN03085","RCN03086","RCN03087","RCN03088","RCN03089","RCN03090","RCN03091","RCN03093","RCN03094","RCN03095","RCN03096","RCN03097","RCN03098","RCN03099","RCN03100","RCN03101","RCN03104","RCN03105","RCN03106","RCN03107","RCN03108","RCN03109","RCN03110","RCN03111","RCN03112","RCN03113","RCN03121","RCN03122","RCN03123","RCN03124","RCN03125","RCN03126","RCN03127","RCN03129","RCN03130","RCN03131","RCN03135","RCN03136","RCN03137","RCN03138","RCN03139","RCN03142","RCN03143","RCN03144","RCN03146","RCN03149","RCN03150","RCN03152","RCN03153","RCN03155","RCN03158","RCN03159","RCN03160","RCN03161","RCN03164","RCN03165","RCN03167","RCN03168","RCN03169","RCN03171","RCN03172","RCN03173","RCN03174","RCN03176","RCN03177","RCN03178","RCN03179","RCN03181","RCN03182","RCN03183","RCN03184","RCN03185","RCN03186","RCN03187","RCN03188","RCN03189","RCN03191","RCN03192","RCN03194","RCN03195","RCN03196","RCN03197","RCN03260","RCN03261","RCN03263","RCN03266","RCN03271","RCN03272","RCN03283","RCN03285","RCN03290","RCN03292","RCN03299","RCN03300","RCN03303","RCN03306","RCN03307","RCN03311","RCN03312","RCN03313","RCN03315","RCN03316","RCN03317","RCN03318","RCN03319","RCN03320","RCN03322","RCN03323","RCN03326","RCN03328","RCN03334","RCN03335","RCN03336","RCN03337","RCN03338","RCN03339","RCN03340","RCN03341","RCN03342","RCN03343","RCN03344","RCN03347","RCN03348","RCN03349","RCN03351","RCN03352","RCN03353","RCN03354","RCN03357","RCN03358","RCN03360","RCN03361","RCN03363","RCN03365","RCN03367","RCN03368","RCN03369","RCN03375","RCN03376","RCN03377","RCN03380","RCN03382","RCN03383","RCN03384","RCN03385","RCN03386","RCN03388","RCN03389","RCN03390","RCN03391","RCN03392","RCN03394","RCN03396","RCN03397","RCN03398","RCN03399","RCN03400","RCN03401","RCN03402","RCN03403","RCN03405","RCN03406","RCN03408","RCN03409","RCN03410","RCN03414","RCN03415","RCN03419","RCN03420","RCN03421","RCN03422","RCN03423","RCN03424","RCN03425","RCN03426","RCN03428","RCN03429","RCN03430","RCN03431","RCN03433","RCN03435","RCN03436","RCN03439","RCN03440","RCN03441","RCN03442","RCN03444","RCN03445","RCN03448","RCN03450","RCN03451","RCN03452","RCN03453","RCN03458","RCN03459","RCN03460","RCN03461","RCN03462","RCN03463","RCN03464","RCN03467","RCN03468","RCN03469","RCN03470","RCN03471","RCN03472","RCN03474","RCN03475","RCN03476","RCN03477","RCN03478","RCN03479","RCN03480","RCN03484","RCN03485","RCN03486","RCN03487","RCN03488","RCN03489","RCN03490","RCN03491","RCN03492","RCN03493","RCN03494","RCN03495","RCN03496","RCN03497","RCN03501","RCN03502","RCN03504","RCN03505","RCN03508","RCN03509","RCN03510","RCN03511","RCN03512","RCN03513","RCN03514","RCN03515","RCN03518","RCN03519","RCN03521","RCN03522","RCN03523","RCN03526","RCN03527","RCN03528","RCN03529","RCN03530","RCN03531","RCN03534","RCN03535","RCN03536","RCN03610","RCN06881","RCN06893","RCN07825","RCN07826","RCN07827","RCN07828","RCN07829","RCN07830","RCN07836","RCN07837","RCN07838","RCN07839","RCN07842","RCN07843","RCN07845","RCN07848","RCN07849","RCN07851","RCN07852","RCN07853","RCN07854","RCN07856","RCN07858","RCN07859","RCN07860","RCN07862","RCN07863","RCN07864","RCN07865","RCN07951","RCN07954","RCN07986","RCN07994","RCN08001","RCN08009","RCN08011","RCN08013","RCN08015","RCN08017","RCN08019","RCN08021","RCN08023","RCN08025","RCN08027","RCN08029","RCN08031","RCN08033","RCN08035","RCN08037","RCN08039","RCN08042","RCN08044","RCN08045","RCN08047","RCN08050","RCN08051","RCN08053","RCN08055","RCN08057","RCN08060","RCN08061","RCN08063","RCN08069","RCN08071","RCN08075","RCN08079","RCN08082","RCN08084","RCN08086","RCN08087","RCN08090","RCN08091","RCN08092","RCN08093","RCN08094","RCN08095","RCN08096","RCN08097","RCN08098","RCN08099","RCN08100","RCN08101","RCN08103","RCN08105","RCN08106","RCN08107","RCN08112","RCN08113","RCN08114","RCN08115","RCN08116","RCN08117","RCN08118","RCN08119","RCN08120","RCN08121","RCN08122","RCN08123","RCN08124","RCN08126","RCN08127","RCN08128","RCN08129","RCN08130","RCN08131","RCN08132","RCN08135","RCN08136","RCN08137","RCN08138","RCN08139","RCN08140","RCN08141","RCN08143","RCN08144","RCN08145","RCN08147","RCN08148","RCN08149","RCN08150","RCN08151","RCN08152","RCN08154","RCN08156","RCN08157","RCN08158","RCN08159","RCN08161","RCN08165","RCN08166","RCN08167","RCN08168","RCN08169","RCN08171","RCN08172","RCN08173","RCN08174","RCN08175","RCN08176","RCN08177","RCN08179","RCN08180","RCN08181","RCN08182","RCN08183","RCN08184","RCN08185","RCN08186","RCN08187","RCN08189","RCN08190","RCN08191","RCN08192","RCN08193","RCN08197","RCN08198","RCN08199","RCN08200","RCN08201","RCN08202","RCN08204","RCN08205","RCN08206","RCN08207","RCN08209","RCN08210","RCN08211","RCN08213","RCN08214","RCN08215","RCN08216","RCN08217","RCN08218","RCN08219","RCN08220","RCN08221","RCN08230","RCN08232","RCN08238","RCN08240","RCN08241","RCN08244","RCN08245","RCN08248","RCN08249","RCN08250","RCN08252","RCN08253","RCN08254","RCN08256","RCN08259","RCN08261","RCN08262","RCN08263","RCN08265","RCN08268","RCN08269","RCN08270","RCN08272","RCN08273","RCN08274","RCN08276","RCN08278","RCN08279","RCN08280","RCN08281","RCN08282","RCN08283","RCN08284","RCN08285","RCN08286","RCN08287","RCN08288","RCN08289","RCN08290","RCN08291","RCN08292","RCN08293","RCN08294","RCN08295","RCN08296","RCN08297","RCN08298","RCN08299","RCN08300","RCN08301","RCN08303","RCN08304","RCN08305","RCN08306","RCN08307","RCN08308","RCN08309","RCN08310","RCN08311","RCN08312","RCN08313","RCN08314","RCN08315","RCN08316","RCN08317","RCN08318","RCN08319","RCN08320","RCN08321","RCN08322","RCN08323","RCN08324","RCN08325","RCN08326","RCN08327","RCN08328","RCN08329","RCN08330","RCN08331","RCN08332","RCN08333","RCN08334","RCN08335","RCN08336","RCN08337","RCN08338","RCN08339","RCN08340","RCN08341","RCN08342","RCN08343","RCN08344","RCN08345","RCN08346","RCN08347","RCN08348","RCN08349","RCN08350","RCN08351","RCN08352","RCN08353","RCN08354","RCN08355","RCN08356","RCN08357","RCN08358","RCN08359","RCN08360","RCN08361","RCN08362","RCN08363","RCN08364","RCN08365","RCN08366","RCN08367","RCN08368","RCN08369","RCN08370","RCN08371","RCN08372","RCN08374","RCN08375","RCN08376","RCN08377","RCN08378","RCN08379","RCN08380","RCN08381","RCN08382","RCN08383","RCN08384","RCN08385","RCN08386","RCN08387","RCN08389","RCN08390","RCN08392","RCN08393","RCN08621","RCN08623","RCN08624","RCN08626","RCN08629","RCN08633","RCN08634","RCN08635","RCN08636","RCN08637","RCN08638","RCN08639","RCN08640","RCN08641","RCN08642","RCN08643","RCN08644","RCN08645","RCN08646","RCN08647","RCN08648","RCN08649","RCN08650","RCN08651","RCN08652","RCN08653","RCN08654","RCN08655","RCN08656","RCN08657","RCN08659","RCN08660","RCN08661","RCN08662","RCN08663","RCN08664","RCN08665","RCN08666","RCN08667","RCN08668","RCN08669","RCN08670","RCN08671","RCN08672","RCN08673","RCN08674","RCN08675","RCN08676","RCN08677","RCN08678","RCN08679","RCN08680","RCN08681","RCN08682","RCN08683","RCN08684","RCN08685","RCN08686","RCN08687","RCN08688","RCN08689","RCN08690","RCN08692","RCN08693","RCN08694","RCN08695","RCN08696","RCN08699","RCN08700","RCN08701","RCN08702","RCN08703","RCN08704","RCN08705","RCN08706","RCN08707","RCN08709","RCN08713","RCN08714","RCN08715","RCN08719","RCN08720","RCN08722","RCN08723","RCN08724","RCN08725","RCN08726","RCN08727","RCN08728","RCN08729","RCN08730","RCN08731","RCN08732","RCN08733","RCN08734","RCN08736","RCN08737","RCN08738","RCN08739","RCN08740","RCN08741","RCN08742","RCN08743","RCN08744","RCN08745","RCN08746","RCN08747","RCN08748","RCN08749","RCN08750","RCN08751","RCN08752","RCN08753","RCN08754","RCN08756","RCN08757","RCN08758","RCN08759","RCN08761","RCN08762","RCN08763","RCN08764","RCN08765","RCN08766","RCN08767","RCN08768","RCN08769","RCN08770","RCN08771","RCN08772","RCN08773","RCN08774","RCN08775","RCN08776","RCN08777","RCN08778","RCN08779","RCN08780","RCN08781","RCN08782","RCN08783","RCN08784","RCN08807","RCN08808","RCN08809","RCN08810","RCN08811","RCN08812","RCN08813","RCN08814","RCN08815","RCN08816","RCN08817","RCN08818","RCN08819","RCN08820","RCN08821","RCN08822","RCN08824","RCN08825","RCN08827","RCN08828","RCN08829","RCN08830","RCN08831","RCN08833","RCN08834","RCN08878","RCN08880","RCN08881","RCN08882","RCN08883","RCN08884","RCN08885","RCN08887","RCN08888","RCN08889","RCN08890","RCN08891","RCN08892","RCN08922","RCN08923","RCN08925","RCN08926","RCN08927","RCN08929","RCN08930","RCN08931","RCN08933","RCN08934","RCN08935","RCN08936","RCN08937","RCN08938","RCN08939","RCN08941","RCN08942","RCN08943","RCN08945","RCN08946","RCN08947","RCN08949","RCN08950","RCN08951","RCN08952","RCN08953","RCN08954","RCN08955","RCN08957","RCN08958","RCN08959","RCN08960","RCN08961","RCN08962","RCN08963","RCN08966","RCN08971","RCN08973","RCN08974","RCN08975","RCN08976","RCN08977","RCN08978","RCN08979","RCN08981","RCN08982","RCN08984","RCN08985","RCN08987","RCN08989","RCN08990","RCN08991","RCN08992","RCN08993","RCN08994","RCN08995","RCN08998","RCN08999","RCN09000","RCN09001","RCN09002","RCN09003","RCN09006","RCN09007","RCN09010","RCN09012","RCN09014","RCN09016","RCN09018","RCN09019","RCN09020","RCN09021","RCN09022","RCN09023","RCN09024","RCN09025","RCN09027","RCN09029","RCN09031","RCN09032","RCN09033","RCN09034","RCN09035","RCN09036","RCN09037","RCN09038","RCN09039","RCN09040","RCN09041","RCN09042","RCN09043","RCN09044","RCN09045","RCN09046","RCN09047","RCN09048","RCN09049","RCN09050","RCN09052","RCN09053","RCN09054","RCN09055","RCN09056","RCN09057","RCN09058","RCN09059","RCN09060","RCN09061","RCN09062","RCN09063","RCN09064","RCN09065","RCN09066","RCN09069","RCN09070","RCN09071","RCN09072","RCN09073","RCN09074","RCN09075","RCN09076","RCN09077","RCN09079","RCN09080","RCN09081","RCN09082","RCN09083","RCN09084","RCN09088","RCN09093","RCN09095","RCN09096","RCN09098","RCN09105","RCN09106","RCN09110","RCN09111","RCN09112","RCN09113","RCN09114","RCN09115","RCN09116","RCN09117","RCN09118","RCN09119","RCN09120","RCN09121","RCN09122","RCN09126","RCN09127","RCN09128","RCN09129","RCN09130","RCN09131","RCN09132","RCN09133","RCN09134","RCN09135","RCN09136","RCN09137","RCN09138","RCN09139","RCN09140","RCN09141","RCN09143","RCN09144","RCN09145","RCN09146","RCN09147","RCN09148","RCN09149","RCN09150","RCN09151","RCN09155","RCN09156","RCN09157","RCN09158","RCN09159","RCN09160","RCN09161","RCN09162","RCN09163","RCN09164","RCN09165","RCN09166","RCN09167","RCN09168","RCN09170","RCN09220","RCN09225","RCN09229","RCN09230","RCN09234","RCN09235","RCN09236","RCN09238","RCN09239","RCN09240","RCN09241","RCN09245","RCN09250","RCN09251","RCN09254","RCN09258","RCN09259","RCN09260","RCN09261","RCN09262","RCN09264","RCN09265","RCN09266","RCN09268","RCN09269","RCN09270","RCN09271","RCN09272","RCN09273","RCN09274","RCN09276","RCN09278","RCN09282","RCN09283","RCN09284","RCN09285","RCN09286","RCN09292","RCN09293","RCN09295","RCN09299","RCN09301","RCN09302","RCN09303","RCN09305","RCN09306","RCN09308","RCN09309","RCN09310","RCN09312","RCN09313","RCN09314","RCN09315","RCN09316","RCN09317","RCN09318","RCN09319","RCN09320","RCN09322","RCN09324","RCN09326","RCN09329","RCN09330","RCN09331","RCN09332","RCN09333","RCN09334","RCN09337","RCN09338","RCN09339","RCN09340","RCN09341","RCN09342","RCN09344","RCN09346","RCN09347","RCN09348","RCN09350","RCN09352","RCN09354","RCN09360","RCN09362","RCN09363","RCN09364","RCN09365","RCN09366","RCN09367","RCN09368","RCN09370","RCN09373","RCN09375","RCN09378","RCN09379","RCN09381","RCN09382","RCN09385","RCN09386","RCN09387","RCN09388","RCN09389","RCN09390","RCN09394","RCN09398","RCN09399","RCN09400","RCN09401","RCN09402","RCN09403","RCN09405","RCN09412","RCN09413","RCN09415","RCN09416","RCN09418","RCN09419","RCN09420","RCN09421","RCN09423","RCN09424","RCN09425","RCN09426","RCN09427","RCN09428","RCN09433","RCN09435","RCN09436","RCN09439","RCN09440","RCN09441","RCN09443","RCN09446","RCN09451","RCN09454","RCN09458","RCN09460","RCN09462","RCN09463","RCN09464","RCN09466","RCN09467","RCN09472","RCN09475","RCN09477","RCN09478","RCN09481","RCN09483","RCN09485","RCN09486","RCN09487","RCN09489","RCN09490","RCN09491","RCN09492","RCN09493","RCN09494","RCN09495","RCN09501","RCN09503","RCN09504","RCN09505","RCN09506","RCN09507","RCN09509","RCN09512","RCN09515","RCN09518","RCN09519","RCN09520","RCN09522","RCN09524","RCN09525","RCN09526","RCN09527","RCN09532","RCN09533","RCN09534","RCN09535","RCN09536","RCN09537","RCN09539","RCN09540","RCN09541","RCN09542","RCN09544","RCN09547","RCN09549","RCN09550","RCN09552","RCN09554","RCN09555","RCN09559","RCN09560","RCN09561","RCN09562","RCN09565","RCN09566","RCN09567","RCN09568","RCN09569","RCN09570","RCN09571","RCN09572","RCN09573","RCN09574","RCN09575","RCN09576","RCN09577","RCN09578","RCN09580","RCN09581","RCN09582","RCN09584","RCN09585","RCN09586","RCN09587","RCN09588","RCN09589","RCN09590","RCN09592","RCN09594","RCN09596","RCN09597","RCN09599","RCN09600","RCN09602","RCN09605","RCN09607","RCN09610","RCN09611","RCN09613","RCN09614","RCN09615","RCN09616","RCN09617","RCN09619","RCN09621","RCN09622","RCN09624","RCN09625","RCN09628","RCN09630","RCN09631","RCN09632","RCN09633","RCN09636","RCN09637","RCN09638","RCN09639","RCN09640","RCN09641","RCN09642","RCN09643","RCN09644","RCN09647","RCN09648","RCN09649","RCN09650","RCN09652","RCN09653","RCN09654","RCN09655","RCN09656","RCN09657","RCN09658","RCN09662","RCN09663","RCN09664","RCN09666","RCN09667","RCN09668","RCN09669","RCN09670","RCN09671","RCN09673","RCN09675","RCN09677","RCN09678","RCN09680","RCN09681","RCN09683","RCN09684","RCN09685","RCN09687","RCN09688","RCN09689","RCN09690","RCN09691","RCN09694","RCN09695","RCN09696","RCN09699","RCN09700","RCN09701","RCN09702","RCN09703","RCN09704","RCN09705","RCN09708","RCN09709","RCN09710","RCN09711","RCN09712","RCN09713","RCN09714","RCN09715","RCN09716","RCN09717","RCN09718","RCN09719","RCN09720","RCN09721","RCN09722","RCN09723","RCN09725","RCN09726","RCN09728","RCN09729","RCN09734","RCN09735","RCN09736","RCN09737","RCN09738","RCN09740","RCN09741","RCN09743","RCN09744","RCN09745","RCN09746","RCN09748","RCN09749","RCN09751","RCN09753","RCN09756","RCN09757","RCN09759","RCN09760","RCN09761","RCN09762","RCN09763","RCN09764","RCN09765","RCN09767","RCN09768","RCN09771","RCN09772","RCN09773","RCN09774","RCN09776","RCN09777","RCN09780","RCN09782","RCN09783","RCN09784","RCN09785","RCN09786","RCN09787","RCN09788","RCN09791","RCN09792","RCN09793","RCN09794","RCN09796","RCN09798","RCN09800","RCN09805","RCN09809","RCN09811","RCN09812","RCN09813","RCN09814","RCN09815","RCN09816","RCN09818","RCN09819","RCN09820","RCN09823","RCN09825","RCN09826","RCN09827","RCN09828","RCN09829","RCN09832","RCN09834","RCN09835","RCN09836","RCN09837","RCN09843","RCN09845","RCN09847","RCN09850","RCN09851","RCN09852","RCN09853","RCN09854","RCN09855","RCN09856","RCN09857","RCN09860","RCN09863","RCN09864","RCN09865","RCN09868","RCN09874","RCN09880","RCN09881","RCN09884","RCN09888","RCN09889","RCN09890","RCN09892","RCN09893","RCN09894","RCN09895","RCN09898","RCN09899","RCN09900","RCN09901","RCN09902","RCN09904","RCN09905","RCN09906","RCN09907","RCN09908","RCN09910","RCN09911","RCN09912","RCN09913","RCN09914","RCN09915","RCN09916","RCN09920","RCN09921","RCN09922","RCN09923","RCN09924","RCN09925","RCN09926","RCN09927","RCN09928","RCN09929","RCN09930","RCN09931","RCN09932","RCN09933","RCN09934","RCN09935","RCN09936","RCN09939","RCN09940","RCN09941","RCN09942","RCN09943","RCN09944","RCN09945","RCN09946","RCN09948","RCN09950","RCN09951","RCN09953","RCN09954","RCN09955","RCN09956","RCN09958","RCN09959","RCN09960","RCN09961","RCN09962","RCN09963","RCN09964","RCN09965","RCN09967","RCN09970","RCN09973","RCN09974","RCN09978","RCN09979","RCN09986","RCN09987","RCN09988","RCN09989","RCN09990","RCN09992","RCN09994","RCN10000","RCN10002","RCN10003","RCN10004","RCN10006","RCN10007","RCN10008","RCN10009","RCN10010","RCN10011","RCN10014","RCN10015","RCN10016","RCN10019","RCN10021","RCN10143","RCN10145","RCN10151","RCN10159","RCN10162","RCN10167","RCN10168","RCN10169","RCN10170","RCN10175","RCN10176","RCN10178","RCN10179","RCN10181","RCN10183","RCN10185","RCN10187","RCN10188","RCN10189","RCN10190","RCN10192","RCN10194","RCN10195","RCN10196","RCN10199","RCN10200","RCN10201","RCN10202","RCN10203","RCN10204","RCN10205","RCN10207","RCN10208","RCN10210","RCN10213","RCN10215","RCN10217","RCN10219","RCN10220","RCN10221","RCN10223","RCN10224","RCN10225","RCN10226","RCN10227","RCN10238","RCN10244","RCN10246","RCN10247","RCN10250","RCN10252","RCN10254","RCN10256","RCN10268","RCN10279","RCN10286","RCN10290","RCN10291","RCN10292","RCN10293","RCN10296","RCN10297","RCN10298","RCN10299","RCN10300","RCN10305","RCN10306","RCN10307","RCN10308","RCN10309","RCN10311","RCN10313","RCN10314","RCN10315","RCN10316","RCN10317","RCN10318","RCN10319","RCN10320","RCN10321","RCN10322","RCN10323","RCN10324","RCN10325","RCN10330","RCN10332","RCN10333","RCN10337","RCN10338","RCN10339","RCN10340","RCN10341","RCN10345","RCN10346","RCN10347","RCN10348","RCN10349","RCN10350","RCN10351","RCN10352","RCN10353","RCN10354","RCN10356","RCN10357","RCN10362","RCN10363","RCN10364","RCN10365","RCN10370","RCN10372","RCN10373","RCN10377","RCN10379","RCN10380","RCN10381","RCN10382","RCN10384","RCN10385","RCN10386","RCN10387","RCN10388","RCN10390","RCN10391","RCN10392","RCN10393","RCN10396","RCN10397","RCN10398","RCN10399","RCN10400","RCN10401","RCN10402","RCN10405","RCN10406","RCN10689","RCN10690","RCN10691","RCN10692","RCN10693","RCN10694","RCN10695","RCN10696","RCN10697","RCN10699","RCN10703","RCN10704","RCN10706","RCN10707","RCN10709","RCN10710","RCN10712","RCN10715","RCN10716","RCN10717","RCN10718","RCN10719","RCN10720","RCN10721","RCN10722","RCN10723","RCN10724","RCN10725","RCN10726","RCN10727","RCN10728","RCN10729","RCN10730","RCN10731","RCN10732","RCN10733","RCN10734","RCN10735","RCN10736","RCN10737","RCN10738","RCN10741","RCN10742","RCN10743","RCN10744","RCN10745","RCN10746","RCN10752","RCN10761","RCN10762","RCN10764","RCN10766","RCN10768","RCN10769","RCN10770","RCN10772","RCN10773","RCN10774","RCN10776","RCN10777","RCN10778","RCN10779","RCN10780","RCN10781","RCN10782","RCN10783","RCN10785","RCN10786","RCN10788","RCN10789","RCN10790","RCN10792","RCN10794","RCN11011","RCN11012","RCN11013","RCN11014","RCN11015","RCN11016","RCN11017","RCN11018","RCN11019","RCN11020","RCN11021","RCN11022","RCN11023","RCN11024","RCN11025","RCN11026","RCN11027","RCN11029","RCN11030","RCN11032","RCN11034","RCN11036","RCN11038","RCN11040","RCN11042","RCN11043","RCN11045","RCN11048","RCN11049","RCN11050","RCN11054","RCN11056","RCN11058","RCN11062","RCN11063","RCN11067","RCN11070","RCN11071","RCN11072","RCN11075","RCN11077","RCN11080","RCN11081","RCN11085","RCN11086","RCN11088","RCN11089","RCN11094","RCN11096","RCN11104","RCN11106","RCN11107","RCN11108","RCN11109","RCN11110","RCN11111","RCN11112","RCN11113","RCN11114","RCN11115","RCN11118","RCN11119","RCN11120","RCN11121","RCN11122","RCN11123","RCN11124","RCN11125","RCN11126","RCN11127","RCN11128","RCN11129","RCN11131","RCN11133","RCN11134","RCN11135","RCN11136","RCN11137","RCN11138","RCN11139","RCN11141","RCN11143","RCN11144","RCN11145","RCN11146","RCN11147","RCN11150","RCN11151","RCN11152","RCN11153","RCN11154","RCN11155","RCN11156","RCN11158","RCN11159","RCN11160","RCN11161","RCN11162","RCN11163","RCN11164","RCN11165","RCN11166","RCN11167","RCN11168","RCN11169","RCN11170","RCN11171","RCN11172","RCN11173","RCN11175","RCN11176","RCN11177","RCN11178","RCN11179","RCN11180","RCN11181","RCN11182","RCN11183","RCN11184","RCN11185","RCN11186","RCN11187","RCN11189","RCN11190","RCN11191","RCN11192","RCN11193","RCN11194","RCN11195","RCN11196","RCN11197","RCN11208","RCN11215","RCN11217","RCN11221","RCN11222","RCN11223","RCN11224","RCN11225","RCN11228","RCN11229","RCN11230","RCN11231","RCN11232","RCN11233","RCN11234","RCN11235","RCN11236","RCN11237","RCN11238","RCN11239","RCN11240","RCN11241","RCN11243","RCN11244","RCN11245","RCN11246","RCN11247","RCN11248","RCN11249","RCN11250","RCN11251","RCN11252","RCN11253","RCN11254","RCN11255","RCN11256","RCN11257","RCN11258","RCN11259","RCN11260","RCN11261","RCN11262","RCN11265","RCN11266","RCN11267","RCN11268","RCN11269","RCN11270","RCN11273","RCN11274","RCN11275","RCN11276","RCN11277","RCN11278","RCN11282","RCN11283","RCN11284","RCN11285","RCN11286","RCN11287","RCN11288","RCN11289","RCN11290","RCN11292","RCN11293","RCN11294","RCN11295","RCN11296","RCN11298","RCN11299","RCN11300","RCN11301","RCN11302","RCN11303","RCN11304","RCN11305","RCN11306","RCN11307","RCN11308","RCN11309","RCN11310","RCN11312","RCN11313","RCN11314","RCN11315","RCN11316","RCN11317","RCN11318","RCN11319","RCN11320","RCN11321","RCN11322","RCN11324","RCN11325","RCN11326","RCN11327","RCN11328","RCN11331","RCN11332","RCN11333","RCN11334","RCN11335","RCN11336","RCN11337","RCN11339","RCN11341","RCN11492","RCN11493","RCN11494","RCN11495","RCN11497","RCN11498","RCN11499","RCN11500","RCN11501","RCN11504","RCN11505","RCN11506","RCN11507","RCN11508","RCN11510","RCN11511","RCN11512","RCN11513","RCN11514","RCN11515","RCN11516","RCN11517","RCN11518","RCN11519","RCN11520","RCN11521","RCN11522","RCN11523","RCN11524","RCN11525","RCN11526","RCN11527","RCN11528","RCN11529","RCN11530","RCN11531","RCN11532","RCN11533","RCN11534","RCN11535","RCN11537","RCN11538","RCN11539","RCN11540","RCN11541","RCN11542","RCN11543","RCN11544","RCN11545","RCN11546","RCN11547","RCN11548","RCN11550","RCN11551","RCN11552","RCN11553","RCN11554","RCN11555","RCN11556","RCN11557","RCN11558","RCN11559","RCN11560","RCN11561","RCN11562","RCN11564","RCN11565","RCN11566","RCN11567","RCN11568","RCN11569","RCN11570","RCN11571","RCN11572","RCN11573","RCN11574","RCN11575","RCN11576","RCN11577","RCN11578","RCN11579","RCN11580","RCN11581","RCN11582","RCN11593","RCN11594","RCN11595","RCN11596","RCN11597","RCN11598","RCN11599","RCN11600","RCN11601","RCN11602","RCN11603","RCN11604","RCN11605","RCN11606","RCN11607","RCN11608","RCN11610","RCN11611","RCN11612","RCN11613","RCN11614","RCN11615","RCN11616","RCN11617","RCN11618","RCN11619","RCN11620","RCN11621","RCN11622","RCN11623","RCN11624","RCN11625","RCN11626","RCN11627","RCN11628","RCN11629","RCN11630","RCN11631","RCN11632","RCN11633","RCN11634","RCN11635","RCN11636","RCN11637","RCN11638","RCN11639","RCN11640","RCN11641","RCN11642","RCN11643","RCN11644","RCN11645","RCN11646","RCN11647","RCN11648","RCN11649","RCN11650","RCN11651","RCN11652","RCN11653","RCN11654","RCN11655","RCN11658","RCN11660","RCN11661","RCN11662","RCN11663","RCN11664","RCN11665","RCN11666","RCN11667","RCN11668","RCN11669","RCN11670","RCN11671","RCN11672","RCN11673","RCN11674","RCN11675","RCN11676","RCN11677","RCN11678","RCN11679","RCN11680","RCN11681","RCN11682","RCN11684","RCN11685","RCN11687","RCN11688","RCN11689","RCN11691","RCN11692","RCN11693","RCN11694","RCN11695","RCN11696","RCN11699","RCN11700","RCN11701","RCN11702","RCN11703","RCN11704","RCN11706","RCN11707","RCN11708","RCN11709","RCN11710","RCN11711","RCN11712","RCN11713","RCN11714","RCN11715","RCN11716","RCN11722","RCN11723","RCN11725","RCN11726","RCN11727","RCN11730","RCN11731","RCN11732","RCN11740","RCN11741","RCN11742","RCN11743","RCN11747","RCN11748","RCN11749","RCN11751","RCN11752","RCN11753","RCN11754","RCN11756","RCN11758","RCN11759","RCN11760","RCN11761","RCN11762","RCN11764","RCN11765","RCN11766","RCN11767","RCN11768","RCN11769","RCN11770","RCN11771","RCN11772","RCN11773","RCN11774","RCN11775","RCN11776","RCN11777","RCN11778","RCN11779","RCN11780","RCN11781","RCN11782","RCN11783","RCN11784","RCN11786","RCN11787","RCN11788","RCN11789","RCN11790","RCN11791","RCN11792","RCN11793","RCN11795","RCN11796","RCN11797","RCN11798","RCN11799","RCN11800","RCN11801","RCN11802","RCN11803","RCN11804","RCN11805","RCN11806","RCN11807","RCN11808","RCN11809","RCN11810","RCN11811","RCN11812","RCN11813","RCN11816","RCN11817","RCN11819","RCN11820","RCN11822","RCN11823","RCN11824","RCN11825","RCN11827","RCN11828","RCN11829","RCN11830","RCN11831","RCN11833","RCN11834","RCN11835","RCN11836","RCN11837","RCN11838","RCN11839","RCN11840","RCN11841","RCN11843","RCN11844","RCN11845","RCN11847","RCN11848","RCN11849","RCN11850","RCN11852","RCN11854","RCN11855","RCN11856","RCN11858","RCN11859","RCN11862","RCN11863","RCN11864","RCN11865","RCN11866","RCN11868","RCN11869","RCN11870","RCN11871","RCN11872","RCN11873","RCN11874","RCN11875","RCN11876","RCN11877","RCN11878","RCN11879","RCN11880","RCN11881","RCN11882","RCN11883","RCN11884","RCN11885","RCN11886","RCN11887","RCN11888","RCN11889","RCN11890","RCN11891","RCN11892","RCN11893","RCN11895","RCN11896","RCN11897","RCN11898","RCN11899","RCN11900","RCN11901","RCN11903","RCN11904","RCN11905","RCN11906","RCN11907","RCN11908","RCN11909","RCN11910","RCN11911","RCN11912","RCN11913","RCN11914","RCN11915","RCN11916","RCN11917","RCN11918","RCN11919","RCN11920","RCN11921","RCN11922","RCN11923","RCN11924","RCN11925","RCN11926","RCN11927","RCN11928","RCN11929","RCN11930","RCN11931","RCN11933","RCN11936","RCN11937","RCN11938","RCN11939","RCN11940","RCN12025","RCN12026","RCN12027","RCN12028","RCN12031","RCN12032","RCN12033","RCN12034","RCN12035","RCN12036","RCN12040","RCN12041","RCN12042","RCN12043","RCN12044","RCN12050","RCN12051","RCN12052","RCN12057","RCN12058","RCN12059","RCN12060","RCN12063","RCN12064","RCN12066","RCN12068","RCN12070","RCN12071","RCN12072","RCN12074","RCN12075","RCN12076","RCN12077","RCN12079","RCN12080","RCN12081","RCN12082","RCN12083","RCN12084","RCN12087","RCN12088","RCN12091","RCN12096","RCN12100","RCN12103","RCN12106","RCN12107","RCN12111","RCN12112","RCN12113","RCN12115","RCN12116","RCN12357","RCN12358","RCN12359","RCN12361","RCN12362","RCN12363","RCN12364","RCN12365","RCN12366","RCN12367","RCN12369","RCN12370","RCN12371","RCN12372","RCN12373","RCN12374","RCN12375","RCN12376","RCN12378","RCN12379","RCN12380","RCN12381","RCN12382","RCN12383","RCN12384","RCN12385","RCN12386","RCN12387","RCN12388","RCN12389","RCN12390","RCN12391","RCN12392","RCN12393","RCN12394","RCN12397","RCN12398","RCN12401","RCN12402","RCN12403","RCN12404","RCN12405","RCN12406","RCN12407","RCN12408","RCN12409","RCN12410","RCN12411","RCN12412","RCN12420","RCN12435","RCN12437","RCN12438","RCN12442","RCN12453","RCN12454","RCN12456","RCN12458","RCN12459","RCN12461","RCN12462","RCN12466","RCN12468","RCN12469","RCN12476","RCN12478","RCN12479","RCN12480","RCN12481","RCN12482","RCN12483","RCN12484","RCN12485","RCN12486","RCN12489","RCN12490","RCN12493","RCN12494","RCN12495","RCN12496","RCN12497","RCN12501","RCN12504","RCN12505","RCN12506","RCN12510","RCN12511","RCN12512","RCN12514","RCN12520","RCN12522","RCN12524","RCN12528","RCN12547","RCN12548","RCN12550","RCN12553","RCN12558","RCN12560","RCN12562","RCN12563","RCN12566","RCN12567","RCN12569","RCN12570","RCN12571","RCN12574","RCN12575","RCN12576","RCN12577","RCN12584","RCN12586","RCN12587","RCN12590","RCN12591","RCN12592","RCN12593","RCN12596","RCN12597","RCN12599","RCN12600","RCN12601","RCN12602","RCN12603","RCN12604","RCN12606","RCN12607","RCN12608","RCN12612","RCN12613","RCN12614","RCN12615","RCN12616","RCN12617","RCN12618","RCN12619","RCN12620","RCN12621","RCN12622","RCN12770","RCN12772","RCN12773","RCN12774","RCN12775","RCN12776","RCN12779","RCN12780","RCN12781","RCN12784","RCN12785","RCN12786","RCN12787","RCN12788","RCN12789","RCN12790","RCN12791","RCN12792","RCN12793","RCN12794","RCN12795","RCN12796","RCN12797","RCN12798","RCN12799","RCN12800","RCN12801","RCN12802","RCN12804","RCN12806","RCN12807","RCN12808","RCN12809","RCN12810","RCN12811","RCN12813","RCN12815","RCN12816","RCN12817","RCN12818","RCN12819","RCN12820","RCN12821","RCN12822","RCN12823","RCN12824","RCN12825","RCN12826","RCN12827","RCN12828","RCN12829","RCN12830","RCN12831","RCN12832","RCN12834","RCN12835","RCN12836","RCN12837","RCN12838","RCN12839","RCN12840","RCN12841","RCN12842","RCN12843","RCN12844","RCN12846","RCN12847","RCN12848","RCN12849","RCN12850","RCN12851","RCN12852","RCN12853","RCN12854","RCN12855","RCN12856","RCN12857","RCN12858","RCN12859","RCN12860","RCN12862","RCN12865","RCN12867","RCN12868","RCN12869","RCN12870","RCN12871","RCN12872","RCN12873","RCN12878","RCN12879","RCN12880","RCN12881","RCN12883","RCN12884","RCN12886","RCN12888","RCN12889","RCN12890","RCN12891","RCN12892","RCN12893","RCN12894","RCN12895","RCN12897","RCN12900","RCN12901","RCN12976","RCN12978","RCN12980","RCN12982","RCN12984","RCN12985","RCN12986","RCN12987","RCN12988","RCN12990","RCN12992","RCN12993","RCN12994","RCN12995","RCN12996","RCN12997","RCN12999","RCN13000","RCN13003","RCN13004","RCN13006","RCN13008","RCN13009","RCN13010","RCN13012","RCN13013","RCN13014","RCN13015","RCN13016","RCN13017","RCN13018","RCN13019","RCN13020","RCN13021","RCN13023","RCN13025","RCN13028","RCN13029","RCN13031","RCN13032","RCN13033","RCN13034","RCN13036","RCN13037","RCN13039","RCN13041","RCN13044","RCN13045","RCN13048","RCN13049","RCN13050","RCN13051","RCN13052","RCN13053","RCN13054","RCN13056","RCN13057","RCN13058","RCN13059","RCN13060","RCN13061","RCN13062","RCN13063","RCN13065","RCN13066","RCN13067","RCN13068","RCN13069","RCN13070","RCN13071","RCN13072","RCN13073","RCN13074","RCN13075","RCN13076","RCN13077","RCN13078","RCN13079","RCN13080","RCN13081","RCN13082","RCN13083","RCN13084","RCN13085","RCN13086","RCN13087","RCN13088","RCN13089","RCN13090","RCN13093","RCN13095","RCN13096","RCN13097","RCN13098","RCN13099","RCN13100","RCN13101","RCN13103","RCN13104","RCN13263","RCN13264","RCN13265","RCN13266","RCN13267","RCN13268","RCN13269","RCN13270","RCN13271","RCN13272","RCN13273","RCN13274","RCN13275","RCN13276","RCN13277","RCN13278","RCN13279","RCN13280","RCN13281","RCN13282","RCN13283","RCN13284","RCN13285","RCN13286","RCN13287","RCN13288","RCN13289","RCN13290","RCN13291","RCN13292","RCN13293","RCN13294","RCN13295","RCN13296","RCN13297","RCN13298","RCN13299","RCN13300","RCN13301","RCN13302","RCN13303","RCN13304","RCN13305","RCN13306","RCN13307","RCN13308","RCN13309","RCN13310","RCN13311","RCN13312","RCN13313","RCN13316","RCN13319","RCN13320","RCN13321","RCN13322","RCN13323","RCN13324","RCN13329","RCN13330","RCN13331","RCN13332","RCN13333","RCN13336","RCN13337","RCN13338","RCN13339","RCN13340","RCN13341","RCN13342","RCN13343","RCN13344","RCN13345","RCN13346","RCN13347","RCN13348","RCN13349","RCN13350","RCN13351","RCN13352","RCN13353","RCN13354","RCN13355","RCN13356","RCN13357","RCN13358","RCN13359","RCN13360","RCN13361","RCN13362","RCN13363","RCN13364","RCN13365","RCN13366","RCN13367","RCN13368","RCN13369","RCN13370","RCN13371","RCN13372","RCN13373","RCN13374","RCN13375","RCN13376","RCN13377","RCN13378","RCN13379","RCN13380","RCN13381","RCN13382","RCN13383","RCN13384","RCN13385","RCN13386","RCN13387","RCN13388","RCN13389","RCN13390","RCN13391","RCN13392","RCN13394","RCN13395","RCN13396","RCN13397","RCN13398","RCN13400","RCN13401","RCN13402","RCN13403","RCN13404","RCN13405","RCN13406","RCN13407","RCN13408","RCN13409","RCN13410","RCN13411","RCN13412","RCN13413","RCN13414","RCN13415","RCN13416","RCN13417","RCN13418","RCN13419","RCN13420","RCN13421","RCN13422","RCN13424","RCN13427","RCN13428","RCN13429","RCN13430","RCN13431","RCN13432","RCN13433","RCN13434","RCN13435","RCN13436","RCN13437","RCN13438","RCN13439","RCN13441","RCN13442","RCN13443","RCN13445","RCN13446","RCN13447","RCN13448","RCN13449","RCN13450","RCN13452","RCN13454","RCN13455","RCN13457","RCN13458","RCN13459","RCN13460","RCN13461","RCN13462","RCN13463","RCN13464","RCN13465","RCN13466","RCN13467","RCN13468","RCN13469","RCN13470","RCN13471","RCN13472","RCN13473","RCN13474","RCN13475","RCN13476","RCN13477","RCN13478","RCN13479","RCN13480","RCN13481","RCN13482","RCN13483","RCN13484","RCN13485","RCN13486","RCN13487","RCN13488","RCN13489","RCN13490","RCN13491","RCN13492","RCN13493","RCN13494","RCN13495","RCN13496","RCN13497","RCN13498","RCN13499","RCN13500","RCN13501","RCN13502","RCN13503","RCN13505","RCN13506","RCN13507","RCN13508","RCN13510","RCN13511","RCN13512","RCN13513","RCN13514","RCN13515","RCN13516","RCN13517","RCN13518","RCN13520","RCN13521","RCN13523","RCN13524","RCN13525","RCN13526","RCN13527","RCN13528","RCN13529","RCN13530","RCN13531","RCN13532","RCN13533","RCN13534","RCN13536","RCN13537","RCN13538","RCN13539","RCN13540","RCN13541","RCN13542","RCN13543","RCN13544","RCN13545","RCN13546","RCN13548","RCN13549","RCN13551","RCN13552","RCN13553","RCN13554","RCN13555","RCN13556","RCN13557","RCN13558","RCN13559","RCN13560","RCN13561","RCN13562","RCN13563","RCN13564","RCN13565","RCN13566","RCN13567","RCN13568","RCN13569","RCN13570","RCN13571","RCN13574","RCN13575","RCN13576","RCN13577","RCN13578","RCN13579","RCN13580","RCN13581","RCN13582","RCN13583","RCN13584","RCN13585","RCN13907","RCN13908","RCN13909","RCN13910","RCN13911","RCN13912","RCN13914","RCN13915","RCN13916","RCN13917","RCN13918","RCN13919","RCN13920","RCN13921","RCN13922","RCN13924","RCN13925","RCN13926","RCN13927","RCN13928","RCN13929","RCN13930","RCN13931","RCN13933","RCN13935","RCN13936","RCN13937","RCN13938","RCN13939","RCN13940","RCN13941","RCN13942","RCN13943","RCN13944","RCN13945","RCN13946","RCN13948","RCN13950","RCN13951","RCN13952","RCN13955","RCN13956","RCN13957","RCN13958","RCN13959","RCN13960","RCN13962","RCN13963","RCN13964","RCN13967","RCN13968","RCN13969","RCN13972","RCN13974","RCN13975","RCN13976","RCN13977","RCN13978","RCN13979","RCN13980","RCN13981","RCN13982","RCN13983","RCN13984","RCN13985","RCN13986","RCN13987","RCN13988","RCN13989","RCN13990","RCN13991","RCN13992","RCN13993","RCN13994","RCN13996","RCN13997","RCN13998","RCN13999","RCN14000","RCN14001","RCN14002","RCN14003","RCN14004","RCN14005","RCN14006","RCN14007","RCN14008","RCN14009","RCN14010","RCN14011","RCN14012","RCN14013","RCN14014","RCN14015","RCN14016","RCN14017","RCN14018","RCN14019","RCN14020","RCN14021","RCN14022","RCN14023","RCN14024","RCN14025","RCN14026","RCN14027","RCN14028","RCN14029","RCN14030","RCN14031","RCN14032","RCN14033","RCN14034","RCN14035","RCN14036","RCN14037","RCN14038","RCN14039","RCN14040","RCN14041","RCN14042","RCN14043","RCN14044","RCN14045","RCN14046","RCN14047","RCN14048","RCN14049","RCN14050","RCN14051","RCN14053","RCN14054","RCN14055","RCN14056","RCN14057","RCN14058","RCN14059","RCN14061","RCN14062","RCN14063","RCN14064","RCN14065","RCN14066","RCN14067","RCN14068","RCN14069","RCN14072","RCN14073","RCN14076","RCN14077","RCN14078","RCN14079","RCN14080","RCN14081","RCN14082","RCN14083","RCN14084","RCN14085","RCN14086","RCN14087","RCN14088","RCN14089","RCN14090","RCN14091","RCN14092","RCN14093","RCN14095","RCN14096","RCN14097","RCN14099","RCN14101","RCN14102","RCN14103","RCN14104","RCN14105","RCN14106","RCN14108","RCN14109","RCN14110","RCN14111","RCN14112","RCN14113","RCN14114","RCN14115","RCN14116","RCN14117","RCN14118","RCN14119","RCN14120","RCN14121","RCN14122","RCN14123","RCN14124","RCN14125","RCN14126","RCN14127","RCN14128","RCN14129","RCN14130","RCN14132","RCN14134","RCN14135","RCN14136","RCN14137","RCN14138","RCN14139","RCN14140","RCN14142","RCN14143","RCN14144","RCN14145","RCN14146","RCN14147","RCN14148","RCN14149","RCN14150","RCN14151","RCN14152","RCN14153","RCN14154","RCN14155","RCN14156","RCN14158","RCN14159","RCN14160","RCN14161","RCN14162","RCN14163","RCN14164","RCN14165","RCN14166","RCN14167","RCN14262","RCN14263","RCN14264","RCN14265","RCN14268","RCN14269","RCN14270","RCN14271","RCN14272","RCN14273","RCN14276","RCN14277","RCN14278","RCN14279","RCN14280","RCN14281","RCN14284","RCN14285","RCN14286","RCN14287","RCN14288","RCN14289","RCN14291","RCN14292","RCN14293","RCN14294","RCN14295","RCN14296","RCN14297","RCN14300","RCN14301","RCN14302","RCN14303","RCN14304","RCN14305","RCN14309","RCN14310","RCN14311","RCN14312","RCN14313","RCN14316","RCN14317","RCN14318","RCN14319","RCN14320","RCN14321","RCN14324","RCN14325","RCN15562","RCN15581","RCN15582","RCN15589","RCN15590","RCN15643","RCN15644","RCN15647","RCN15650","RCN15653","RCN15658","RCN15659","RCN15660","RCN15661","RCN15662","RCN15663","RCN15666","RCN15676","RCN15679","RCN15680","RCN15682","RCN15683","RCN15688","RCN15693","RCN15699","RCN15712","RCN15714","RCN15718","RCN15720","RCN15729","RCN15730","RCN15736","RCN15738","RCN15746","RCN15749","RCN15750","RCN15751","RCN15755","RCN15756","RCN15757","RCN15764","RCN15770","RCN15771","RCN15772","RCN15781","RCN15786","RCN15792","RCN15800","RCN15804","RCN15812","RCN15820","RCN15821","RCN15823","SPT00018","SPT00020","SPT00022","SPT00023","SPT00025","SPT00028","SPT00029","SPT00032","SPT00035","SPT00037","SPT00039","SPT00041","SPT00044","SPT00046","SPT00050","SPT00052","SPT00053","SPT00055","SPT00057","SPT00059","SPT00063","SPT00065","SPT00066","SPT00068","SPT00072","SPT00073","SPT00077","SPT00079","SPT00082","SPT00085","SPT00087","SPT00089","SPT00093","SPT00095","SPT00192","SPT00195","SPT00198","SPT00200","SPT00202","SPT00203","SPT00204","SPT00205","SPT00207","SPT00208","SPT00209","SPT00211","SPT00212","SPT00214","SPT00216","SPT00219","SPT00222","SPT00752","SPT00753","SPT00759","SPT00761","SPT00763","SPT00764","SPT00768","SPT00769","SPT00770","SPT00772","SPT00773","SPT00775","SPT00778","SPT00779","SPT00782","SPT00784","SPT00788","SPT00789","SPT00790","SPT00791","SPT00792","SPT00794","SPT00795","SPT00796","SPT00797","SPT00798","SPT00800","SPT00801","SPT00802","SPT00803","SPT00804","SPT00807","SPT00808","SPT00809","SPT00810","SPT00811","SPT00813","SPT00815","SPT00817","SPT00818","SPT00819","SPT00820","SPT00821","SPT00822","SPT00824","SPT00825","SPT00826","SPT00827","SPT00828","SPT00829","SPT00830","SPT00831","SPT00832","SPT00833","SPT00834","SPT00835","SPT00837","SPT00839","SPT00841","SPT00843","SPT00844","SPT00845","SPT00846","SPT00847","SPT00848","SPT00849","SPT00850","SPT00851","SPT00857","SPT00858","SPT00859","SPT00860","SPT00861","SPT00862","SPT00864","SPT00866","SPT00867","SPT00868","SPT00869","SPT00871","SPT00872","SPT00873","SPT00875","SPT00876","SPT00877","SPT00879","SPT00880","SPT00881","SPT00883","SPT00889","SPT00890","SPT00892","SPT00898","SPT00899","SPT00901","SPT00902","SPT00903","SPT12071","SPT12073","SPT12074","SPT12075","SPT12076","SPT12077","SPT12078","SPT12080","SPT12082","SPT12084","SPT12085","SPT12086","SPT12087","SPT12088","SPT12089","SPT12090","SPT12091","SPT12092","SPT12093","SPT12095","SPT12096","SPT12097","SPT12098","SPT12100","SPT12102","SPT12103","SPT12104","SPT12105","SPT12109","SPT12110","SPT12111","SPT12112","SPT12114","SPT12115","SPT12116","SPT12117","SPT12118","SPT12119","SPT12120","SPT12121","SPT12122","SPT12124","SPT12126","SPT12128","SPT12129","SPT12131","SPT12132","SPT12133","SPT12134","SPT12135","SPT12136","SPT12138","SPT12139","SPT12226","SPT12230","SPT12263","SPT12264","SPT12265","SPT12266","SPT12269","SPT12271","SPT12272","SPT12273","SPT12274","SPT12275","SPT12277","SPT12278","SPT12279","SPT12281","SPT12283","SPT12292","SPT12293","SPT12297","SPT12299","SPT12302","SPT12304","SPT12307","SPT12309","SPT12317","SPT12319","SPT12321","SPT12323","SPT12327","SPT12329","SPT12330","SPT12332","SPT12335","SPT12336","SPT12337","SPT12338","SPT12339","SPT12340","SPT12342","SPT12343","SPT12344","SPT12345","SPT12346","SPT12347","SPT12348","SPT12349","SPT12350","SPT12351","SPT12353","SPT12354","SPT12355","SPT12356","SPT12357","SPT12358","SPT12367","SPT12368","SPT12370","SPT12372","SPT12450","SPT12454","SPT12455","SPT12458","SPT12459","SPT12460","SPT12461","SPT12463","SPT12467","SPT12470","SPT12471","SPT12472","SPT12477","SPT12478","SPT12479","SPT12480","SPT12482","SPT12485","SPT12486","SPT12488","SPT12490","SPT12492","SPT12493","SPT12494","SPT12495","SPT12497","SPT12498","SPT12499","SPT12500","SPT12502","SPT12505","SPT12506","SPT12507","SPT12508","SPT12510","SPT12513","SPT12514","SPT12515","SPT12516","SPT12517","SPT12518","SPT12519","SPT12520","SPT12521","SPT12522","SPT12523","SPT12524","SPT12525","SPT12526","SPT12527","SPT12528","SPT12529","SPT12535","SPT12536","SPT12537","SPT12540","SPT12543","SPT12544","SPT12552","SPT12555","SPT12556","SPT12557","SPT12558","SPT12559","SPT12560","SPT12563","SPT12564","SPT12566","SPT12568","SPT12569","SPT12572","SPT12573","SPT12576","SPT12578","SPT12579","SPT12580","SPT12581","SPT12583","SPT12584","SPT12622","SPT12652","SPT12666","SPT12705","SPT12727","SPT12741","SPT12796","SPT12807","SPT12863","SPT12916","SPT12920","SPT12964","SPT12969","SPT12972","SPT12980","SPT12982","SPT12985","SPT12990","SPT13003","SPT13038","SPT13073","SPT14937","SPT14938","SPT14939","SPT14940","SPT14941","SPT14942","SPT14944","SPT14945","SPT14946","SPT14947","SPT14949","SPT14951","SPT14952","SPT14953","SPT14954","SPT14955","SPT14956","SPT14957","SPT14958","SPT14959","SPT14960","SPT14961","SPT14962","SPT14963","SPT14964","SPT14965","SPT14966","SPT14967","SPT14968","SPT14969","SPT14970","SPT14971","SPT14972","SPT14973","SPT14974","SPT14975","SPT14976","SPT14977","SPT14978","SPT14979","SPT14980","SPT14981","SPT14982","SPT14983","SPT14984","SPT14985","SPT14986","SPT14987","SPT14988","SPT14989","SPT14990","SPT14991","SPT14992","SPT14993","SPT14994","SPT14995","SPT14996","SPT14997","SPT14998","SPT15000","SPT15001","SPT15002","SPT15003","SPT15004","SPT15005","SPT15006","SPT15007","SPT15008","SPT15009","SPT15010","SPT15011","SPT15013","SPT15014","SPT15015","SPT15016","SPT15017","SPT15018","SPT15019","SPT15020","SPT15021","SPT15022","SPT15023","SPT15024","SPT15025","SPT15026","SPT15027","SPT15028","SPT15029","SPT15030","SPT15031","SPT15032","SPT15033","SPT15034","SPT15035","SPT15036","SPT15037","SPT15038","SPT15039","SPT15040","SPT15041","SPT15042","SPT15044","SPT15045","SPT15046","SPT15047","SPT15048","SPT15049","SPT15050","SPT15051","SPT15052","SPT15053","SPT15054","SPT15055","SPT15056","SPT15057","SPT15058","SPT15059","SPT15060","SPT15061","SPT15062","SPT15063","SPT15064","SPT15065","SPT15066","SPT15067","SPT15068","SPT15069","SPT15070","SPT15071","SPT15073","SPT15074","SPT15075","SPT15076","SPT15077","SPT15078","SPT15079","SPT15080","SPT15081","SPT15082","SPT15083","SPT15084","SPT15085","SPT15086","SPT15087","SPT15088","SPT15089","SPT15090","SPT15091","SPT15092","SPT15093","SPT15094","SPT15095","SPT15096","SPT15097","SPT15098","SPT15099","SPT15100","SPT15101","SPT15102","SPT15103","SPT15104","SPT15105","SPT15106","SPT15107","SPT15108","SPT15109","SPT15110","SPT15111","SPT15112","SPT15113","SPT15114","SPT15115","SPT15116","SPT15117","SPT15118","SPT15119","SPT15120","SPT15121","SPT15122","SPT15123","SPT15124","SPT15125","SPT15126","SPT15127","SPT15128","SPT15129","SPT15130","SPT15131","SPT15132","SPT15133","SPT15134","SPT15135","SPT15136","SPT15137","SPT15138","SPT15139","SPT15140","SPT15141","SPT15143","SPT15144","SPT15146","SPT15147","SPT15148","SPT15149","SPT15150","SPT15151","SPT15152","SPT15153","SPT15154","SPT15155","SPT15156","SPT15157","SPT15158","SPT15159","SPT15160","SPT15173","SPT15174","SPT15175","SPT15176","SPT15177","SPT15178","SPT15180","SPT15181","SPT15182","SPT15183","SPT15184","SPT15185","SPT15186","SPT15187","SPT15188","SPT15190","SPT15191","SPT15192","SPT15193","SPT15194","SPT15195","SPT15197","SPT15198","SPT15199","SPT15200","SPT15201","SPT15202","SPT15203","SPT15204","SPT15205","SPT15206","SPT15207","SPT15208","SPT15209","SPT15210","SPT15211","SPT15212","SPT15213","SPT15214","SPT15215","SPT15216","SPT15217","SPT15218","SPT15219","SPT15220","SPT15221","SPT15222","SPT15223","SPT15224","SPT15225","SPT15226","SPT15227","SPT15228","SPT15229","SPT15230","SPT15231","SPT15232","SPT15233","SPT15234","SPT15235","SPT15236","SPT15237","SPT15239","SPT15240","SPT15241","SPT15242","SPT15243","SPT15244","SPT15245","SPT15246","SPT15247","SPT15248","SPT15249","SPT15250","SPT15251","SPT15252","SPT15253","SPT15254","SPT15255","SPT15256","SPT15257","SPT15258","SPT15259","SPT15260","SPT15261","SPT15262","SPT15263","SPT15264","SPT15265","SPT15294","SPT15296","SPT15297","SPT15298","SPT15304","SPT15309","SPT15314","SPT15325","SPT15327","SPT15330","SPT15333","SPT15335","SPT15340","SPT15342","SPT15346","SPT15348","SPT15349","SPT15352","SPT15356","SPT15372","SPT15376","SPT15381","SPT15389","SPT15407","SPT15413","SPT15414","SPT15417","SPT15420","SPT15443","SPT15446","SPT15448","SPT15455","SPT15456","SPT15459","SPT15471","SPT15483","SPT15486","SPT15491","SPT15504","SPT15506","SPT15511","SPT15517","SPT15519","SPT15522","SPT15528","SPT15529","SPT15531","SPT15532","SPT15535","SPT15536","SPT15537","SPT15541","SPT15542","SPT15543","SPT15545","SPT15547","SPT15548","SPT15549","SPT15558","SPT15562","SPT15563","SPT15564","SPT15565","SPT15566","SPT15567","SPT15571","SPT15577","SPT15579","SPT15580","SPT15586","SPT15587","SPT15588","SPT15589","SPT15590","SPT15591","SPT15593","SPT15594","SPT15595","SPT15598","SPT15599","SPT15600","SPT15602","SPT15603","SPT15604","SPT15605","SPT15606","SPT15607","SPT15608","SPT15609","SPT15611","SPT15612","SPT15614","SPT15615","SPT15616","SPT15617","SPT15619","SPT15620","SPT15621","SPT15622","SPT15623","SPT15625","SPT15626","SPT15627","SPT15628","SPT15630","SPT15631","SPT15632","SPT15633","SPT15634","SPT15635","SPT15636","SPT15637","SPT15638","SPT15639","SPT15640","SPT15641","SPT15642","SPT15643","SPT15644","SPT15645","SPT15646","SPT15647","SPT15649","SPT15650","SPT15653","SPT15654","SPT15655","SPT15656","SPT15658","SPT15659","SPT15660","SPT15663","SPT15665","SPT15666","SPT15668","SPT15669","SPT15670","SPT15671","SPT15672","SPT15678","SPT15679","SPT15680","SPT15681","SPT15682","SPT15683","SPT15684","SPT15685","SPT15686","SPT15687","SPT15688","SPT15689","SPT15690","SPT15691","SPT15692","SPT15694","SPT15695","SPT15696","SPT15697","SPT15698","SPT15699","SPT15700","SPT15701","SPT15702","SPT15703","SPT15704","SPT15705","SPT15706","SPT15707","SPT15708","SPT15709","SPT15710","SPT15711","SPT15712","SPT15713","SPT15714","SPT15715","SPT15716","SPT15717","SPT15718","SPT15719","SPT15720","SPT15721","SPT15722","SPT15723","SPT15724","SPT15725","SPT15726","SPT15727","SPT15728","SPT15729","SPT15730","SPT15731","SPT15732","SPT15733","SPT15734","SPT15735","SPT15736","SPT15737","SPT15738","SPT15740","SPT15742","SPT15744","SPT15745","SPT15747","SPT15749","SPT15752","SPT15753","SPT15756","SPT15757","SPT15758","SPT15761","SPT15764","SPT15766","SPT15767","SPT15768","SPT15770","SPT15772","SPT15773","SPT15775","SPT15777","SPT15779","SPT15782","SPT15785","SPT15787","SPT15789","SPT15792","SPT15794","SPT15797","SPT15799","SPT15800","SPT15804","SPT15805","SPT15807","SPT15808","SPT15810","SPT15814","SPT15817","SPT15818","SPT15820","SPT15823","SPT15832","SPT15834","SPT15838","SPT15846","SPT15849","SPT15850","SPT15851","SPT15852","SPT15853","SPT15854","SPT15855","SPT15856","SPT15857","SPT15858","SPT15859","SPT15860","SPT15861","SPT15862","SPT15863","SPT15865","SPT15866","SPT15867","SPT15869","SPT15870","SPT15871","SPT15873","SPT15874","SPT15875","SPT15876","SPT15877","SPT15879","SPT15880","SPT16597","SPT16598","SPT16601","SPT16604","SPT16605","SPT16609","SPT16612","SPT16614","SPT16617","SPT16618","SPT16622","SPT16628","SPT16630","SPT16633","SPT16638","SPT16642","SPT16646","SPT16647","SPT16654","SPT16658","SPT16662","SPT16666","SPT16669","SPT16672","SPT16676","SPT16680","SPT16684","SPT16685","SPT16686","SPT16689","SPT16691","SPT16694","SPT16696","SPT16699","SPT16700","SPT16703","SPT16707","SPT16709","SPT16710","SPT16711","SPT16715","SPT16716","SPT16718","SPT16719","SPT16720","SPT16723","SPT16726","SPT16730","SPT16734","SPT16737","SPT16738","SPT16739","SPT16742","SPT16744","SPT16745","SPT16747","SPT16748","SPT16749","SPT16753","SPT16755","SPT16759","SPT16762","SPT16766","SPT16767","SPT16768","SPT16770","SPT16771","SPT16778","SPT16782","SPT16787","SPT16788","SPT16789","SPT16790","SPT16793","SPT16795","SPT16798","SPT16799","SPT16800","SPT16803","SPT16804","SPT16805","SPT16806","SPT16807","SPT16811","SPT16812","SPT16813","SPT16816","SPT16817","SPT16818","SPT16819","SPT16821","SPT16823","SPT16824","SPT16826","SPT16827","SPT16828","SPT16829","SPT16830","SPT16831","SPT16834","SPT16835","SPT16836","SPT16837","SPT16838","SPT16839","SPT16840","SPT16843","SPT16845","SPT16846","SPT16847","SPT16848","SPT16849","SPT16850","SPT16851","SPT16868","SPT16869","SPT16870","SPT16871","SPT16872","SPT16873","SPT16874","SPT16875","SPT16876","SPT16877","SPT16878","SPT16879","SPT16880","SPT16882","SPT16883","SPT16884","SPT16885","SPT16886","SPT16887","SPT16888","SPT16889","SPT16890","SPT16891","SPT16893","SPT16894","SPT16896","SPT16897","SPT16898","SPT16899","SPT16900","SPT16901","SPT16902","SPT16904","SPT16905","SPT16906","SPT16907","SPT16908","SPT16909","SPT16910","SPT16911","SPT16912","SPT16914","SPT16915","SPT16916","SPT16917","SPT16919","SPT16920","SPT16921","SPT16922","SPT16923","SPT16924","SPT16925","SPT16926","SPT16927","SPT16928","SPT16929","SPT16932","SPT16934","SPT16936","SPT16937","SPT16938","SPT16939","SPT16940","SPT16941","SPT16942","SPT16943","SPT16944","SPT16946","SPT16947","SPT16948","SPT16949","SPT16950","SPT16951","SPT16953","SPT16954","SPT16955","SPT16957","SPT16958","SPT16959","SPT16961","SPT16962","SPT16963","SPT16964","SPT16966","SPT16967","SPT16968","SPT16969","SPT16970","SPT16971","SPT16972","SPT16973","SPT16976","SPT16977","SPT16978","SPT16982","SPT16983","SPT16985","SPT16986","SPT16987","SPT16988","SPT16989","SPT16994","SPT16995","SPT16996","SPT16997","SPT16998","SPT16999","SPT17000","SPT17001","SPT17002","SPT17003","SPT17004","SPT17006","SPT17007","SPT17009","SPT17010","SPT17011","SPT17012","SPT17013","SPT17015","SPT17017","SPT17018","SPT17020","SPT17022","SPT17023","SPT17025","SPT17027","SPT17028","SPT17029","SPT17030","SPT17031","SPT17034","SPT17035","SPT17036","SPT17037","SPT17039","SPT17040","SPT17042","SPT17043","SPT17044","SPT17045","SPT17046","SPT17047","SPT17049","SPT17050","SPT17051","SPT17052","SPT17053","SPT17054","SPT17055","SPT17056","SPT17057","SPT17058","SPT17059","SPT17060","SPT17061","SPT17062","SPT17063","SPT17064","SPT17065","SPT17066","SPT17067","SPT17069","SPT17070","SPT17072","SPT17073","SPT17075","SPT17076","SPT17077","SPT17078","SPT17079","SPT17793","SPT17794","SPT17795","SPT17800","SPT17802","SPT17803","SPT17804","SPT17805","SPT17806","SPT17807","SPT17808","SPT17810","SPT17811","SPT17812","SPT17813","SPT17816","SPT17817","SPT17818","SPT17819","SPT17821","SPT17823","SPT17825","SPT17826","SPT17828","SPT17829","SPT17830","SPT17832","SPT17833","SPT17836","SPT17837","SPT17838","SPT17841","SPT17843","SPT17844","SPT17845","SPT17852","SPT17853","SPT17855","SPT17876","SPT17878","SPT17881","SPT17894","SPT17902","SPT17916","SPT17917","SPT17918","SPT17922","SPT17924","SPT17927","SPT17929","SPT17931","SPT17933","SPT17936","SPT17937","SPT17938","SPT17940","SPT17941","SPT17942","SPT17943","SPT17944","SPT17945","SPT17947","SPT17949","SPT17950","SPT17951","SPT17954","SPT17955","SPT17956","SPT17958","SPT17960","SPT17963","SPT17965","SPT17966","SPT17969","SPT17970","SPT17972","SPT17975","SPT17978","SPT17981","SPT17987","SPT17990","SPT17993","SPT17995","SPT18001","SPT18002","SPT18003","SPT18018","SPT18026","SPT18037","SPT18045","SPT18049","SPT18050","SPT18052","SPT18055","SPT18056","SPT18058","SPT18421","SPT18423","SPT18427","SPT18428","SPT18429","SPT18431","SPT18434","SPT18435","SPT18436","SPT18440","SPT18445","SPT18448","SPT18453","SPT18457","SPT18458","SPT18459","SPT18461","SPT18462","SPT18465","SPT18466","SPT18468","SPT18470","SPT18472","SPT18473","SPT18474","SPT18475","SPT18478","SPT18479","SPT18482","SPT18490","SPT18491","SPT18492","SPT18493","SPT18523","SPT18525","SPT18535","SPT18537","SPT18545","SPT18549","SPT18552","SPT18553","SPT18554","SPT18555","SPT18557","SPT18558","SPT18559","SPT18561","SPT18562","SPT18566","SPT18572","SPT18575","SPT18576","SPT18578","SPT18579","SPT18580","SPT18581","SPT18583","SPT18584","SPT18585","SPT18586","SPT18589","SPT18591","SPT18592","SPT18595","SPT18598","SPT18603","SPT18604","SPT18607","SPT18609","SPT18611","SPT18612","SPT18613","SPT18614","SPT18617","SPT18618","SPT18619","SPT18620","SPT18621","SPT18622","SPT18623","SPT18624","SPT18625","SPT18626","SPT18629","SPT18631","SPT18633","SPT18634","SPT18636","SPT18637","SPT18638","SPT18639","SPT18640","SPT18641","SPT18644","SPT18646","SPT18647","SPT18649","SPT18650","SPT18652","SPT18653","SPT18654","SPT18655","SPT18657","SPT18660","SPT18661","SPT18662","SPT18663","SPT18669","SPT18670","SPT18671","SPT18673","SPT18674","SPT18678","SPT18680","SPT18681","SPT18682","SPT18683","SPT18685","SPT18687","SPT18689","SPT18929","SPT18932","SPT18937","SPT18940","SPT18941","SPT18942","SPT18943","SPT18947","SPT18949","SPT18952","SPT18955","SPT18956","SPT18957","SPT18958","SPT18959","SPT18962","SPT18968","SPT18969","SPT18970","SPT18971","SPT18973","SPT18975","SPT18976","SPT18978","SPT18980","SPT18981","SPT18983","SPT18984","SPT18985","SPT18986","SPT18987","SPT18988","SPT18989","SPT18991","SPT18992","SPT18993","SPT18994","SPT18995","SPT19002","SPT19003","SPT19005","SPT19011","SPT19012","SPT19013","SPT19014","SPT19017","SPT19018","SPT19019","SPT19021","SPT19023","SPT19024","SPT19025","SPT19026","SPT19028","SPT19029","SPT19030","SPT19031","SPT19032","SPT19034","SPT19236","SPT19238","SPT19241","SPT19242","SPT19243","SPT19246","SPT19248","SPT19251","SPT19254","SPT19256","SPT19258","SPT19262","SPT19265","SPT19268","SPT19270","SPT19273","SPT19275","SPT19279","SPT19281","SPT19285","SPT19287","SPT19289","SPT19291","SPT19293","SPT19294","SPT19296","SPT19297","SPT19301","SPT19306","SPT19308","SPT19309","SPT19311","SPT19315","SPT19316","SPT19319","SPT19321","SPT19324","SPT19326","SPT19328","SPT19329","SPT19332","SPT19336","SPT19337","SPT19338","SPT19340","SPT19341","SPT19347","SPT19350","SPT19351","SPT19353","SPT19356","SPT19359","SPT19360","SPT19361","SPT19363","SPT19364","SPT19367","SPT19368","SPT19371","SPT19372","SPT19374","SPT19376","SPT19377","SPT19382","SPT19383","SPT19385","SPT19387","SPT19388","SPT19390","SPT19391","SPT19394","SPT19396","SPT19397","SPT19399","SPT19400","SPT19404","SPT19406","SPT19407","SPT19409","SPT19411","SPT19415","SPT19416","SPT19419","SPT19421","SPT19423","SPT19429","SPT19433","SPT19436","SPT19440","SPT19443","SPT19446","SPT19448","SPT19487","SPT19489","SPT19492","SPT19493","SPT19494","SPT19495","SPT19496","SPT19498","SPT19499","SPT19500","SPT19503","SPT19505","SPT19506","SPT19510","SPT19512","SPT19514","SPT19517","SPT19521","SPT19524","SPT19525","SPT19527","SPT19529","SPT19530","SPT19532","SPT19533","SPT19539","SPT19540","SPT19542","SPT19547","SPT19548","SPT19549","SPT19550","SPT19558","SPT19559","SPT19564","SPT19565","SPT19567","SPT19569","SPT19573","SPT19581","SPT19582","SPT19587","SPT19588","SPT19589","SPT19593","SPT19594","SPT19595","SPT19597","SPT19600","SPT19601","SPT19602","SPT19603","SPT19604","SPT19606","SPT19607","SPT19610","SPT19612","SPT19613","SPT19614","SPT19616","SPT19618","SPT19621","SPT19627","SPT19629","SPT19631","SPT19636","SPT19637","SPT19641","SPT19645","SPT19646","SPT19649","SPT19656","SPT19657","SPT19658","SPT19659","SPT19660","SPT19665","SPT19668","SPT19673","SPT19677","SPT19678","SPT19679","SPT19680","SPT19681","SPT19682","SPT19685","SPT19689","SPT19691","SPT19692","SPT19693","SPT19694","SPT19696","SPT19699","SPT19701","SPT19704","SPT19706","SPT19708","SPT19712","SPT19715","SPT19716","SPT19718","SPT19720","SPT19721","SPT19728","SPT19729","SPT19733","SPT19742","SPT19743","SPT19746","SPT19749","SPT19753","SPT19755","SPT19756","SPT19757","SPT19763","SPT19764","SPT19765","SPT19766","SPT19767","SPT19768","SPT19772","SPT19774","SPT19782","SPT19789","SPT19790","SPT19791","SPT19792","SPT19811","SPT19821","SPT19824","SPT19832","SPT19835","SPT19840","SPT19841","SPT19842","SPT19843","SPT19845","SPT19846","SPT19847","SPT19850","SPT19851","SPT19854","SPT19855","SPT19856","SPT19857","SPT19859","SPT19860","SPT19863","SPT19864","SPT19865","SPT19867","SPT19868","SPT19869","SPT19870","SPT19871","SPT19872","SPT19874","SPT19875","SPT19876","SPT19880","SPT19882","SPT19883","SPT19884","SPT19886","SPT19888","SPT19889","SPT19890","SPT19891","SPT19892","SPT19893","SPT19895","SPT19896","SPT19897","SPT19899","SPT19902","SPT19903","SPT19905","SPT19906","SPT19907","SPT19908","SPT19909","SPT19910","SPT19914","SPT19919","SPT19920","SPT19921","SPT19922","SPT19924","SPT19925","SPT19927","SPT19928","SPT19929","SPT19930","SPT19931","SPT19932","SPT19935","SPT19936","SPT19938","SPT19939","SPT19940","SPT19941","SPT19943","SPT19944","SPT19945","SPT19946","SPT19947","SPT19949","SPT19950","SPT19951","SPT19952","SPT19953","SPT19954","SPT19955","SPT19956","SPT19957","SPT19959","SPT19960","SPT19961","SPT19962","SPT19963","SPT19964","SPT19966","SPT19967","SPT19968","SPT19969","SPT19970","SPT19971","SPT19972","SPT19973","SPT19974","SPT19976","SPT19977","SPT19979","SPT19980","SPT19981","SPT19983","SPT19985","SPT19987","SPT19988","SPT19990","SPT19991","SPT19992","SPT19995","SPT19996","SPT19997","SPT19998","SPT19999","SPT20000","SPT20001","SPT20002","SPT20003","SPT20005","SPT20006","SPT20007","SPT20008","SPT20009","SPT20011","SPT20012","SPT20013","SPT20014","SPT20015","SPT20016","SPT20019","SPT20020","SPT20025","SPT20026","SPT20028","SPT20029","SPT20030","SPT20031","SPT20032","SPT20036","SPT20037","SPT20038","SPT20040","SPT20042","SPT20043","SPT20044","SPT20045","SPT20490","SPT20491","SPT20494","SPT20495","SPT20498","SPT20499","SPT20507","SPT20509","SPT20510","SPT20519","SPT20526","SPT20532","SPT20533","SPT20537","SPT20541","SPT20546","SPT20550","SPT20556","SPT20557","SPT20562","SPT20563","SPT20565","SPT20569","SPT21249","SPT21250","SPT21251","SPT21255","SPT21261","SPT21270","SPT21277","SPT21278","SPT21279","SPT21281","SPT21283","SPT21284","SPT21294","SPT21295","SPT21300","SPT21302","SPT21305","SPT21306","SPT21309","SPT21319","SPT21321","SPT21322","SPT21323","SPT21324","SPT21326","SPT21331","SPT21659","SPT21661","SPT21667","SPT21668","SPT21675","SPT21677","SPT21685","SPT21686","SPT21690","SPT21691","SPT21692","SPT21705","SPT21707","SPT21708","SPT21710","SPT21712","SPT21713","SPT21714","SPT21715","SPT21716","SPT21717","SPT21719","SPT21721","SPT21722","SPT21723","SPT21724","SPT21725","SPT21726","SPT21727","SPT21730","SPT21731","SPT21732","SPT21734","SPT21735","SPT21737","SPT21738","SPT21739","SPT21740","SPT21741","SPT21742","SPT21743","SPT21744","SPT21750","SPT21751","SPT21756","SPT21759","SPT21765","SPT21769","SPT21773","SPT21774","SPT21778","SPT21785","SPT21786","SPT21788","SPT21789","SPT21793","SPT21796","SPT21797","SPT21804","SPT21805","SPT21807","SPT21808","SPT21813","SPT21817","SPT21818","SPT21824","SPT21831","SPT21833","SPT21835","SPT21836","SPT21837","SPT21838","SPT21839","SPT21840","SPT21841","SPT21844","SPT21846","SPT21863","SPT21866","SPT21872","SPT21873","SPT21874","SPT21875","SPT21876","SPT21877","SPT21878","SPT21880","SPT21883","SPT21884","SPT21885","SPT21887","SPT21899","SPT21906","SPT21909","SPT21910","SPT21913","SPT21914","SPT21915","SPT21916","SPT21919","SPT21923","SPT21924","SPT21927","SPT21946","SPT21948","SPT21953","SPT21954","SPT21955","SPT21965","SPT21966","SPT21967","SPT21979","SPT22200","SPT22201","SPT22204","SPT22207","SPT22211","SPT22213","SPT22214","SPT22220","SPT22221","SPT22222","SPT22223","SPT22227","SPT22230","SPT22238","SPT22240","SPT22241","SPT22243","SPT22248","SPT22249","SPT22251","SPT22252","SPT22253","SPT22254","SPT22255","SPT22256","SPT22258","SPT22261","SPT22265","SPT22270","SPT22273","SPT22274","SPT22278","SPT22281","SPT22282","SPT22407","SPT22410","SPT22415","SPT22416","SPT22422","SPT22438","SPT22441","SPT22445","SPT22447","SPT22449","SPT22456","SPT22457","SPT22467","SPT22469","SPT22471","SPT22474","SPT22480","SPT22485","SPT22489","SPT22500","SPT22503","SPT22509","SPT22510","SPT22511","SPT22518","SPT22520","SPT22525","SPT22526","SPT22527","SPT22529","SPT22536","SPT22539","SPT22545","SPT22547","SPT22551","SPT22552","SPT22553","SPT22554","SPT22555","SPT22556","SPT22558","SPT22559","SPT22562","SPT22565","SPT22566","SPT22567","SPT22568","SPT22570","SPT22571","SPT22572","SPT22574","SPT22575","SPT22577","SPT22578","SPT22580","SPT22584","SPT22587","SPT22588","SPT22591","SPT22592","SPT22593","SPT22594","SPT22598","SPT22600","SPT22605","SPT22606","SPT22607","SPT22608","SPT22610","SPT22612","SPT22613","SPT22615","SPT22616","SPT22617","SPT22619","SPT22620","SPT22621","SPT22628","SPT22634","SPT22637","SPT22638","SPT22640","SPT22642","SPT22649","SPT23007","SPT23009","SPT23010","SPT23011","SPT23012","SPT23014","SPT23015","SPT23018","SPT23019","SPT23020","SPT23021","SPT23022","SPT23023","SPT23028","SPT23029","SPT23031","SPT23032","SPT23035","SPT23036","SPT23039","SPT23040","SPT23043","SPT23047","SPT23052","SPT23053","SPT23055","SPT23059","SPT23065","SPT23067","SPT23069","SPT23071","SPT23073","SPT23079","SPT23082","SPT23085","SPT23086","SPT23087","SPT23091","SPT23095","SPT23098","SPT23105","SPT23106","SPT23107","SPT23108","SPT23110","SPT23114","SPT23116","SPT23118","SPT23123","SPT23125","SPT23127","SPT24139","SPT24141","SPT24142","SPT24147","SPT24150","SPT24151","SPT24152","SPT24153","SPT24154","SPT24159","SPT24160","SPT24161","SPT24162","SPT24163","SPT24164","SPT24165","SPT24166","SPT24167","SPT24173","SPT24175","SPT24180","SPT24181","SPT24182","SPT24183","SPT24184","SPT24188","SPT24189","SPT24191","SPT24192","SPT24193","SPT24373","SPT24374","SPT24375","SPT24376","SPT24379","SPT24380","SPT24381","SPT24383","SPT24385","SPT24386","SPT24388","SPT24392","SPT24393","SPT24394","SPT24395","SPT24399","SPT24410","SPT24411","SPT24413","SPT24414","SPT24415","SPT24419","SPT24421","SPT24423","SPT24424","SPT24426","SPT24432","SPT24433","SPT24436","SPT24442","SPT24445","SPT24447","SPT24448","SPT24449","SPT24451","SPT24452","SPT24454","SPT24455","SPT24456","SPT24457","SPT24458","SPT24459","SPT24461","SPT24462","SPT24463","SPT24464","SPT24481","SPT24486","SPT24489","SPT24490","SPT24491","SPT24493","SPT24496","SPT24497","SPT24499","SPT24500","SPT24501","SPT24502","SPT24503","SPT24504","SPT24510","SPT24512","SPT24513","SPT24516","SPT24518","SPT24519","SPT24520","SPT24521","SPT24522","SPT24523","SPT24525","SPT24526","SPT24527","SPT24528","SPT24529","SPT24530","SPT24531","SPT24532","SPT24533","SPT24535","SPT24538","SPT24539","SPT24540","SPT24541","SPT24542","SPT24543","SPT24546","SPT24547","SPT24548","SPT24553","SPT24554","SPT24555","SPT24557","SPT24558","SPT24560","SPT24561","SPT24562","SPT24563","SPT24564","SPT24566","SPT24567","SPT24568","SPT24581","SPT24583","SPT24584","SPT24585","SPT24587","SPT24588","SPT24589","SPT24590","SPT24591","SPT24592","SPT24593","SPT24595","SPT24601","SPT24602","SPT24603","SPT24604","SPT24607","SPT24608","SPT24610","SPT24613","SPT24616","SPT24617","SPT24618","SPT24620","SPT24621","SPT24624","SPT24625","SPT24626","SPT24627","SPT24628","SPT24630","SPT24634","SPT24635","SPT24636","SPT24637","SPT24638","SPT24639","SPT24641","SPT24642","SPT24643","SPT24644","SPT24645","SPT24646","SPT24647","SPT24648","SPT24649","SPT24650","SPT24651","SPT24653","SPT24654","SPT24656","SPT24657","SPT24658","SPT24659","SPT24660","SPT24661","SPT24662","SPT24663","SPT24664","SPT24665","SPT24666","SPT24667","SPT24668","SPT24669","SPT24670","SPT24671","SPT24672","SPT24673","SPT24674","SPT24675","SPT24676","SPT24677","SPT24678","SPT24679","SPT24680","SPT24681","SPT24682","SPT24683","SPT24684","SPT24685","SPT24686","SPT24687","SPT24688","SPT24689","SPT24690","SPT24691","SPT24692","SPT24693","SPT24694","SPT24695","SPT24708","SPT24709","SPT24710","SPT24714","SPT24716","SPT24717","SPT24719","SPT24720","SPT24723","SPT24724","SPT24725","SPT24730","SPT24733","SPT24737","SPT24738","SPT24740","SPT24741","SPT24742","SPT24743","SPT24744","SPT24748","SPT24749","SPT24755","SPT24756","SPT24758","SPT24759","SPT24760","SPT24761","SPT24763","SPT24764","SPT24766","SPT24770","SPT24771","SPT24772","SPT24773","SPT24774","SPT24776","SPT24777","SPT24778","SPT24780","SPT24781","SPT24782","SPT24783","SPT24785","SPT24786","SPT24787","SPT24789","SPT24790","SPT24792","SPT24795","SPT24796","SPT24797","SPT24801","SPT24802","SPT24804","SPT24807","SPT24809","SPT24812","SPT24814","SPT24819","SPT24826","SPT24853","SPT24861","SPT24863","SPT24876","SPT24885","SPT24886","SPT24898","SPT24899","SPT24953","SPT24997","SPT25002","SPT25003","SPT25008","SPT25012","SPT25013","SPT25016","SPT25034","SPT25103","SPT25104","SPT25106","SPT25108","SPT25109","SPT25110","SPT25111","SPT25112","SPT25114","SPT25115","SPT25116","SPT25118","SPT25119","SPT25120","SPT25121","SPT25122","SPT25123","SPT25124","SPT25125","SPT25126","SPT25127","SPT25128","SPT25129","SPT25131","SPT25132","SPT25133","SPT25135","SPT25136","SPT25138","SPT25139","SPT25144","SPT25145","SPT25146","SPT25147","SPT25148","SPT25149","SPT25151","SPT25154","SPT25157","SPT25158","SPT25159","SPT25160","SPT25162","SPT26227","SPT26228","SPT26229","SPT26230","SPT26232","SPT26233","SPT26234","SPT26235","SPT26237","SPT26238","SPT26240","SPT26242","SPT26243","SPT26244","SPT26248","SPT26254","SPT26256","SPT26258","SPT26260","SPT26262","SPT26263","SPT26265","SPT26266","SPT26269","SPT26270","SPT26273","SPT26274","SPT26275","SPT26276","SPT26278","SPT26281","SPT26282","SPT26283","SPT26286","SPT26288","SPT26289","SPT26290","SPT26293","SPT26294","SPT26295","SPT26296","SPT26300","SPT26302","SPT26304","SPT26305","SPT26307","SPT26308","SPT26315","SPT26319","SPT26322","SPT26324","SPT26325","SPT26326","SPT26327","SPT26329","SPT26331","SPT26335","SPT26336","SPT26337","SPT26338","SPT26339","SPT26340","SPT26341","SPT26342","SPT26343","SPT26345","SPT26360","SPT26361","SPT26362","SPT26363","SPT26364","SPT26366","SPT26367","SPT26369","SPT26370","SPT26371","SPT26372","SPT26374","SPT26375","SPT26380","SPT26381","SPT26382","SPT26384","SPT26386","SPT26387","SPT26389","SPT26390","SPT26396","SPT26397","SPT26399","SPT26400","SPT26401","SPT26402","SPT26403","SPT26408","SPT26410","SPT26413","SPT26415","SPT26418","SPT26424","SPT26426","SPT26427","SPT26428","SPT26429","SPT26430","SPT26431","SPT26434","SPT26440","SPT26441","SPT26442","SPT26443","SPT26445","SPT26446","SPT26447","SPT26449","SPT26450","SPT26451","SPT26452","SPT26453","SPT26454","SPT26455","SPT26456","SPT26457","SPT26458","SPT26460","SPT26461","SPT26463","SPT26464","SPT26465","SPT26466","SPT26468","SPT26469","SPT26470","SPT26478","SPT26480","SPT26483","SPT26484","SPT26486","SPT26487","SPT26489","SPT26490","SPT26491","SPT26492","SPT26493","SPT26494","SPT26495","SPT26496","SPT26497","SPT26502","SPT26503","SPT26504","SPT26505","SPT26506","SPT26507","SPT26508","SPT26509","SPT26510","SPT26511","SPT26512","SPT26513","SPT26514","SPT26515","SPT26516","SPT26517","SPT26518","SPT26519","SPT26520","SPT26521","SPT26522","SPT26523","SPT26524","SPT26525","SPT26526","SPT26527","SPT26528","SPT26529","SPT26530","SPT26531","SPT26532","SPT26533","SPT26534","SPT26535","SPT26536","SPT26537","SPT26538","SPT26539","SPT26540","SPT26541","SPT26542","SPT26543","SPT26545","SPT26546","SPT26547","SPT26548","SPT26549","SPT26551","SPT26554","SPT26555","SPT26557","SPT26558","SPT26559","SPT26560","SPT26561","SPT26563","SPT26564","SPT26566","SPT26569","SPT26572","SPT26573","SPT26575","SPT26576","SPT26577","SPT26579","SPT26580","SPT26581","SPT26583","SPT26584","SPT26585","SPT26586","SPT26588","SPT26589","SPT26591","SPT26594","SPT26595","SPT26601","SPT26602","SPT26603","SPT26605","SPT26612","SPT26613","SPT26615","SPT26616","SPT26618","SPT26619","SPT26621","SPT26623","SPT26624","SPT26625","SPT26626","SPT26627","SPT26628","SPT26629","SPT26630","SPT26633","SPT26634","SPT26635","SPT26638","SPT26640","SPT26642","SPT26833","SPT26835","SPT26837","SPT26846","SPT26847","SPT26849","SPT26853","SPT26857","SPT26859","SPT26860","SPT26861","SPT26862","SPT26864","SPT26865","SPT26866","SPT26867","SPT26868","SPT26870","SPT26874","SPT26875","SPT26877","SPT26878","SPT26879","SPT26883","SPT26884","SPT26885","SPT26886","SPT26888","SPT26889","SPT26891","SPT26892","SPT26894","SPT26895","SPT26897","SPT26899","SPT26900","SPT26904","SPT26906","SPT26907","SPT26908","SPT26909","SPT26910","SPT26911","SPT26915","SPT26918","SPT26919","SPT26922","SPT26925","SPT26926","SPT26927","SPT26928","SPT26929","SPT26931","SPT26932","SPT26935","SPT26936","SPT26937","SPT26938","SPT26939","SPT26940","SPT26942","SPT26943","SPT26944","SPT26945","SPT26946","SPT26947","SPT26950","SPT26951","SPT26955","SPT26961","SPT26963","SPT26968","SPT26972","SPT26973","SPT26976","SPT26978","SPT26979","SPT34224","SPT34225","SPT34226","SPT34227","SPT34229","SPT34230","SPT34231","SPT34232","SPT34233","SPT34234","SPT34235","SPT34236","SPT34237","SPT34238","SPT34239","SPT34240","SPT34241","SPT34242","SPT34243","SPT34244","SPT34246","SPT34247","SPT34248","SPT34249","SPT34250","SPT34251","SPT34252","SPT34253","SPT34254","SPT34255","SPT34256","SPT34257","SPT34258","SPT34259","SPT34260","SPT34261","SPT34262","SPT34263","SPT34264","SPT34265","SPT34266","SPT34267","SPT34269","SPT34270","SPT34272","SPT34273","SPT34274","SPT34275","SPT34276","SPT34279","SPT34280","SPT34281","SPT34283","SPT34284","SPT34285","SPT34286","SPT34287","SPT34288","SPT34289","SPT34290","SPT34292","SPT34293","SPT34294","SPT34295","SPT34296","SPT34297","SPT34298","SPT34299","SPT34300","SPT34301","SPT34302","SPT34303","SPT34304","SPT34307","SPT34308","SPT34309","SPT34310","SPT34311","SPT34312","SPT34314","SPT34315","SPT34316","SPT34317","SPT34318","SPT34319","SPT34320","SPT34321","SPT34322","SPT34323","SPT34324","SPT34325","SPT34327","SPT34328","SPT34330","SPT34331","SPT34332","SPT34334","SPT34335","SPT34336","SPT34337","SPT34338","SPT34340","SPT34344","SPT34345","SPT34349","SPT34352","SPT34353","SPT34355","SPT34356","SPT34357","SPT34358","SPT34359","SPT34362","SPT34363","SPT34366","SPT34368","SPT34369","SPT34370","SPT34371","SPT34372","SPT34373","SPT34377","SPT34378","SPT34379","SPT34380","SPT34381","SPT34382","SPT34383","SPT34386","SPT34387","SPT34388","SPT34389","SPT34390","SPT34391","SPT34393","SPT34394","SPT34395","SPT34396","SPT34398","SPT34399","SPT34400","SPT34402","SPT34404","SPT34405","SPT34406","SPT34407","SPT34408","SPT34410","SPT34411","SPT34412","SPT34413","SPT34414","SPT34415","SPT34416","SPT34417","SPT34418","SPT34420","SPT34421","SPT34424","SPT34426","SPT34427","SPT34428","SPT34429","SPT34431","SPT34433","SPT34434","SPT34435","SPT34436","SPT34437","SPT34438","SPT34439","SPT34440","SPT34441","SPT34443","SPT34444","SPT34445","SPT34449","SPT34450","SPT34451","SPT34452","SPT34453","SPT34459","SPT34460","SPT34461","SPT34463","SPT34464","SPT34465","SPT34466","SPT34467","SPT34468","SPT34469","SPT34470","SPT34471","SPT34472","SPT34473","SPT34474","SPT34475","SPT34476","SPT34479","SPT34480","SPT34481","SPT34482","SPT34484","SPT34485","SPT34486","SPT34487","SPT34489","SPT34490","SPT34491","SPT34492","SPT34493","SPT34494","SPT34495","SPT34496","SPT34497","SPT34498","SPT34499","SPT34500","SPT34501","SPT34502","SPT34503","SPT34504","SPT34507","SPT34508","SPT34509","SPT34510","SPT34512","SPT34513","SPT34514","SPT34515","SPT34517","SPT34518","SPT34519","SPT34520","SPT34615","SPT34616","SPT34618","SPT34620","SPT34621","SPT34622","SPT34623","SPT34624","SPT34625","SPT34626","SPT34627","SPT34628","SPT34629","SPT34630","SPT34631","SPT34632","SPT34757","SPT34759","SPT34765","SPT34769","SPT34771","SPT34772","SPT34773","SPT34774","SPT34776","SPT34779","SPT34781","SPT34782","SPT34784","SPT34786","SPT34788","SPT34790","SPT34792","SPT34794","SPT34796","SPT34797","SPT34798","SPT34800","SPT34801","SPT34805","SPT34806","SPT34807","SPT34808","SPT34809","SPT34811","SPT34812","SPT34813","SPT34815","SPT34816","SPT34817","SPT34819","SPT34820","SPT34821","SPT34822","SPT34823","SPT34824","SPT34825","SPT34826","SPT34827","SPT34828","SPT34830","SPT34831","SPT34832","SPT34835","SPT34836","SPT34838","SPT34839","SPT34840","SPT34841","SPT34844","SPT34845","SPT34846","SPT34848","SPT34849","SPT34850","SPT34851","SPT34852","SPT34853","SPT34854","SPT34856","SPT34857","SPT34858","SPT34860","SPT34861","SPT34863","SPT34864","SPT34865","SPT34866","SPT34867","SPT34868","SPT34869","SPT34871","SPT34872","SPT34873","SPT34875","SPT34876","SPT34877","SPT34879","SPT34881","SPT34882","SPT34884","SPT34885","SPT34886","SPT34887","SPT34889","SPT34890","SPT34891","SPT34892","SPT34893","SPT34894","SPT34895","SPT34896","SPT34898","SPT34900","SPT34901","SPT34902","SPT34903","SPT34904","SPT34905","SPT34906","SPT34908","SPT34909","SPT34910","SPT34912","SPT34913","SPT34914","SPT34915","SPT34916","SPT34918","SPT34920","SPT34922","SPT34924","SPT34926","SPT34927","SPT34929","SPT34930","SPT34931","SPT34933","SPT34934","SPT34936","SPT34937","SPT34938","SPT34940","SPT34943","SPT34944","SPT34945","SPT34946","SPT34947","SPT34950","SPT34952","SPT34953","SPT34955","SPT34956","SPT34957","SPT34958","SPT34959","SPT34960","SPT34961","SPT34964","SPT34969","SPT34970","SPT34971","SPT34972","SPT34974","SPT34976","SPT34977","SPT34979","SPT34980","SPT34981","SPT34983","SPT34985","SPT34987","SPT34988","SPT34989","SPT34990","SPT34991","SPT34993","SPT34994","SPT34995","SPT34996","SPT34997","SPT35000","SPT35001","SPT35004","SPT35005","SPT35008","SPT35009","SPT35011","SPT35012","SPT35014","SPT35015","SPT35017","SPT35018","SPT35019","SPT35020","SPT35021","SPT35023","SPT35026","SPT35028","SPT35030","SPT35031","SPT35032","SPT35033","SPT35036","SPT35037","SPT35038","SPT35039","SPT35040","SPT35042","SPT35044","SPT35045","SPT35047","SPT35050","SPT35052","SPT35060","SPT35065","SPT35066","SPT35069","SPT35070","SPT35071","SPT35072","SPT35077","SPT35078","SPT35079","SPT35080","SPT35085","SPT35087","SPT35091","SPT35092","SPT35093","SPT35094","SPT35096","SPT35099","SPT35101","SPT35102","SPT35103","SPT35104","SPT35105","SPT35106","SPT35107","SPT35109","SPT35114","SPT35115","SPT35116","SPT35117","SPT35121","SPT35125","SPT35128","SPT35130","SPT35132","SPT35133","SPT35134","SPT35135","SPT35136","SPT35137","SPT35139","SPT35140","SPT35141","SPT35142","SPT35145","SPT35147","SPT35148","SPT35149","SPT35150","SPT35151","SPT35152","SPT35153","SPT35154","SPT35155","SPT35156","SPT35157","SPT35158","SPT35159","SPT35160","SPT35161","SPT35162","SPT35163","SPT35164","SPT35165","SPT35166","SPT35167","SPT35168","SPT35169","SPT35170","SPT35171","SPT35172","SPT35173","SPT35174","SPT35175","SPT35176","SPT35177","SPT35178","SPT35179","SPT35180","SPT35181","SPT35182","SPT35183","SPT35185","SPT35186","SPT35187","SPT35188","SPT35189","SPT35190","SPT35191","SPT35192","SPT35193","SPT35194","SPT35198","SPT35199","SPT35200","SPT35201","SPT35202","SPT35204","SPT35206","SPT35207","SPT35208","SPT35209","SPT35210","SPT35211","SPT35212","SPT35213","SPT35214","SPT35216","SPT35217","SPT35218","SPT35219","SPT35220","SPT35221","SPT35222","SPT35223","SPT35224","SPT35227","SPT35228","SPT35229","SPT35230","SPT35231","SPT35232","SPT35233","SPT35234","SPT35235","SPT35236","SPT35237","SPT35238","SPT35239","SPT35240","SPT35241","SPT35243","SPT35244","SPT35245","SPT35246","SPT35247","SPT35248","SPT35249","SPT35250","SPT35251","SPT35252","SPT35253","SPT35254","SPT35255","SPT35256","SPT35257","SPT35258","SPT35260","SPT35261","SPT35262","SPT35263","SPT35264","SPT35265","SPT35266","SPT35267","SPT35268","SPT35269","SPT35270","SPT35271","SPT35272","SPT35274","SPT35275","SPT35276","SPT35277","SPT35278","SPT35279","SPT35280","SPT35281","SPT35282","SPT35283","SPT35284","SPT35285","SPT35286","SPT35287","SPT35288","SPT35289","SPT35290","SPT35291","SPT35293","SPT35294","SPT35295","SPT35296","SPT35297","SPT35298","SPT35299","SPT35300","SPT35301","SPT35302","SPT35303","SPT35304","SPT35306","SPT35308","SPT35309","SPT35310","SPT35313","SPT35315","SPT35316","SPT35317","SPT35318","SPT35320","SPT35321","SPT35322","SPT35323","SPT35324","SPT35325","SPT35328","SPT35330","SPT35331","SPT35332","SPT35333","SPT35334","SPT35336","SPT35337","SPT35338","SPT35339","SPT35340","SPT35341","SPT35342","SPT35344","SPT35346","SPT35347","SPT35348","SPT35349","SPT35351","SPT35352","SPT35354","SPT35355","SPT35356","SPT35357","SPT35360","SPT35361","SPT35362","SPT35363","SPT35364","SPT35365","SPT35366","SPT35368","SPT35369","SPT35370","SPT35372","SPT35373","SPT35374","SPT35377","SPT35378","SPT35379","SPT35382","SPT35384","SPT35389","SPT35390","SPT35405","SPT35407","SPT35410","SPT35412","SPT35414","SPT35417","SPT35418","SPT35419","SPT35420","SPT35423","SPT35424","SPT35425","SPT35427","SPT35430","SPT35431","SPT35435","SPT35436","SPT35437","SPT35440","SPT35443","SPT35444","SPT35446","SPT35447","SPT35448","SPT35449","SPT35450","SPT35453","SPT35458","SPT35459","SPT35461","SPT35462","SPT35463","SPT35467","SPT35468","SPT35469","SPT35470","SPT35471","SPT35472","SPT35474","SPT35475","SPT35476","SPT35485","SPT35487","SPT35499","SPT35500","SPT35501","SPT35502","SPT35503","SPT35506","SPT35508","SPT35509","SPT35510","SPT35511","SPT35514","SPT35515","SPT35516","SPT35517","SPT35518","SPT35519","SPT35520","SPT35521","SPT35522","SPT35523","SPT35524","SPT35525","SPT35526","SPT35529","SPT35532","SPT35537","SPT35538","SPT35540","SPT35541","SPT35543","SPT35544","SPT35546","SPT35547","SPT35551","SPT35552","SPT35553","SPT35556","SPT35557","SPT35561","SPT35563","SPT35564","SPT35566","SPT35567","SPT35571","SPT35573","SPT35575","SPT35576","SPT35577","SPT35580","SPT35581","SPT35583","SPT35584","SPT35585","SPT35587","SPT35588","SPT35589","SPT35590","SPT35592","SPT35673","SPT35681","SPT35752","SPT35766","SPT35780","SPT35835","SPT35850","SPT35876","SPT35877","SPT35879","SPT35880","SPT35883","SPT35888","SPT35896","SPT35902","SPT35904","SPT35922","SPT35923","SPT35924","SPT35933","SPT36041","SPT36045","SPT36050","SPT36051","SPT36055","SPT36057","SPT36058","SPT36059","SPT36060","SPT36061","SPT36066","SPT36069","SPT36072","SPT36076","SPT36079","SPT36085","SPT36087","SPT36089","SPT36090","SPT36094","SPT36095","SPT36096","SPT36097","SPT36099","SPT36102","SPT36104","SPT36106","SPT36107","SPT36108","SPT36109","SPT36110","SPT36111","SPT36113","SPT36114","SPT36116","SPT36117","SPT36119","SPT36121","SPT36122","SPT36123","SPT36124","SPT36126","SPT36127","SPT36128","SPT36151","SPT36157","SPT36159","SPT36161","SPT36163","SPT36165","SPT36168","SPT36175","SPT36179","SPT36185","SPT36187","SPT36189","SPT36190","SPT36194","SPT36200","SPT36205","SPT36210","SPT36211","SPT36217","SPT36221","SPT36230","SPT36233","SPT36238","SPT36242","SPT36243","SPT36245","SPT36247","SPT36250","SPT36251","SPT36253","SPT36254","SPT36255","SPT36258","SPT36260","SPT36262","SPT36263","SPT36264","SPT36265","SPT36266","SPT36269","SPT36270","SPT36271","SPT36273","SPT36277","SPT36281","SPT36283","SPT36285","SPT36287","SPT36290","SPT36291","SPT36292","SPT36293","SPT36294","SPT36296","SPT36298","SPT36301","SPT36302","SPT36306","SPT36307","SPT36310","SPT36311","SPT36314","SPT36316","SPT36317","SPT36319","SPT36344","SPT36350","SPT36356","SPT36357","SPT36358","SPT36361","SPT36363","SPT36369","SPT36375","SPT36379","SPT36384","SPT36386","SPT36415","SPT36446","SPT36477","SPT36483","SPT36485","SPT36495","SPT36599","SPT36600","SPT36601","SPT36602","SPT36603","SPT36605","SPT36606","SPT36607","SPT36609","SPT36610","SPT36612","SPT36613","SPT36614","SPT36616","SPT36617","SPT36618","SPT36619","SPT36621","SPT36622","SPT36624","SPT36625","SPT36626","SPT36627","SPT36628","SPT36629","SPT36630","SPT36632","SPT36633","SPT36635","SPT36636","SPT36637","SPT36638","SPT36640","SPT36641","SPT36642","SPT36643","SPT36644","SPT36645","SPT36646","SPT36647","SPT36648","SPT36649","SPT36650","SPT36651","SPT36653","SPT36654","SPT36655","SPT36656","SPT36657","SPT36658","SPT36659","SPT36660","SPT36661","SPT36662","SPT36664","SPT36665","SPT36666","SPT36667","SPT36669","SPT36670","SPT36671","SPT36672","SPT36673","SPT36674","SPT36675","SPT36676","SPT36677","SPT36678","SPT36679","SPT36680","SPT36681","SPT36682","SPT36683","SPT36684","SPT36685","SPT36686","SPT36687","SPT36688","SPT36689","SPT36690","SPT36691","SPT36693","SPT36694","SPT36696","SPT36697","SPT36703","SPT36705","SPT36709","SPT36710","SPT36711","SPT36713","SPT36861","SPT36862","SPT36863","SPT36864","SPT36865","SPT36866","SPT36867","SPT36868","SPT36869","SPT36870","SPT36871","SPT36872","SPT36874","SPT36875","SPT36876","SPT36877","SPT36878","SPT36879","SPT36880","SPT36881","SPT36882","SPT36883","SPT36884","SPT36885","SPT36886","SPT36887","SPT36888","SPT36889","SPT36890","SPT36891","SPT36892","SPT36893","SPT36894","SPT36896","SPT36897","SPT36898","SPT36899","SPT36900","SPT36901","SPT36902","SPT36903","SPT36904","SPT36905","SPT36906","SPT36907","SPT36908","SPT36909","SPT36910","SPT36911","SPT36912","SPT36913","SPT36914","SPT36915","SPT36916","SPT36918","SPT36919","SPT36920","SPT36921","SPT36922","SPT36923","SPT36924","SPT36925","SPT36926","SPT38302","SPT38303","SPT38304","SPT38305","SPT38306","SPT38307","SPT38308","SPT38311","SPT38314","SPT38315","SPT38317","SPT38318","SPT38319","SPT38323","SPT38325","SPT38329","SPT38330","SPT38331","SPT38332","SPT38333","SPT38334","SPT38336","SPT38337","SPT38341","SPT38343","SPT38344","SPT38345","SPT38346","SPT38347","SPT38348","SPT38349","SPT38350","SPT38351","SPT38352","SPT38353","SPT38354","SPT38355","SPT38356","SPT38359","SPT38360","SPT38363","SPT38364","SPT38365","SPT38366","SPT38367","SPT38368","SPT38370","SPT38372","SPT38373","SPT38374","SPT38375","SPT38376","SPT38380","SPT38383","SPT38387","SPT38388","SPT38389","SPT38390","SPT38391","SPT38392","SPT38393","SPT38394","SPT38502","SPT38504","SPT38519","SPT38537","SPT38540","SPT40497","SPT40498","SPT40499","SPT40501","SPT40502","SPT40503","SPT40505","SPT40506","SPT40508","SPT40509","SPT40510","SPT40511","SPT40513","SPT40514","SPT40515","SPT40516","SPT40517","SPT40518","SPT40519","SPT40520","SPT40521","SPT40523","SPT40524","SPT40525","SPT40527","SPT40528","SPT40529","SPT40530","SPT40532","SPT40533","SPT40534","SPT40536","SPT40537","SPT40538","SPT40539","SPT40540","SPT40541","SPT40542","SPT40543","SPT40544","SPT40545","SPT40547","SPT40548","SPT40549","SPT40550","SPT40551","SPT40552","SPT40553","SPT40554","SPT40555","SPT40559","SPT40560","SPT40561","SPT40562","SPT40565","SPT40568","SPT40576","SPT40580","SPT40582","SPT40588","SPT40589","SPT40590","SPT40596","SPT40597","SPT40599","SPT40600","SPT40601","SPT40602","SPT40603","SPT40604","SPT40605","SPT40606","SPT40607","SPT40608","SPT40609","SPT40610","SPT40616","SPT40617","SPT40618","SPT40619","SPT40620","SPT40621","SPT40622","SPT40623","SPT40626","SPT40627","SPT40630","SPT40631","SPT40632","SPT40633","SPT40634","SPT40635","SPT40637","SPT40638","SPT40639","SPT40640","SPT40642","SPT40643","SPT40644","SPT40645","SPT40646","SPT40647","SPT40648","SPT40649","SPT40650","SPT40651","SPT40652","SPT40653","SPT40655","SPT40656","SPT40657","SPT40659","SPT40660","SPT40661","SPT40663","SPT40664","SPT40665","SPT40666","SPT40668","SPT40669","SPT40670","SPT40671","SPT40672","SPT40673","SPT40674","SPT40675","SPT40676","SPT40677","SPT40678","SPT40679","SPT40680","SPT40681","SPT40682","SPT40683","SPT40684","SPT40685","SPT40686","SPT40687","SPT40689","SPT40690","SPT40692","SPT40693","SPT40695","SPT40696","SPT40697","SPT40701","SPT40705","SPT40707","SPT40713","SPT40716","SPT40717","SPT40718","SPT40720","SPT40727","SPT40729","SPT40733","SPT40736","SPT40737","SPT40742","SPT40744","SPT40748","SPT40749","SPT40750","SPT40752","SPT40753","SPT40755","SPT40756","SPT40759","SPT40760","SPT40770","SPT40771","SPT40772","SPT40773","SPT40774","SPT40776","SPT40777","SPT40779","SPT40780","SPT40781","SPT40782","SPT40783","SPT40786","SPT40787","SPT40789","SPT40791","SPT40794","SPT40795","SPT40798","SPT40799","SPT40800","SPT40801","SPT40802","SPT40803","SPT40804","SPT40807","SPT40808","SPT40810","SPT40812","SPT40814","SPT40815","SPT40816","SPT40817","SPT40818","SPT40820","SPT40821","SPT40824","SPT40825","SPT40826","SPT40827","SPT40828","SPT40829","SPT40830","SPT40831","SPT40832","SPT40834","SPT40835","SPT40836","SPT40838","SPT40840","SPT40841","SPT40842","SPT40843","SPT40844","SPT40845","SPT40847","SPT40848","SPT40849","SPT40850","SPT40852","SPT40853","SPT40854","SPT40855","SPT40856","SPT40857","SPT40858","SPT40860","SPT40861","SPT40863","SPT40864","SPT40865","SPT40867","SPT40868","SPT40870","SPT40871","SPT40873","SPT40875","SPT40877","SPT40878","SPT40883","SPT40887","SPT40890","SPT40892","SPT40893","SPT40894","SPT40896","SPT40908","SPT40909","SPT40910","SPT40915","SPT40916","SPT40917","SPT40920","SPT40921","SPT40922","SPT40923","SPT40925","SPT40932","SPT40934","SPT40935","SPT40937","SPT40938","SPT40939","SPT40940","SPT40941","SPT40942","SPT40943","SPT40944","SPT40945","SPT40947","SPT40948","SPT40950","SPT40953","SPT40954","SPT40955","SPT40956","SPT40958","SPT40959","SPT40960","SPT40961","SPT40962","SPT40963","SPT40966","SPT40967","SPT40969","SPT40970","SPT40971","SPT40973","SPT40975","SPT40976","SPT40978","SPT40979","SPT40980","SPT40981","SPT40982","SPT40983","SPT40984","SPT40987","SPT40988","SPT40989","SPT40991","SPT40992","SPT40993","SPT40994","SPT40996","SPT40997","SPT40998","SPT40999","SPT41000","SPT41001","SPT41002","SPT41003","SPT41004","SPT41005","SPT41006","SPT41007","SPT41008","SPT41009","SPT41010","SPT41011","SPT41012","SPT41013","SPT41014","SPT41015","SPT41016","SPT41017","SPT41018","SPT41019","SPT41020","SPT41022","SPT41023","SPT41024","SPT41025","SPT41026","SPT41027","SPT41028","SPT41029","SPT41030","SPT41031","SPT41032","SPT41033","SPT41035","SPT41036","SPT41037","SPT41038","SPT41039","SPT41041","SPT41042","SPT41043","SPT41044","SPT41045","SPT41046","SPT41047","SPT41048","SPT41049","SPT41050","SPT41052","SPT41053","SPT41055","SPT41056","SPT41058","SPT41059","SPT41061","SPT41062","SPT41063","SPT41064","SPT41065","SPT41066","SPT41068","SPT41069","SPT41070","SPT41072","SPT41073","SPT41074","SPT41075","SPT41076","SPT41077","SPT41078","SPT41080","SPT41083","SPT41084","SPT41085","SPT41086","SPT41088","SPT41089","SPT41090","SPT41091","SPT41092","SPT41094","SPT41096","SPT41097","SPT41098","SPT41099","SPT41101","SPT41102","SPT41103","SPT41104","SPT41105","SPT41106","SPT41107","SPT41108","SPT41109","SPT41110","SPT41112","SPT41113","SPT41114","SPT41115","SPT41117","SPT41118","SPT41120","SPT41121","SPT41122","SPT41123","SPT41124","SPT41125","SPT41126","SPT41128","SPT41129","SPT41130","SPT41131","SPT41132","SPT41133","SPT41135","SPT41136","SPT41137","SPT41138","SPT41139","SPT41140","SPT41141","SPT41142","SPT41143","SPT41144","SPT41145","SPT41146","SPT41147","SPT41151","SPT41152","SPT41154","SPT41250","SPT41251","SPT41253","SPT41254","SPT41255","SPT41256","SPT41258","SPT41260","SPT41262","SPT41263","SPT41264","SPT41266","SPT41267","SPT41268","SPT41269","SPT41270","SPT41271","SPT41272","SPT41273","SPT41274","SPT41275","SPT41276","SPT41278","SPT41279","SPT41280","SPT41281","SPT41282","SPT41283","SPT41284","SPT41841","SPT41843","SPT41844","SPT41847","SPT41854","SPT41860","SPT41861","SPT41862","SPT41865","SPT41867","SPT41870","SPT41873","SPT41885","SPT41901","SPT41903","SPT41905","SPT41906","SPT41911","SPT41913","SPT41914","SPT41915","SPT41918","SPT41922","SPT41928","SPT41930","SPT41931","SPT41932","SPT41938","SPT41939","SPT41940","SPT41946","SPT41947","SPT41950","SPT41952","SPT41954","SPT41956","SPT41957","SPT41958","SPT41964","SPT41966","SPT41968","SPT41970","SPT41973","SPT41974","SPT41975","SPT41983","SPT41988","SPT41990","SPT41993","SPT41996","SPT41997","SPT41999","SPT42002","SPT42003","SPT42006","SPT42007","SPT42008","SPT42010","SPT42011","SPT42013","SPT42016","SPT42020","SPT42022","SPT42023","SPT42024","SPT42026","SPT42028","SPT42029","SPT42031","SPT42035","SPT42039","SPT42042","SPT42045","SPT42046","SPT42048","SPT42050","SPT42052","SPT42053","SPT42055","SPT42057","SPT42058","SPT42825","SPT42826","SPT42827","SPT42828","SPT42829","SPT42830","SPT42831","SPT42832","SPT42833","SPT42834","SPT42835","SPT42836","SPT42837","SPT42838","SPT42839","SPT42840","SPT42841","SPT42842","SPT42843","SPT42844","SPT42845","SPT42846","SPT42847","SPT42848","SPT42849","SPT42850","SPT42851","SPT42852","SPT42853","SPT42854","SPT42855","SPT42856","SPT42857","SPT42858","SPT42859","SPT42860","SPT42861","SPT42862","SPT42863","SPT42864","SPT42865","SPT42866","SPT42867","SPT42868","SPT42869","SPT42870","SPT42872","SPT42873","SPT42874","SPT42875","SPT42876","SPT42877","SPT42878","SPT42879","SPT42880","SPT42881","SPT42882","SPT42883","SPT42884","SPT42885","SPT42886","SPT42887","SPT42888","SPT42889","SPT42890","SPT42891","SPT42892","SPT42893","SPT42894","SPT42895","SPT42896","SPT42897","SPT42898","SPT42899","SPT42900","SPT42901","SPT42902","SPT42903","SPT42904","SPT42905","SPT42906","SPT42907","SPT42908","SPT42909","SPT42910","SPT42911","SPT42912","SPT42913","SPT42914","SPT42915","SPT42916","SPT42917","SPT42918","SPT42919","SPT42920","SPT42921","SPT42922","SPT42923","SPT42924","SPT42925","SPT42926","SPT42927","SPT42928","SPT42929","SPT42930","SPT42931","SPT42932","SPT42933","SPT42935","SPT42936","SPT42937","SPT42938","SPT42939","SPT42940","SPT42941","SPT42942","SPT42943","SPT42944","SPT42945","SPT42946","SPT42947","SPT42948","SPT42949","SPT42950","SPT42951","SPT42952","SPT42954","SPT42955","SPT42956","SPT42957","SPT42958","SPT42959","SPT42960","SPT42961","SPT42962","SPT42963","SPT42964","SPT42965","SPT42966","SPT42967","SPT42968","SPT42969","SPT42970","SPT42971","SPT42972","SPT42973","SPT42974","SPT42975","SPT42976","SPT42977","SPT42978","SPT42979","SPT42980","SPT42981","SPT42982","SPT42983","SPT42984","SPT42986","SPT42987","SPT42988","SPT42989","SPT42990","SPT42991","SPT42992","SPT42993","SPT42994","SPT42995","SPT42996","SPT42997","SPT42998","SPT42999","SPT43000","SPT43001","SPT43002","SPT43003","SPT43004","SPT43005","SPT43006","SPT43007","SPT43009","SPT43010","SPT43011","SPT43012","SPT43013","SPT43014","SPT43015","SPT43016","SPT43019","SPT43021","SPT43022","SPT43023","SPT43026","SPT43028","SPT43029","SPT43030","SPT43031","SPT43032","SPT43035","SPT43036","SPT43037","SPT43038","SPT43041","SPT43043","SPT43044","SPT43045","SPT43046","SPT43047","SPT43048","SPT43049","SPT43050","SPT43051","SPT43052","SPT43053","SPT43054","SPT43056","SPT43057","SPT43058","SPT43059","SPT43060","SPT43061","SPT43062","SPT43063","SPT43065","SPT43066","SPT43067","SPT43068","SPT43069","SPT43070","SPT43071","SPT43072","SPT43075","SPT43076","SPT43077","SPT43078","SPT43079","SPT43080","SPT43081","SPT43083","SPT43084","SPT43085","SPT43086","SPT43087","SPT43088","SPT43089","SPT43093","SPT43094","SPT43096","SPT43097","SPT43098","SPT43099","SPT43100","SPT43101","SPT43102","SPT43103","SPT43104","SPT43105","SPT43106","SPT43107","SPT43108","SPT43109","SPT43110","SPT43111","SPT43112","SPT43113","SPT43115","SPT43116","SPT43117","SPT43118","SPT43119","SPT43120","SPT43121","SPT43122","SPT43123","SPT43124","SPT43125","SPT43126","SPT43127","SPT43128","SPT43129","SPT43130","SPT43131","SPT43133","SPT43134","SPT43135","SPT43136","SPT43137","SPT43138","SPT43139","SPT43140","SPT43141","SPT43142","SPT43143","SPT43144","SPT43145","SPT43146","SPT43147","SPT43148","SPT43149","SPT43150","SPT43151","SPT43152","SPT43153","SPT43154","SPT43155","SPT43156","SPT43157","SPT43158","SPT43159","SPT43160","SPT43161","SPT43162","SPT43163","SPT43164","SPT43165","SPT43166","SPT43167","SPT43168","SPT43169","SPT43170","SPT43171","SPT43172","SPT43175","SPT43176","SPT43177","SPT43178","SPT43179","SPT43180","SPT43183","SPT43184","SPT43185","SPT43186","SPT43187","SPT43188","SPT43191","SPT43192","SPT43193","SPT43194","SPT43195","SPT43196","SPT43197","SPT43198","SPT43199","SPT43200","SPT43201","SPT43202","SPT43203","SPT43204","SPT43205","SPT43206","SPT43208","SPT43209","SPT43210","SPT43211","SPT43212","SPT43213","SPT43214","SPT43215","SPT43216","SPT43217","SPT43218","SPT43219","SPT43220","SPT43222","SPT43223","SPT43224","SPT43225","SPT43226","SPT43227","SPT43228","SPT43229","SPT43230","SPT43231","SPT43232","SPT43233","SPT43234","SPT43235","SPT43236","SPT43237","SPT43238","SPT43239","SPT43240","SPT43241","SPT43242","SPT43243","SPT43244","SPT43246","SPT43247","SPT43248","SPT43249","SPT43250","SPT43251","SPT43252","SPT43253","SPT43254","SPT43255","SPT43256","SPT43257","SPT43258","SPT43259","SPT43260","SPT43261","SPT43262","SPT43263","SPT43264","SPT43265","SPT43266","SPT43267","SPT43268","SPT43269","SPT43270","SPT43271","SPT43272","SPT43273","SPT43274","SPT43275","SPT43276","SPT43277","SPT43279","SPT43280","SPT43281","SPT43282","SPT43283","SPT43284","SPT43285","SPT43286","SPT43287","SPT43288","SPT43289","SPT43290","SPT43291","SPT43292","SPT43293","SPT43294","SPT43295","SPT43296","SPT43297","SPT43298","SPT43299","SPT43300","SPT43301","SPT43302","SPT43303","SPT43304","SPT43305","SPT43306","SPT43307","SPT43308","SPT43310","SPT43311","SPT43312","SPT43313","SPT43314","SPT43315","SPT43316","SPT43317","SPT43318","SPT43319","SPT43320","SPT43321","SPT43322","SPT43323","SPT43324","SPT43325","SPT43326","SPT43328","SPT43329","SPT43330","SPT43331","SPT43333","SPT43334","SPT43335","SPT43336","SPT43337","SPT43338","SPT43339","SPT43340","SPT43341","SPT43342","SPT43343","SPT43344","SPT43345","SPT43346","SPT43347","SPT43348","SPT43349","SPT43350","SPT43351","SPT43352","SPT43355","SPT43356","SPT43357","SPT43358","SPT43359","SPT43360","SPT43361","SPT43362","SPT43363","SPT43364","SPT43365","SPT43366","SPT43367","SPT43368","SPT43369","SPT43370","SPT43371","SPT43372","SPT43373","SPT43374","SPT43375","SPT43376","SPT43377","SPT43390","SPT43391"}

In [12]:
## Add the pf7 samples that pass QC:
# ltu.add_one_set(ns=namespace, 
#                 ws=workspace, 
#                 etype="sample_set",
#                 ename="Pf7_qc_passed",
#                 member_type="sample",
#                 members=qc_pass_pf7_samples,
#                 attributes=None
#                )

In [19]:
## Add the pf7 samples that pass QC in chunks:
cohort_size=1500
indices = [i for i in range(0,len(qc_pass_pf7_samples), cohort_size)]
indices.append(len(qc_pass_pf7_samples))
chunks = [(indices[i-1], indices[i]) for i in range(1,len(indices))]

qc_pass_sample_list = sorted(list(qc_pass_pf7_samples))

for i, (start, end) in enumerate(chunks):
    print(f"Adding chunk {i} - ({start}, {end})")
#     ltu.add_one_set(ns=namespace, 
#                 ws=workspace, 
#                 etype="sample_set",
#                 ename=f"Pf7_qc_passed_chunk_{i}",
#                 member_type="sample",
#                 members=qc_pass_sample_list[start:end],
#                 attributes=None
#                )

Adding chunk 0 - (0, 1500)
Adding chunk 1 - (1500, 3000)
Adding chunk 2 - (3000, 4500)
Adding chunk 3 - (4500, 6000)
Adding chunk 4 - (6000, 7500)
Adding chunk 5 - (7500, 9000)
Adding chunk 6 - (9000, 10500)
Adding chunk 7 - (10500, 12000)
Adding chunk 8 - (12000, 13500)
Adding chunk 9 - (13500, 15000)
Adding chunk 10 - (15000, 16203)


## Add `sample_set`s for the samples in the chunks of 2022 data:

In [57]:
sample_table = ltu.fetch_existing_root_table(namespace, workspace, "sample")
sample_table = fix_nans(sample_table)

sample_set_table = ltu.fetch_existing_root_table(namespace, workspace, "sample_set")
sample_set_table = fix_nans(sample_set_table)

In [17]:
samples_2022 = sample_table[sample_table["dataset_provenance"] == 'Broad 2022 Senegal Dataset']
samples_2022_new = samples_2022[samples_2022["aligned_bai"] == ""]

In [26]:
samples_2022_new.loc[:, "input_bam"] = samples_2022_new["unaligned_bam"]
samples_2022_new.loc[:, "input_bai"] = samples_2022_new["unaligned_bai"]

In [32]:
new_samples = set(samples_2022_new['sample'])
new_samples = set([s for s in new_samples if '2022' in s])
new_samples

In [67]:
# saved_new_samples = new_samples
list(samples_2022_new["input_bai"])

['gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN.PD1_0213.bam.bai',
 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN.PD1_0237.bam.bai',
 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN.PD2_0151.bam.bai',
 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN.PD2_0169.bam.bai',
 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN.PD2_0170.bam.bai',
 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN.PD2_0172.bam.bai',
 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN.PD2_0173.bam.bai',
 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN.PD2_0174.bam.bai',
 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN.PD2_0177.bam.bai',
 'gs://broad-dsp-lrma-sr-malaria/falciparum/senegal/2022_samples/SEN_2022_BAN.PD2_0179.bam.bai',
 'gs://broad-dsp-lrma-sr-malar

In [66]:
samples_2022_new.loc[:, "input_bai"] = samples_2022_new["input_bam"] + ".bai"

In [59]:
samples_2022_new = sample_table[sample_table["sample"].isin(new_samples)]
samples_2022_new

,sample,ENA,additional_metadata,aligned_bai,aligned_bam,aligned_est_fold_cov,aligned_frac_bases,aligned_num_bases,aligned_num_reads,aligned_read_length,...,qc_pass,raw_est_fold_cov,read_length,read_qual_mean,read_qual_median,sample_type,study,unaligned_bai,unaligned_bam,year
22327,SEN_2022_BAN.PD1_0213,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22328,SEN_2022_BAN.PD1_0237,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22329,SEN_2022_BAN.PD2_0151,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22330,SEN_2022_BAN.PD2_0169,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22331,SEN_2022_BAN.PD2_0170,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22679,SEN_2022_SES.PD3_0174,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22680,SEN_2022_SES.PD3_0180,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22681,SEN_2022_SES.PD3_0181,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22682,SEN_2022_SES.PD3_0182,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022


In [51]:
# ltu.add_one_set(ns=namespace, 
#             ws=workspace, 
#             etype="sample_set",
#             ename=f"Broad_2022_Senegal_Dataset_20231205",
#             member_type="sample",
#             members=new_samples,
#             attributes=None
#            )

In [68]:
df_for_upload = samples_2022_new
df_for_upload = df_for_upload.rename(columns={"sample": "entity:sample_id"})
df_for_upload

,entity:sample_id,ENA,additional_metadata,aligned_bai,aligned_bam,aligned_est_fold_cov,aligned_frac_bases,aligned_num_bases,aligned_num_reads,aligned_read_length,...,qc_pass,raw_est_fold_cov,read_length,read_qual_mean,read_qual_median,sample_type,study,unaligned_bai,unaligned_bam,year
22327,SEN_2022_BAN.PD1_0213,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22328,SEN_2022_BAN.PD1_0237,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22329,SEN_2022_BAN.PD2_0151,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22330,SEN_2022_BAN.PD2_0169,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22331,SEN_2022_BAN.PD2_0170,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22679,SEN_2022_SES.PD3_0174,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22680,SEN_2022_SES.PD3_0180,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22681,SEN_2022_SES.PD3_0181,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022
22682,SEN_2022_SES.PD3_0182,,,,,,,,,,...,,,,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,2022


In [69]:
# ltu.upload_root_table(namespace, workspace, df_for_upload)

## Add `sample_set_set`s for the samples in the chunks of PF7

In [60]:
sample_table = ltu.fetch_existing_root_table(namespace, workspace, "sample")
sample_table = fix_nans(sample_table)

t = ltu.fetch_existing_root_table(namespace, workspace, "sample_set")
t = fix_nans(t)

In [28]:
# get sample sets of the chunks of PF7:
chunks = []
for s in t["sample_set"]:
    if "chunk" in s:
        chunks.append(s)
chunks

['Pf7_qc_passed_chunk_0',
 'Pf7_qc_passed_chunk_1',
 'Pf7_qc_passed_chunk_2',
 'Pf7_qc_passed_chunk_3',
 'Pf7_qc_passed_chunk_4',
 'Pf7_qc_passed_chunk_5',
 'Pf7_qc_passed_chunk_6',
 'Pf7_qc_passed_chunk_7',
 'Pf7_qc_passed_chunk_8',
 'Pf7_qc_passed_chunk_9',
 'Pf7_qc_passed_chunk_10']

In [66]:
# Not all samples in each chunk passed processing. 
# Let's filter out the bad ones:
chunk0_failed_samples = {"PD0115-Cx", "PD0116-Cx", "PD0136-C", "PD0137-C", "PD0140-C", "PD0141-C", "PD0142-C"}
chunk1_failed_samples = {"PF0208-C", "PF0214-C", "PF0248-C", "PF0249-C", "PF0252-C", "PF0254-C", "PF0258-C", "PF0260-C", "PF0269-C", "PF0281-C", "PF0289-C", "PG0607-C", "PG0608-C", "PG0609-C", "PG0610-C", "PG0611-C", "PH0126-CW", "PH0294-C"}
chunk2_failed_samples = {"PJ0129-Cx", "PJ0130-Cx", "PJ0133-C", "PJ0134-C", "PJ0135-Cx", "PJ0136-Cx", "PJ0137-Cx", "PJ0138-Cx", "PJ0140-C", "PJ0144-Cx", "PJ0146-Cx", "PJ0152-Cx", "PJ0156-C", "PJ0160-C", "PJ0162-C", "PJ0163-C", "PJ0164-C", "PJ0168-C", "PJ0170-C", "PJ0171-C", "PJ0172-C", "PJ0173-C", "PJ0174-C", "PJ0175-C", "PJ0176-C", "PJ0177-C", "PJ0178-C", "PJ0179-C", "PJ0180-C", "PJ0181-C", "PJ0183-C", "PJ0186-C", "PJ0189-C", "PJ0190-C", "PJ0192-C", "PJ0193-C", "PJ0194-C", "PJ0195-C", "PJ0196-C", "PJ0197-C", "PJ0198-C", "PJ0199-C", "PJ0200-C", "PJ0205-C", "PJ0206-C", "PJ0208-C", "PJ0209-C", "PJ0210-C", "PJ0222-C", "PJ0223-C", "PJ0225-C", "PJ0230-C", "PJ0233-C", "PJ0234-C", "PJ0237-C", "PJ0238-C", "PJ0239-C", "PJ0240-C", "PJ0241-C", "PJ0242-C", "PJ0243-C", "PJ0244-C", "PJ0245-C", "PJ0246-C", "PJ0247-C", "PJ0251-C", "PJ0252-C", "PJ0258-C", "PJ0259-C", "PJ0262-C", "PJ0263-C", "PJ0264-C", "PJ0265-C", "PJ0266-C", "PJ0267-C", "PJ0268-C", "PJ0269-C", "PJ0270-C", "PJ0271-C", "PJ0272-C", "PM0805-C", "PM0808-C", "PM0811-C"}
chunk3_failed_samples = set()

failed_participants = set()
for l in [chunk0_failed_samples, chunk1_failed_samples, chunk2_failed_samples, chunk3_failed_samples]:
    failed_participants.update(sample_table[sample_table["sample"].isin(l)]["participant"])
print(f"Num failed participants: {len(failed_participants)}")

Num failed participants: 108


In [75]:
for c, chunk_failed_samples in zip([0, 1, 2, 3], [chunk0_failed_samples, chunk1_failed_samples, chunk2_failed_samples, chunk3_failed_samples]):

    chunk_name = f"Pf7_qc_passed_chunk_{c}"
    
    chunk_samples_dict = next(iter(t[t["sample_set"] == chunk_name]["samples"]))
    chunk_sample_names = {s["entityName"] for s in chunk_samples_dict["items"]}
    chunk_participant_names = set([next(iter(sample_table[sample_table["sample"] == s]["participant"])) for s in chunk_sample_names])
    
    chunk_passed_participants = chunk_participant_names - failed_participants
    
    # We know that there were 1500 samples in each chunk:
    print(f"Chunk: {c}")
    print(f"Failed participants: {len(chunk_participant_names) - len(chunk_passed_participants)}")
    print(f"Passed participants: {len(chunk_passed_participants)}")
    print()
    
    print(f"Adding Chunk {c} passed participants...", end="\t")
#     ltu.add_one_set(ns=namespace, 
#                 ws=workspace, 
#                 etype="sample_set_set",
#                 ename=f"Pf7_qc_passed_chunk_{c}_passed",
#                 member_type="sample_set",
#                 members=chunk_passed_participants,
#                 attributes=None
#                )
    print("Done")


Chunk: 0
Failed participants: 7
Passed participants: 1493

Adding Chunk 0 passed participants...	Done
Chunk: 1
Failed participants: 18
Passed participants: 1482

Adding Chunk 1 passed participants...	Done
Chunk: 2
Failed participants: 83
Passed participants: 1417

Adding Chunk 2 passed participants...	Done
Chunk: 3
Failed participants: 0
Passed participants: 1500

Adding Chunk 3 passed participants...	Done


## Add new `sample_set_set` based on other `sample_set_set`s

In [79]:
t = ltu.fetch_existing_root_table(namespace, workspace, "sample_set_set")
t = fix_nans(t)

In [87]:
sets_to_combine = {"Broad_2022_Senegal_Dataset_2", "Broad_2019_Senegal_Dataset_With_VCFs_2"}

samples_in_combined_set = set()

for s in sets_to_combine:
    samples_dict = next(iter(t[t["sample_set_set"] == s]["sample_sets"]))
    sample_set_names = {s["entityName"] for s in samples_dict["items"]}
    samples_in_combined_set.update(sample_set_names)
print(f"New set will contain {len(samples_in_combined_set)} sample sets.")

New set will contain 1315 sample sets.


In [88]:
# ltu.add_one_set(ns=namespace, 
#             ws=workspace, 
#             etype="sample_set_set",
#             ename=f"Broad_2019_2022_Senegal_Dataset",
#             member_type="sample_set",
#             members=samples_in_combined_set,
#             attributes=None
#            )

In [ ]:
raise Exception("DO NOT RUN ALL CELLS!")